## Introduction ##


This is a very basic Kernel on how to read in the data, and correct weight each record with the **PWGTP** field.


Because this is a very large Dataset, you'll probably want to load only the columns you plan to use.  The Data Dictionary can be found at the bottom of this Kernel.

There's also the [PUMSDataDict15][2]; but, you'll need to skip down to the PERSON's section, since it starts with housing data, which isn't included in this Dataset.


  [1]: http://#dataDictionary
  [2]: http://www2.census.gov/programs-surveys/acs/tech_docs/pums/data_dict/PUMSDataDict15.txt

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

sns.set(style="white", color_codes=True)

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# ST,    STATE
# SEX    1 Make, 2 Female
# PWGTP  (Weighting of record. Very important)
# AGEP   Age
# MSP    Married, spouse present/spouse absent
# FOD1P  Recoded field of degree - first entry (College)
# PERNP  Total person's earnings
# ADJINC Use ADJINC to adjust PERNP to constant dollars.
# SCHL   Highest education allocation flag
# SCH    School enrollment
# ESR    Employment status recode
# WKHP   Usual hours worked per week past 12 months allocation flag
# RAC1P  Recoded detailed race code 1) White  2) Black ... see below
# ACCESS http://www2.census.gov/programs-surveys/acs/tech_docs/pums/data_dict/PUMSDataDict15.pdf
             

# Careful -- you don't want to select every column in the Dataset. That's too much.    
fields = ["PUMA", "ST", "SEX","PWGTP","AGEP", "MSP","FOD1P","PERNP","ADJINC","SCHL","ESR","WKHP","RAC1P","ACCESS", "FIBEROP", "ADJHSG", "MODEM", "OTHSVCEX", "SATELLITE", "RESMODE", "FACCESSP", "FFIBEROPP", "FMODEMP", "FOTHSVCEXP", "FSATELLITEP"]

# Two files...each file is about 1.5G in size.
data_a=pd.read_csv("../input/ss15pusa.csv",skipinitialspace=True, usecols=fields)
data_b=pd.read_csv("../input/ss15pusb.csv",skipinitialspace=True, usecols=fields)

In [ ]:
# Concatinate -- we'll work with everything.
d=pd.concat([data_a,data_b])

# Get the correct income in 2015 dollars
d['INCOME']=d['ADJINC']*d['PERNP']/1000000

# Take a look
d.head()

In [ ]:
# We don't need these
del data_a, data_b

## PWGTP (Weights) Must Be Used ##

This data is weighted.  For example, if you wanted to find the total number
of people age 30, you'd have to sum the weights.  Each record in the set
may represent multiple people.  We'll prove this at the end, when you can 
see the total population size.

In [ ]:
# For example how many males are in the US, age 30?
# SEX:  Male = 1,  Female = 2
d[(d['AGEP']==30) & (d['SEX'] == 1)]['PWGTP'].sum()

In [ ]:
# Quick Check
# How many people in the US, age 0 to 99?
#   Note, AGEP only goes to 99.

# The number should be around 320 million... This
# is just to demonstrate the importance of using PWGTP in your
# calculations.

total_population = d[(d['AGEP']>=0) & (d['AGEP']<100)]['PWGTP'].sum()

# This is just for displaying ','
"{:,}".format(total_population)

In [ ]:
# Okay, since I'm curious, how many people 90 or older?
total_population = d[(d['AGEP']>=90)]['PWGTP'].sum()
"{:,}".format(total_population)

RAC1P
-----
<pre>       
        Recoded detailed race code
                1 .White alone
                2 .Black or African American alone
                3 .American Indian alone
                4 .Alaska Native alone
                5 .American Indian and Alaska Native tribes specified; or American
                  .Indian or Alaska Native, not specified and no other races
                6 .Asian alone
                7 .Native Hawaiian and Other Pacific Islander alone
                8 .Some Other Race alone
                9 .Two or More Races
</pre>

----------

In [ ]:
# Graph the population by Race. Only take Males

white=[]
black=[]
other=[]

for i in range(0,100):
    white.append(d[(d['AGEP']==i) & (d['SEX']==1) & (d['RAC1P']==1)]['PWGTP'].sum())
    black.append(d[(d['AGEP']==i) & (d['SEX']==1) & (d['RAC1P']==2)]['PWGTP'].sum())
    other.append(d[(d['AGEP']==i) & (d['SEX']==1) & (d['RAC1P'] >2)]['PWGTP'].sum())

In [ ]:
from matplotlib.ticker import FuncFormatter

fig, ax = plt.subplots()
ax.spines["top"].set_visible(False)    
ax.spines["bottom"].set_visible(False)    
ax.spines["right"].set_visible(False)    
ax.spines["left"].set_visible(False) 

ax.plot(white,color = 'blue')
ax.plot(black,color = 'black')
ax.plot(other,color = 'green')
ax.set_title("2015 US Population\n(Age and Race)", fontsize=12, color='darkslateblue')
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:,}'.format(int(y))))

ax.grid(b=True, which='major', color='wheat', linestyle='-')

plt.tick_params(axis="both", which="both", bottom="off", top="off",    
                labelbottom="on", left="off", right="off", labelleft="on")    
 
plt.text(71, 1400000, "White Males", fontsize=10, ha="center",color="blue")   
plt.text(60, 351900, "Black Males", fontsize=10, ha="center",color="black") 
plt.text(30, 431900, "Other Males", fontsize=10, ha="center",color="green") 
plt.show()

In [ ]:
# Graph the population by Race. Female.

whiteF=[]
blackF=[]
otherF=[]

for i in range(0,100):
    whiteF.append(d[(d['AGEP']==i) & (d['SEX']==2) & (d['RAC1P']==1)]['PWGTP'].sum())
    blackF.append(d[(d['AGEP']==i) & (d['SEX']==2) & (d['RAC1P']==2)]['PWGTP'].sum())
    otherF.append(d[(d['AGEP']==i) & (d['SEX']==2) & (d['RAC1P'] >2)]['PWGTP'].sum())

In [ ]:
from matplotlib.ticker import FuncFormatter

fig, ax = plt.subplots()
ax.spines["top"].set_visible(False)    
ax.spines["bottom"].set_visible(False)    
ax.spines["right"].set_visible(False)    
ax.spines["left"].set_visible(False) 

ax.plot(white,color = 'blue')
ax.plot(black,color = 'black')
ax.plot(other,color = 'green')

ax.plot(whiteF,color = 'turquoise')
ax.plot(blackF,color = 'grey')
ax.plot(otherF,color = 'green')


ax.set_title("2015 US Population\n(Age and Race)", fontsize=12, color='darkslateblue')
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:,}'.format(int(y))))

ax.grid(b=True, which='major', color='wheat', linestyle='-')

plt.tick_params(axis="both", which="both", bottom="off", top="off",    
                labelbottom="on", left="off", right="off", labelleft="on")    
 
plt.text(55, 1400000, "White Male", fontsize=10, ha="center",color="blue")   
plt.text(79, 1500000, "White Female", fontsize=10, ha="center",color="turquoise")

plt.text(40, 150900, "Black Male", fontsize=10, ha="center",color="black") 
plt.text(60, 350900, "Black Female", fontsize=10, ha="center",color="grey") 



plt.text(30, 431900, "Other", fontsize=10, ha="center",color="green") 
plt.show()

In [ ]:
# Clean up
del white,black,other,whiteF,blackF

Income Distribution
-------------------

Because of the weighting, PWGTP show approximate the number of real people in the
US for a single row, we'll need to expand the Dataframe to the true number.  A numpy
array will be preallocated, then, records will be inserted.

In [ ]:
k=d[(d['AGEP']>=20) & (d['AGEP']<=60) & (d['SEX']==1) & (d['RAC1P']==1)][['PWGTP','AGEP','SCH','INCOME']]
numberOfRows=k['PWGTP'].sum()

I=0
A=np.zeros((numberOfRows+1,3))
def f(t):
    global A,I
    z=[t[1],t[2],t[3]]
    idx= int(t[0])
    
    for i in range(0,idx):
        A[I]=z
        I+=1
    
k.apply(f,axis=1);
df = pd.DataFrame(A,columns = ['AGEP','SCH','INCOME'])

In [ ]:



plt.style.use('fivethirtyeight')

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 10
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 12


a=df[(df['AGEP']>=20) &(df['AGEP']<30)]['INCOME']
b=df[(df['AGEP']>=30) &(df['AGEP']<40)]['INCOME']
c=df[(df['AGEP']>=40) &(df['AGEP']<55)]['INCOME']
sns.distplot(a,bins=100,  hist=False,  label="20 to 30 (Age in years)"  );
sns.distplot(b,bins=100,  hist=False,  label="30 to 40"  );
g=sns.distplot(c,bins=100,  hist=False,  label="40 to 55"  );
#ax.xaxis.set_major_formatter(FuncFormatter(lambda x, _: '{:,}'.format(int(x))))
sns.despine()
g.tick_params(labelsize=12,labelcolor="black")
#ax.xaxis.set_major_formatter(tck.FormatStrFormatter('{:,}'))
#x.xaxis.set_major_formatter(x_format)
plt.xlim(0, 200000)
plt.xticks([50000,100000,150000,200000], ['$50K', '$100K', '$120K', '$150K','$200K'])
plt.title("Income Distribution\nby Age, White Males", fontname='Ubuntu', fontsize=14,
            fontstyle='italic', fontweight='bold')

plt.legend();

Data Dictionary
---------------


<a id='dataDictionary'>PERSON RECORD</a>
<pre>
RT		1	
	Record Type		
		P .Person Record## Heading ##

SERIALNO   	7	
	Housing unit/GQ person serial number	
		0000001..9999999 .Unique identifier

SPORDER		2	
	Person number
		01..20 .Person number

PUMA		5	
	Public use microdata area code (PUMA) based on 2010 Census definition	
		00100..70301 .Public use microdata area codes
				 
Note: Public use microdata areas (PUMAs) designate areas of 100,000 or more population.  Use with ST for unique code.

ST		2	
	State Code
		01 .Alabama/AL		
		02 .Alaska/AK		
		04 .Arizona/AZ		
		05 .Arkansas/AR		
		06 .California/CA		
		08 .Colorado/CO		
		09 .Connecticut/CT		
		10 .Delaware/DE		
		11 .District of Columbia/DC		
		12 .Florida/FL		
		13 .Georgia/GA		
		15 .Hawaii/HI		
		16 .Idaho/ID		
		17 .Illinois/IL		
		18 .Indiana/IN		
		19 .Iowa/IA		
		20 .Kansas/KS		
		21 .Kentucky/KY		
		22 .Louisiana/LA		
		23 .Maine/ME		
		24 .Maryland/MD		
		25 .Massachusetts/MA		
		26 .Michigan/MI		
		27 .Minnesota/MN		
		28 .Mississippi/MS		
		29 .Missouri/MO		
		30 .Montana/MT		
		31 .Nebraska/NE		
		32 .Nevada/NV		
		33 .New Hampshire/NH		
		34 .New Jersey/NJ		
		35 .New Mexico/NM		
		36 .New York/NY		
		37 .North Carolina/NC		
		38 .North Dakota/ND
		39 .Ohio/OH		
		40 .Oklahoma/OK		
		41 .Oregon/OR		
		42 .Pennsylvania/PA		
		44 .Rhode Island/RI		
		45 .South Carolina/SC		
		46 .South Dakota/SD		
		47 .Tennessee/TN		
		48 .Texas/TX		
		49 .Utah/UT		
		50 .Vermont/VT		
		51 .Virginia/VA		
		53 .Washington/WA		
		54 .West Virginia/WV		
		55 .Wisconsin/WI		
		56 .Wyoming/WY		
		72 .Puerto Rico/PR

ADJINC		7	
Adjustment factor for income and earnings dollar amounts (6 implied decimal places)
		1001264 .2015 factor (1.001264)

Note: The value of ADJINC inflation-adjusts reported income to 2015 dollars.  ADJINC applies to variables FINCP and HINCP in the housing record, and variables INTP, OIP, PAP, PERNP, PINCP, RETP, SEMP, SSIP, SSP, and WAGP in the person record. 

PWGTP		5	
	Person's weight   
		00001..09999 .Integer weight of person

AGEP		2	
	Age
		00 	.Under 1 year	
		01..99  .1 to 99 years (Top-coded***)

CIT		1	
	Citizenship status
		1 .Born in the U.S.
		2 .Born in Puerto Rico, Guam, the U.S. Virgin Islands,
		  .or the Northern Marianas
		3 .Born abroad of American parent(s)
		4 .U.S. citizen by naturalization
		5 .Not a citizen of the U.S.

CITWP		4	
	Year of naturalization write-in
		bbbb .Not eligible – not naturalized		
		1928 .1928 or earlier (Bottom-coded)
		1929 .1929 – 1933
		1934 .1934 – 1939
		1940 .1940 - 1942
		1943 .1943 - 1944
		1945 .1945
		1946 .1946 – 1947
		1948 .1948
		1949 .1949
		1950 .1950
		1951 .1951
		1952 .1952
		1953 .1953
		1954 .1954
		1955 .1955
		1956 .1956
		1957 .1957
		1958 .1958
		1959 .1959
		1960 .1960
		1961 .1961
		1962 .1962
		1963 .1963
		1964 .1964
		1965 .1965
		1966 .1966
		1967 .1967
		1968 .1968
		1969 .1969
		1970 .1970
		1971 .1971
		1972 .1972
		1973 .1973
		1974 .1974
		1975 .1975
		1976 .1976
		1977 .1977
		1978 .1978
		1979 .1979
		1980 .1980
		1981 .1981
		1982 .1982
		1983 .1983
		1984 .1984
		1985 .1985
		1986 .1986
		1987 .1987
		1988 .1988
		1989 .1989
		1990 .1990
		1991 .1991
		1992 .1992
		1993 .1993
		1994 .1994
		1995 .1995
		1996 .1996
		1997 .1997
		1998 .1998
		1999 .1999
		2000 .2000
		2001 .2001
		2002 .2002
		2003 .2003
		2004 .2004
		2005 .2005
		2006 .2006
		2007 .2007
		2008 .2008
		2009 .2009
		2010 .2010
		2011 .2011
		2012 .2012
		2013 .2013
		2014 .2014
		2015 .2015

COW		1	
	Class of worker
		b .N/A (less than 16 years old/NILF who last
		  .worked more than 5 years ago or never worked)
		1 .Employee of a private for-profit company or
		  .business, or of an individual, for wages,
		  .salary, or commissions
		2 .Employee of a private not-for-profit,
		  .tax-exempt, or charitable organization
		3 .Local government employee (city, county, etc.)
		4 .State government employee
		5 .Federal government employee
		6 .Self-employed in own not incorporated
		  .business, professional practice, or farm
		7 .Self-employed in own incorporated
		  .business, professional practice or farm
		8 .Working without pay in family business or farm
		9 .Unemployed and last worked 5 years ago or earlier or never
                  .worked
   
DDRS		1	
	Self-care difficulty
		b .N/A (Less than 5 years old)
		1 .Yes
		2 .No

DEAR		1
	Hearing difficulty
		1 .Yes
		2 .No

DEYE		1	
	Vision difficulty
		1 .Yes
		2 .No

DOUT		1	
	Independent living difficulty
		b .N/A (Less than 15 years old)
		1 .Yes
		2 .No

DPHY		1	
	Ambulatory difficulty
		b .N/A (Less than 5 years old)
		1 .Yes
		2 .No

DRAT		1
	Veteran service connected disability rating (percentage)
		b .N/A (No service-connected disability/never served in military) 
		1 .0 percent
		2 .10 or 20 percent
		3 .30 or 40 percent
		4 .50 or 60 percent
		5 .70, 80, 90, or 100 percent
		6 .Not reported

DRATX		1
	Veteran service connected disability rating (checkbox)
		b .N/A (Less than 17 years old/never served in military) 
		1 .Yes
		2 .No

DREM		1	
	Cognitive difficulty 
		b .N/A (Less than 5 years old)
		1 .Yes
		2 .No

ENG		1	
	Ability to speak English
		b .N/A (less than 5 years old/speaks only English)
		1 .Very well
		2 .Well
		3 .Not well
		4 .Not at all

FER		1	
	Gave birth to child within the past 12 months
		b .N/A (less than 15 years/greater than 50 years/
		  .male)
		1 .Yes	
		2 .No

GCL		1	
	Grandparents living with grandchildren 
		b .N/A (less than 30 years/institutional GQ)
		1 .Yes	
		2 .No		

GCM		1	
	Length of time responsible for grandchildren
		b .N/A (less than 30 years/grandparent not responsible for
		  .grandchild/institutional GQ)
		1 .Less than 6 months
		2 .6 to 11 months
		3 .1 or 2 years
		4 .3 or 4 years
		5 .5 or more years

GCR		1	
	Grandparents responsible for grandchildren	
       		b .N/A (less than 30 years/institutional GQ/grandparent not 
        	  .living with grandchild)
		1 .Yes	
		2 .No   

HINS1		1
	Insurance through a current or former employer or union
		1 .Yes
		2 .No

HINS2		1	
	Insurance purchased directly from an insurance company
		1 .Yes
		2 .No

HINS3		1	
	Medicare, for people 65 and older, or people with certain disabilities
		1 .Yes
		2 .No

HINS4		1	
	Medicaid, Medical Assistance, or any kind of government-assistance plan 
	for those with low incomes or a disability
		1 .Yes
		2 .No

HINS5		1	
	TRICARE or other military health care
		1 .Yes
		2 .No

HINS6		1	
	VA (including those who have ever used or enrolled for VA health care)
		1 .Yes
		2 .No

HINS7		1	
	Indian Health Service
		1 .Yes
		2 .No 

INTP		6	
	Interest, dividends, and net rental income past 12 months (signed)
		bbbbbb		.N/A (less than 15 years old)
		000000 		.None
		-09999..-00001	.Loss of $1 to $9999 (Rounded and bottom-coded)
		000001 		.$1 or break even
		000002..999999 	.$2 to $999999 (Rounded and top-coded)

Note: Use ADJINC to adjust INTP to constant dollars.

JWMNP		3	
	Travel time to work
		bbb 		.N/A (not a worker or worker who worked at 
				.home)
		001..200 	.1 to 200 minutes to get to work (Top-coded)

JWRIP		2	
	Vehicle occupancy
		bb .N/A (not a worker or worker whose means of
		   .transportation to work was not car, truck,
		   .or van)
		01 .Drove alone
		02 .In 2-person carpool
		03 .In 3-person carpool
		04 .In 4-person carpool
		05 .In 5-person carpool
		06 .In 6-person carpool
		07 .In 7-person carpool
		08 .In 8-person carpool
		09 .In 9-person carpool
		10 .In 10-person or more carpool (Top-coded)
	
JWTR		2	
	Means of transportation to work
		bb .N/A (not a worker--not in the labor force,
		   .including persons under 16 years; unemployed;
		   .employed, with a job but not at work; Armed
		   .Forces, with a job but not at work)
		01 .Car, truck, or van
		02 .Bus or trolley bus
		03 .Streetcar or trolley car (carro publico in Puerto Rico)
		04 .Subway or elevated
		05 .Railroad
		06 .Ferryboat		
		07 .Taxicab
		08 .Motorcycle
		09 .Bicycle
		10 .Walked
		11 .Worked at home
		12 .Other method

LANX		1	
	Language other than English spoken at home
		b .N/A (less than 5 years old)
		1 .Yes, speaks another language
		2 .No, speaks only English

MAR		1	
	Marital status
		1 .Married
		2 .Widowed
		3 .Divorced
		4 .Separated
		5 .Never married or under 15 years old

MARHD		1
	Divorced in the past 12 months
		b .N/A (age less than 15 years; never married)
		1 .Yes
		2 .No

MARHM		1
	Married in the past 12 months
		b .N/A (age less than 15 years; never married)
		1 .Yes
		2 .No

MARHT		1
	Number of times married
		b .N/A (age less than 15 years; never married)
		1 .One time
		2 .Two times
		3 .Three or more times

MARHW		1
	Widowed in the past 12 months
		b .N/A (age less than 15 years; never married)
		1 .Yes
		2 .No

MARHYP		4
	Year last married
		bbbb .N/A (age less than 15 years; never married)
		1932 .1932 or earlier (Bottom-coded)
		1933 .1933
		1934 .1934
		1935 .1935
		1936 .1936
		1937 .1937
		1938 .1938
		1939 .1939
		1940 .1940
		1941 .1941
		1942 .1942
		1943 .1943
		1944 .1944
		1945 .1945
		1946 .1946
		1947 .1947
		1948 .1948
		1949 .1949
		1950 .1950
		1951 .1951
		1952 .1952
		1953 .1953
		1954 .1954
		1955 .1955
		1956 .1956
		1957 .1957
		1958 .1958
		1959 .1959
		1960 .1960
		1961 .1961
		1962 .1962
		1963 .1963
		1964 .1964
		1965 .1965
		1966 .1966
		1967 .1967
		1968 .1968
		1969 .1969
		1970 .1970
		1971 .1971
		1972 .1972
		1973 .1973
		1974 .1974
		1975 .1975
		1976 .1976
		1977 .1977
		1978 .1978
		1979 .1979
		1980 .1980
		1981 .1981
		1982 .1982
		1983 .1983
		1984 .1984
		1985 .1985
		1986 .1986
		1987 .1987
		1988 .1988
		1989 .1989
		1990 .1990
		1991 .1991
		1992 .1992
		1993 .1993
		1994 .1994
		1995 .1995
		1996 .1996
		1997 .1997
		1998 .1998
		1999 .1999
		2000 .2000
		2001 .2001
		2002 .2002
		2003 .2003
		2004 .2004
		2005 .2005
		2006 .2006
		2007 .2007
		2008 .2008
		2009 .2009
		2010 .2010
		2011 .2011
		2012 .2012
		2013 .2013
		2014 .2014
		2015 .2015

MIG		1	
	Mobility status (lived here 1 year ago)
		b .N/A (less than 1 year old)
		1 .Yes, same house (nonmovers)
		2 .No, outside US and Puerto Rico
		3 .No, different house in US or Puerto Rico

MIL		1	
	Military service
		b .N/A (less than 17 years old)
		1 .Now on active duty
		2 .On active duty in the past, but not now
		3 .Only on active duty for training in Reserves/National Guard 
		4 .Never served in the military

MLPA		1	
	Served September 2001 or later
		b .N/A (less than 17 years old/no active duty)
		0 .Did not serve this period		
		1 .Served this period

MLPB		1	
	Served August 1990 - August 2001 (including Persian Gulf War)
		b .N/A (less than 17 years old/no active duty)
		0 .Did not serve this period		
		1 .Served this period   

MLPCD		1	
	Served May 1975 - July 1990
		b .N/A (less than 17 years old/no active duty)
		0 .Did not serve this period		
		1 .Served this period

MLPE		1	
	Served Vietnam era (August 1964 - April 1975)
		b .N/A (less than 17 years old/no active duty)
		0 .Did not serve this period		
		1 .Served this period

MLPFG		1	
	Served February 1955 - July 1964
		b .N/A (less than 17 years old/no active duty)
		0 .Did not serve this period		
		1 .Served this period

MLPH		1	
	Served Korean War (July 1950 - January 1955)
		b .N/A (less than 17 years old/no active duty)
		0 .Did not serve this period		
		1 .Served this period

MLPI		1	
	Served January 1947 - June 1950
		b .N/A (less than 17 years old/no active duty)
		0 .Did not serve this period		
		1 .Served this period

MLPJ		1	
	Served World War II (December 1941 - December 1946)
		b .N/A (less than 17 years old/no active duty)
		0 .Did not serve this period
		1 .Served this period

MLPK		1	
	Served November 1941 or earlier
		b .N/A (less than 17 years old/no active duty)
		0 .Did not serve this period		
		1 .Served this period

NWAB		1 (UNEDITED - See "Employment Status Recode" (ESR))
	Temporary absence from work 
		b .N/A (less than 16 years old/at work/on layoff)
		1 .Yes
		2 .No
		3 .Did not report

NWAV		1 (UNEDITED - See "Employment Status Recode" (ESR))	
	Available for work
		b .N/A (less than 16 years/at work/not looking)
		1 .Yes
		2 .No, temporarily ill
		3 .No, other reasons
		4 .No, unspecified
		5 .Did not report

NWLA		1 (UNEDITED - See "Employment Status Recode" (ESR))		
	On layoff from work
		b .N/A (less than 16 years old/at work)	
		1 .Yes		
		2 .No
		3 .Did not report

NWLK		1 (UNEDITED - See “Employment Status Recode” (ESR))   
	Looking for work
		b .N/A (less than 16 years old/at work/temporarily
		  .absent/informed of recall)
		1 .Yes
		2 .No
		3 .Did not report

NWRE		1 (UNEDITED - See “Employment Status Recode” (ESR))  
	Informed of recall	
		b .N/A (less than 16 years old/at work/not on layoff)
		1 .Yes
		2 .No
		3 .Did not report

OIP		6	
	All other income past 12 months
		bbbbbb 		.N/A (less than 15 years old)
		000000 		.None
		000001..999999 	.$1 to $999999 (Rounded and top-coded)

Note: Use ADJINC to adjust OIP to constant dollars.

PAP		5	
	Public assistance income past 12 months
		bbbbb 		.N/A (less than 15 years old)
		00000 		.None

		00001..99999	.$1 to $99999 (Rounded)
 
Note: Use ADJINC to adjust PAP to constant dollars.

RELP		2	
	Relationship
		00 .Reference person
		01 .Husband/wife
		02 .Biological son or daughter
		03 .Adopted son or daughter
		04 .Stepson or stepdaughter   
		05 .Brother or sister
		06 .Father or mother
		07 .Grandchild
		08 .Parent-in-law
		09 .Son-in-law or daughter-in-law
		10 .Other relative
		11 .Roomer or boarder
		12 .Housemate or roommate
		13 .Unmarried partner
		14 .Foster child
		15 .Other nonrelative
		16 .Institutionalized group quarters population
		17 .Noninstitutionalized group quarters population

RETP		6	
	Retirement income past 12 months
		bbbbbb 		.N/A (less than 15 years old)
		000000 		.None
		000001..999999	.$1 to $999999 (Rounded and top-coded)

Note: Use ADJINC to adjust RETP to constant dollars.

SCH		1	
	School enrollment
		b .N/A (less than 3 years old)
		1 .No, has not attended in the last 3 months
		2 .Yes, public school or public college	
		3 .Yes, private school or college or home school

SCHG		2
	Grade level attending
		bb .N/A (not attending school)
		01 .Nursery school/preschool
		02 .Kindergarten
		03 .Grade 1
		04 .Grade 2
		05 .Grade 3
		06 .Grade 4
		07 .Grade 5
		08 .Grade 6
		09 .Grade 7
		10 .Grade 8
		11 .Grade 9
		12 .Grade 10
		13 .Grade 11
		14 .Grade 12
		15 .College undergraduate years (freshman to senior)
		16 .Graduate or professional school beyond a bachelor's degree

SCHL		2	
	Educational attainment
		bb .N/A (less than 3 years old)
		01 .No schooling completed
		02 .Nursery school, preschool   
		03 .Kindergarten
		04 .Grade 1
		05 .Grade 2
		06 .Grade 3		
		07 .Grade 4
		08 .Grade 5
		09 .Grade 6
		10 .Grade 7		
		11 .Grade 8  
		12 .Grade 9
		13 .Grade 10
		14 .Grade 11		
		15 .12th grade - no diploma   
		16 .Regular high school diploma
		17 .GED or alternative credential
		18 .Some college, but less than 1 year
		19 .1 or more years of college credit, no degree
		20 .Associate's degree		
		21 .Bachelor's degree
		22 .Master's degree
		23 .Professional degree beyond a bachelor's degree
		24 .Doctorate degree

SEMP		6	
	Self-employment income past 12 months (signed)
		bbbbbb 	 	 .N/A (less than 15 years old)
		000000 	 	 .None		
		-10000..-00001	 .Loss of $1 to $10000 (Rounded and bottom-coded)
		000001 		 .$1 or break even	
		000002..999999	 .$2 to $999999 (Rounded and top-coded)

Note: Use ADJINC to adjust SEMP to constant dollars.

SEX		1	
	Sex
		1 .Male
		2 .Female

SSIP		5	
	Supplementary Security Income past 12 months
		bbbbb 	     .N/A (less than 15 years old)
		00000 	     .None
		00001..99999 .$1 to $99999 (Rounded)

Note: Use ADJINC to adjust SSIP to constant dollars.

SSP		5	

	Social Security income past 12 months
		bbbbb 	     .N/A (less than 15 years old)
		00000 	     .None
		00001..99999 .$1 to $99999 (Rounded)

Note: Use ADJINC to adjust SSP to constant dollars.

WAGP		6	
	Wages or salary income past 12 months
		bbbbbb 	 	.N/A (less than 15 years old)
		000000 		.None
		000001..999999 	.$1 to 999999 (Rounded and top-coded)

Note: Use ADJINC to adjust WAGP to constant dollars.

WKHP		2	
	Usual hours worked per week past 12 months
		bb 	 .N/A (less than 16 years old/did not work 
			 .during the past 12 months)
		01..98   .1 to 98 usual hours
		99 	 .99 or more usual hours

WKL		1	
	When last worked
		b .N/A (less than 16 years old)
		1 .Within the past 12 months
		2 .1-5 years ago
		3 .Over 5 years ago or never worked

WKW		1	
	Weeks worked during past 12 months
		b .N/A (less than 16 years old/did not work 
		  .during the past 12 months)
		1 .50 to 52 weeks worked during past 12 months
		2 .48 to 49 weeks worked during past 12 months
		3 .40 to 47 weeks worked during past 12 months
		4 .27 to 39 weeks worked during past 12 months
		5 .14 to 26 weeks worked during past 12 months 
		6 .less than 14 weeks worked during past 12 months

WRK		1	
	Worked last week
		b .N/A (not reported)
		1 .Worked
		2 .Did not work

YOEP		4	
	Year of entry
		bbbb .Not eligible - Born in the US		
		1921 .1921 or earlier (Bottom-coded)
		1922 .1922 - 1923
		1924 .1924 - 1925
		1926 .1926 - 1927
		1928 .1928 - 1929
		1930 .1930 – 1931
		1932 .1932 - 1934
		1935 .1935 - 1936
		1937 .1937 - 1938
		1939 .1939
		1940 .1940
		1941 .1941
		1942 .1942
		1943 .1943 - 1944
		1945 .1945
		1946 .1946
		1947 .1947
		1948 .1948
		1949 .1949
		1950 .1950
		1951 .1951
		1952 .1952
		1953 .1953
		1954 .1954
		1955 .1955
		1956 .1956
		1957 .1957
		1958 .1958
		1959 .1959
		1960 .1960
		1961 .1961
		1962 .1962
		1963 .1963
		1964 .1964
		1965 .1965
		1966 .1966
		1967 .1967
		1968 .1968
		1969 .1969
		1970 .1970
		1971 .1971
		1972 .1972
		1973 .1973
		1974 .1974
		1975 .1975
		1976 .1976
		1977 .1977
		1978 .1978
		1979 .1979
		1980 .1980
		1981 .1981
		1982 .1982
		1983 .1983
		1984 .1984
		1985 .1985
		1986 .1986
		1987 .1987
		1988 .1988
		1989 .1989
		1990 .1990
		1991 .1991
		1992 .1992
		1993 .1993
		1994 .1994
		1995 .1995
		1996 .1996
		1997 .1997
		1998 .1998
		1999 .1999
		2000 .2000
		2001 .2001
		2002 .2002
		2003 .2003
		2004 .2004
		2005 .2005
		2006 .2006
		2007 .2007
		2008 .2008
		2009 .2009
		2010 .2010
		2011 .2011
		2012 .2012
		2013 .2013
		2014 .2014
		2015 .2015
	
ANC		1	
	Ancestry recode
		1 .Single
		2 .Multiple
		3 .Unclassified
		4 .Not reported

ANC1P		3	
	Recoded Detailed Ancestry - first entry	
		001 .Alsatian
		003 .Austrian
		005 .Basque
		008 .Belgian
		009 .Flemish
		011 .British
		012 .British Isles
		020 .Danish
		021 .Dutch
		022 .English
		024 .Finnish
		026 .French
		032 .German
		040 .Prussian
		046 .Greek
		049 .Icelander
		050 .Irish
		051 .Italian
		068 .Sicilian
		077 .Luxemburger
		078 .Maltese
		082 .Norwegian
		084 .Portuguese
		087 .Scotch Irish
		088 .Scottish
		089 .Swedish
		091 .Swiss
		094 .Irish Scotch
		097 .Welsh
		098 .Scandinavian
		099 .Celtic
		100 .Albanian
		102 .Belorussian
		103 .Bulgarian
		109 .Croatian
		111 .Czech
		112 .Bohemian
		114 .Czechoslovakian
		115 .Estonian
		122 .German Russian
		124 .Rom
		125 .Hungarian
		128 .Latvian
		129 .Lithuanian
		130 .Macedonian
		131 .Montenegrin
		142 .Polish
		144 .Romanian
		146 .Moldavian
		148 .Russian
		152 .Serbian
		153 .Slovak
		154 .Slovene
		168 .Turkestani
		169 .Uzbeg
		170 .Georgia CIS
		171 .Ukrainian
		176 .Yugoslavian
		177 .Bosnian and Herzegovinian
		178 .Slavic
		179 .Slavonian
		181 .Central European
		183 .Northern European
		185 .Southern European
		187 .Western European
		190 .Eastern European
		194 .Germanic
		195 .European
		200 .Spaniard
		210 .Mexican
		211 .Mexican American
		212 .Mexicano
		213 .Chicano
		215 .Mexican American Indian
		218 .Mexican State
		219 .Mexican Indian
		221 .Costa Rican
		222 .Guatemalan
		223 .Honduran
		224 .Nicaraguan
		225 .Panamanian
		226 .Salvadoran
		227 .Central American
		231 .Argentinean
		232 .Bolivian
		233 .Chilean
		234 .Colombian
		235 .Ecuadorian
		236 .Paraguayan
		237 .Peruvian
		238 .Uruguayan
		239 .Venezuelan
		249 .South American
		250 .Latin American
		251 .Latin
		252 .Latino
		261 .Puerto Rican
		271 .Cuban
		275 .Dominican
		290 .Hispanic
		291 .Spanish
		295 .Spanish American
		300 .Bahamian
		301 .Barbadian
		302 .Belizean
		308 .Jamaican
		310 .Dutch West Indian
		314 .Trinidadian Tobagonian
		322 .British West Indian
		325 .Antigua and Barbuda
		329 .Grenadian
		330 .Vincent-Grenadine Islander
		331 .St Lucia Islander
		335 .West Indian
		336 .Haitian
		359 .Other West Indian
		360 .Brazilian
		370 .Guyanese
		400 .Algerian
		402 .Egyptian
		406 .Moroccan
		411 .North African
		416 .Iranian
		417 .Iraqi
		419 .Israeli
		421 .Jordanian
		425 .Lebanese
		427 .Saudi Arabian
		429 .Syrian
		431 .Armenian
		434 .Turkish
		435 .Yemeni
		442 .Kurdish
		465 .Palestinian
		483 .Assyrian
		484 .Chaldean
		490 .Mideast
		495 .Arab
		496 .Arabic
		499 .Other Arab
		508 .Cameroonian
		510 .Cape Verdean
		515 .Congolese
		522 .Ethiopian
		523 .Eritrean
		529 .Ghanaian
		534 .Kenyan
		541 .Liberian
		553 .Nigerian
		564 .Senegalese
		566 .Sierra Leonean
		568 .Somali
		570 .South African
		576 .Sudanese
		587 .Other Subsaharan African
		588 .Ugandan
		598 .Western African
		599 .African
		600 .Afghan
		603 .Bangladeshi
		607 .Bhutanese
		609 .Nepali
		615 .Asian Indian
		618 .Bengali
		620 .East Indian
		650 .Punjabi
		680 .Pakistani
		690 .Sri Lankan
		700 .Burmese
		703 .Cambodian
		706 .Chinese
		707 .Cantonese
		712 .Mongolian
		714 .Tibetan
		720 .Filipino
		730 .Indonesian
		740 .Japanese
		748 .Okinawan
		750 .Korean
		765 .Laotian
		768 .Hmong
		770 .Malaysian
		776 .Thai
		782 .Taiwanese
		785 .Vietnamese
		793 .Eurasian
		795 .Asian
		799 .Other Asian
		800 .Australian
		803 .New Zealander
		808 .Polynesian
		811 .Hawaiian
		814 .Samoan
		815 .Tongan
		820 .Micronesian
		821 .Guamanian
		822 .Chamorro		
       		825 .Marshallese
		841 .Fijian
		850 .Pacific Islander
		899 .Other Pacific
		900 .Afro American
		901 .Afro
		902 .African American
		903 .Black
		904 .Negro
		907 .Creole
		913 .Central American Indian
		914 .South American Indian
		917 .Native American
		918 .Indian
		919 .Cherokee
		920 .American Indian
		921 .Aleut
		922 .Eskimo
		924 .White
		925 .Anglo
		927 .Appalachian
		929 .Pennsylvania German
		931 .Canadian
		935 .French Canadian
		937 .Cajun
		939 .American		
       		940 .United States
		983 .Texas
		994 .North American
		995 .Mixture
		996 .Uncodable entries
		997 .Other groups
		998 .Other responses
		999 .Not reported

ANC2P		3	
	Recoded Detailed Ancestry - second entry	
		001 .Alsatian
		003 .Austrian
		005 .Basque
		008 .Belgian
		009 .Flemish
		011 .British
		012 .British Isles
		020 .Danish
		021 .Dutch
		022 .English
		024 .Finnish
		026 .French
		032 .German
		040 .Prussian
		046 .Greek
		049 .Icelander
		050 .Irish
		051 .Italian
		068 .Sicilian
		077 .Luxemburger
		078 .Maltese
		082 .Norwegian
		084 .Portuguese
		087 .Scotch Irish
		088 .Scottish
		089 .Swedish
		091 .Swiss
		094 .Irish Scotch
		097 .Welsh
		098 .Scandinavian
		099 .Celtic
		100 .Albanian
		102 .Belorussian
		103 .Bulgarian
		109 .Croatian
		111 .Czech
		112 .Bohemian
		114 .Czechoslovakian
		115 .Estonian
		122 .German Russian
		124 .Rom
		125 .Hungarian
		128 .Latvian
		129 .Lithuanian
		130 .Macedonian
		131 .Montenegrin
		142 .Polish
		144 .Romanian
		146 .Moldavian
		148 .Russian
		152 .Serbian
		153 .Slovak
		154 .Slovene
		168 .Turkestani
		169 .Uzbeg
		170 .Georgia CIS
		171 .Ukrainian
		176 .Yugoslavian
		177 .Bosnian and Herzegovinian
		178 .Slavic
		179 .Slavonian
		181 .Central European
		183 .Northern European
		185 .Southern European
		187 .Western European
		190 .Eastern European
		194 .Germanic
		195 .European
		200 .Spaniard
		210 .Mexican
		211 .Mexican American
		212 .Mexicano
		213 .Chicano
		215 .Mexican American Indian
		218 .Mexican State
		219 .Mexican Indian
		221 .Costa Rican
		222 .Guatemalan
		223 .Honduran
		224 .Nicaraguan
		225 .Panamanian
		226 .Salvadoran
		227 .Central American
		231 .Argentinean
		232 .Bolivian
		233 .Chilean
		234 .Colombian
		235 .Ecuadorian
		236 .Paraguayan
		237 .Peruvian
		238 .Uruguayan
		239 .Venezuelan
		249 .South American
		250 .Latin American
		251 .Latin
		252 .Latino
		261 .Puerto Rican
		271 .Cuban
		275 .Dominican
		290 .Hispanic
		291 .Spanish
		295 .Spanish American
		300 .Bahamian
		301 .Barbadian
		302 .Belizean
		308 .Jamaican
		310 .Dutch West Indian
		314 .Trinidadian Tobagonian
		322 .British West Indian
		325 .Antigua and Barbuda
		329 .Grenadian
		330 .Vincent-Grenadine Islander
		331 .St Lucia Islander
		335 .West Indian
		336 .Haitian
		359 .Other West Indian
		360 .Brazilian
		370 .Guyanese
		400 .Algerian
		402 .Egyptian
		406 .Moroccan
		411 .North African
		416 .Iranian
		417 .Iraqi
		419 .Israeli
		421 .Jordanian
		425 .Lebanese
		427 .Saudi Arabian
		429 .Syrian
		431 .Armenian
		434 .Turkish
		435 .Yemeni
		442 .Kurdish
		465 .Palestinian
		483 .Assyrian
		484 .Chaldean
		490 .Mideast
		495 .Arab
		496 .Arabic
		499 .Other Arab
		508 .Cameroonian
		510 .Cape Verdean
		515 .Congolese
		522 .Ethiopian
		523 .Eritrean
		529 .Ghanaian
		534 .Kenyan
		541 .Liberian
		553 .Nigerian
		564 .Senegalese
		566 .Sierra Leonean
		568 .Somali
		570 .South African
		576 .Sudanese
		587 .Other Subsaharan African
		588 .Ugandan
		598 .Western African
		599 .African
		600 .Afghan
		603 .Bangladeshi
		607 .Bhutanese
		609 .Nepali
		615 .Asian Indian
		618 .Bengali
		620 .East Indian
		650 .Punjabi
		680 .Pakistani
		690 .Sri Lankan
		700 .Burmese
		703 .Cambodian
		706 .Chinese
		707 .Cantonese
		712 .Mongolian
		714 .Tibetan
		720 .Filipino
		730 .Indonesian
		740 .Japanese
		748 .Okinawan
		750 .Korean
		765 .Laotian
		768 .Hmong
		770 .Malaysian
		776 .Thai
		782 .Taiwanese
		785 .Vietnamese
		793 .Eurasian
		795 .Asian
		799 .Other Asian
		800 .Australian
		803 .New Zealander
		808 .Polynesian
		811 .Hawaiian
		814 .Samoan
		815 .Tongan
		820 .Micronesian
		821 .Guamanian
		822 .Chamorro		
       		825 .Marshallese
		841 .Fijian
		850 .Pacific Islander
		899 .Other Pacific
		900 .Afro American
		901 .Afro
		902 .African American
		903 .Black
		904 .Negro
		907 .Creole
		913 .Central American Indian
		914 .South American Indian
		917 .Native American
		918 .Indian
		919 .Cherokee
		920 .American Indian
		921 .Aleut
		922 .Eskimo
		924 .White
		925 .Anglo
		927 .Appalachian
		929 .Pennsylvania German
		931 .Canadian
		935 .French Canadian
		937 .Cajun
		939 .American 
		940 .United States
		983 .Texas
		994 .North American
		995 .Mixture
		996 .Uncodable entries
		997 .Other groups
		998 .Other responses
		999 .Not reported

DECADE		1	
	Decade of entry
		b .N/A (Born in the US)
		1 .Before 1950
		2 .1950 - 1959
		3 .1960 - 1969
		4 .1970 - 1979
		5 .1980 - 1989
		6 .1990 - 1999
		7 .2000 or later 

DIS		1	
	Disability recode
		1 .With a disability
		2 .Without a disability

DRIVESP		1	
	Number of vehicles calculated from JWRI
		b .N/A (Nonworker or worker who does not drive to work)
		1 .1.000 vehicles (Drove alone)
		2 .0.500 vehicles (In a 2-person carpool)
		3 .0.333 vehicles (In a 3-person carpool)
		4 .0.250 vehicles (In a 4-person carpool)
		5 .0.200 vehicles (In a 5- or 6-person carpool)
		6 .0.143 vehicles (In a 7-or-more person carpool)

ESP		1	
	Employment status of parents
		b .N/A (not own child of householder, and not
		  .child in subfamily)
		  .Living with two parents:
		1 .Both parents in labor force 
		2 .Father only in labor force
		3 .Mother only in labor force
		4 .Neither parent in labor force
		  .Living with one parent:
		  .Living with father:
		5 .Father in the labor force	
		6 .Father not in labor force
		  .Living with mother:
		7 .Mother in the labor force	
		8 .Mother not in labor force

ESR		1	
	Employment status recode
		b .N/A (less than 16 years old)
		1 .Civilian employed, at work
		2 .Civilian employed, with a job but not at work
		3 .Unemployed
		4 .Armed forces, at work
		5 .Armed forces, with a job but not at work
		6 .Not in labor force

FOD1P		4
	Recoded field of degree - first entry
		bbbb .N/A (less than bachelor's degree)
		1100 .GENERAL AGRICULTURE
		1101 .AGRICULTURE PRODUCTION AND MANAGEMENT
		1102 .AGRICULTURAL ECONOMICS
		1103 .ANIMAL SCIENCES
		1104 .FOOD SCIENCE
		1105 .PLANT SCIENCE AND AGRONOMY
		1106 .SOIL SCIENCE
		1199 .MISCELLANEOUS AGRICULTURE
		1301 .ENVIRONMENTAL SCIENCE
		1302 .FORESTRY
		1303 .NATURAL RESOURCES MANAGEMENT
		1401 .ARCHITECTURE
		1501 .AREA ETHNIC AND CIVILIZATION STUDIES
		1901 .COMMUNICATIONS
		1902 .JOURNALISM
		1903 .MASS MEDIA
		1904 .ADVERTISING AND PUBLIC RELATIONS
		2001 .COMMUNICATION TECHNOLOGIES
		2100 .COMPUTER AND INFORMATION SYSTEMS
		2101 .COMPUTER PROGRAMMING AND DATA PROCESSING
		2102 .COMPUTER SCIENCE
		2105 .INFORMATION SCIENCES
		2106 .COMPUTER ADMINISTRATION MANAGEMENT AND SECURITY
		2107 .COMPUTER NETWORKING AND TELECOMMUNICATIONS
		2201 .COSMETOLOGY SERVICES AND CULINARY ARTS
		2300 .GENERAL EDUCATION
		2301 .EDUCATIONAL ADMINISTRATION AND SUPERVISION
		2303 .SCHOOL STUDENT COUNSELING
		2304 .ELEMENTARY EDUCATION
		2305 .MATHEMATICS TEACHER EDUCATION
		2306 .PHYSICAL AND HEALTH EDUCATION TEACHING
		2307 .EARLY CHILDHOOD EDUCATION
		2308 .SCIENCE AND COMPUTER TEACHER EDUCATION
		2309 .SECONDARY TEACHER EDUCATION
		2310 .SPECIAL NEEDS EDUCATION
		2311 .SOCIAL SCIENCE OR HISTORY TEACHER EDUCATION
		2312 .TEACHER EDUCATION: MULTIPLE LEVELS
		2313 .LANGUAGE AND DRAMA EDUCATION
		2314 .ART AND MUSIC EDUCATION
		2399 .MISCELLANEOUS EDUCATION
		2400 .GENERAL ENGINEERING
		2401 .AEROSPACE ENGINEERING
		2402 .BIOLOGICAL ENGINEERING
		2403 .ARCHITECTURAL ENGINEERING
		2404 .BIOMEDICAL ENGINEERING
		2405 .CHEMICAL ENGINEERING
		2406 .CIVIL ENGINEERING
		2407 .COMPUTER ENGINEERING
		2408 .ELECTRICAL ENGINEERING
		2409 .ENGINEERING MECHANICS PHYSICS AND SCIENCE
		2410 .ENVIRONMENTAL ENGINEERING
		2411 .GEOLOGICAL AND GEOPHYSICAL ENGINEERING
		2412 .INDUSTRIAL AND MANUFACTURING ENGINEERING
		2413 .MATERIALS ENGINEERING AND MATERIALS SCIENCE
		2414 .MECHANICAL ENGINEERING
		2415 .METALLURGICAL ENGINEERING
		2416 .MINING AND MINERAL ENGINEERING
		2417 .NAVAL ARCHITECTURE AND MARINE ENGINEERING
		2418 .NUCLEAR ENGINEERING
		2419 .PETROLEUM ENGINEERING
		2499 .MISCELLANEOUS ENGINEERING
		2500 .ENGINEERING TECHNOLOGIES
		2501 .ENGINEERING AND INDUSTRIAL MANAGEMENT
		2502 .ELECTRICAL ENGINEERING TECHNOLOGY
		2503 .INDUSTRIAL PRODUCTION TECHNOLOGIES
		2504 .MECHANICAL ENGINEERING RELATED TECHNOLOGIES
		2599 .MISCELLANEOUS ENGINEERING TECHNOLOGIES
		2601 .LINGUISTICS AND COMPARATIVE LANGUAGE AND LITERATURE
		2602 .FRENCH GERMAN LATIN AND OTHER COMMON FOREIGN LANGUAGE 
		     .STUDIES
		2603 .OTHER FOREIGN LANGUAGES
		2901 .FAMILY AND CONSUMER SCIENCES
		3201 .COURT REPORTING
		3202 .PRE-LAW AND LEGAL STUDIES
		3301 .ENGLISH LANGUAGE AND LITERATURE
		3302 .COMPOSITION AND RHETORIC
		3401 .LIBERAL ARTS
		3402 .HUMANITIES
		3501 .LIBRARY SCIENCE
		3600 .BIOLOGY
		3601 .BIOCHEMICAL SCIENCES
		3602 .BOTANY
		3603 .MOLECULAR BIOLOGY
		3604 .ECOLOGY
		3605 .GENETICS
		3606 .MICROBIOLOGY
		3607 .PHARMACOLOGY
		3608 .PHYSIOLOGY
		3609 .ZOOLOGY
		3611 .NEUROSCIENCE
		3699 .MISCELLANEOUS BIOLOGY
		3700 .MATHEMATICS
		3701 .APPLIED MATHEMATICS
		3702 .STATISTICS AND DECISION SCIENCE
		3801 .MILITARY TECHNOLOGIES
		4000 .MULTI/INTERDISCIPLINARY STUDIES
		4001 .INTERCULTURAL AND INTERNATIONAL STUDIES
		4002 .NUTRITION SCIENCES
		4005 .MATHEMATICS AND COMPUTER SCIENCE
		4006 .COGNITIVE SCIENCE AND BIOPSYCHOLOGY
		4007 .INTERDISCIPLINARY SOCIAL SCIENCES
		4101 .PHYSICAL FITNESS PARKS RECREATION AND LEISURE
		4801 .PHILOSOPHY AND RELIGIOUS STUDIES
		4901 .THEOLOGY AND RELIGIOUS VOCATIONS
		5000 .PHYSICAL SCIENCES
		5001 .ASTRONOMY AND ASTROPHYSICS
		5002 .ATMOSPHERIC SCIENCES AND METEOROLOGY
		5003 .CHEMISTRY
		5004 .GEOLOGY AND EARTH SCIENCE
		5005 .GEOSCIENCES
		5006 .OCEANOGRAPHY
		5007 .PHYSICS
		5008 .MATERIALS SCIENCE
		5098 .MULTI-DISCIPLINARY OR GENERAL SCIENCE
		5102 .NUCLEAR, INDUSTRIAL RADIOLOGY, AND BIOLOGICAL TECHNOLOGIES
		5200 .PSYCHOLOGY
		5201 .EDUCATIONAL PSYCHOLOGY
		5202 .CLINICAL PSYCHOLOGY
		5203 .COUNSELING PSYCHOLOGY
		5205 .INDUSTRIAL AND ORGANIZATIONAL PSYCHOLOGY
		5206 .SOCIAL PSYCHOLOGY
		5299 .MISCELLANEOUS PSYCHOLOGY
		5301 .CRIMINAL JUSTICE AND FIRE PROTECTION
		5401 .PUBLIC ADMINISTRATION
		5402 .PUBLIC POLICY
		5403 .HUMAN SERVICES AND COMMUNITY ORGANIZATION
		5404 .SOCIAL WORK
		5500 .GENERAL SOCIAL SCIENCES
		5501 .ECONOMICS
		5502 .ANTHROPOLOGY AND ARCHEOLOGY
		5503 .CRIMINOLOGY
		5504 .GEOGRAPHY
		5505 .INTERNATIONAL RELATIONS
		5506 .POLITICAL SCIENCE AND GOVERNMENT
		5507 .SOCIOLOGY
		5599 .MISCELLANEOUS SOCIAL SCIENCES
		5601 .CONSTRUCTION SERVICES
		5701 .ELECTRICAL, MECHANICAL, AND PRECISION TECHNOLOGIES AND
		     .PRODUCTION
		5901 .TRANSPORTATION SCIENCES AND TECHNOLOGIES
		6000 .FINE ARTS
		6001 .DRAMA AND THEATER ARTS
		6002 .MUSIC
		6003 .VISUAL AND PERFORMING ARTS
		6004 .COMMERCIAL ART AND GRAPHIC DESIGN
		6005 .FILM VIDEO AND PHOTOGRAPHIC ARTS
		6006 .ART HISTORY AND CRITICISM
		6007 .STUDIO ARTS
		6099 .MISCELLANEOUS FINE ARTS
		6100 .GENERAL MEDICAL AND HEALTH SERVICES
		6102 .COMMUNICATION DISORDERS SCIENCES AND SERVICES
		6103 .HEALTH AND MEDICAL ADMINISTRATIVE SERVICES
		6104 .MEDICAL ASSISTING SERVICES
		6105 .MEDICAL TECHNOLOGIES TECHNICIANS
		6106 .HEALTH AND MEDICAL PREPARATORY PROGRAMS
		6107 .NURSING
		6108 .PHARMACY PHARMACEUTICAL SCIENCES AND ADMINISTRATION
		6109 .TREATMENT THERAPY PROFESSIONS
		6110 .COMMUNITY AND PUBLIC HEALTH
		6199 .MISCELLANEOUS HEALTH MEDICAL PROFESSIONS
		6200 .GENERAL BUSINESS
		6201 .ACCOUNTING
		6202 .ACTUARIAL SCIENCE
		6203 .BUSINESS MANAGEMENT AND ADMINISTRATION
		6204 .OPERATIONS LOGISTICS AND E-COMMERCE
		6205 .BUSINESS ECONOMICS
		6206 .MARKETING AND MARKETING RESEARCH
		6207 .FINANCE
		6209 .HUMAN RESOURCES AND PERSONNEL MANAGEMENT
		6210 .INTERNATIONAL BUSINESS
		6211 .HOSPITALITY MANAGEMENT
		6212 .MANAGEMENT INFORMATION SYSTEMS AND STATISTICS
		6299 .MISCELLANEOUS BUSINESS & MEDICAL ADMINISTRATION
		6402 .HISTORY
		6403 .UNITED STATES HISTORY

FOD2P		4
	Recoded field of degree - second entry
		bbbb .N/A (less than bachelor's degree)
		1100 .GENERAL AGRICULTURE
		1101 .AGRICULTURE PRODUCTION AND MANAGEMENT
		1102 .AGRICULTURAL ECONOMICS
		1103 .ANIMAL SCIENCES
		1104 .FOOD SCIENCE
		1105 .PLANT SCIENCE AND AGRONOMY
		1106 .SOIL SCIENCE
		1199 .MISCELLANEOUS AGRICULTURE
		1301 .ENVIRONMENTAL SCIENCE
		1302 .FORESTRY
		1303 .NATURAL RESOURCES MANAGEMENT
		1401 .ARCHITECTURE
		1501 .AREA ETHNIC AND CIVILIZATION STUDIES
		1901 .COMMUNICATIONS
		1902 .JOURNALISM
		1903 .MASS MEDIA
		1904 .ADVERTISING AND PUBLIC RELATIONS
		2001 .COMMUNICATION TECHNOLOGIES
		2100 .COMPUTER AND INFORMATION SYSTEMS
		2101 .COMPUTER PROGRAMMING AND DATA PROCESSING
		2102 .COMPUTER SCIENCE
		2105 .INFORMATION SCIENCES
		2106 .COMPUTER ADMINISTRATION MANAGEMENT AND SECURITY
		2107 .COMPUTER NETWORKING AND TELECOMMUNICATIONS
		2201 .COSMETOLOGY SERVICES AND CULINARY ARTS
		2300 .GENERAL EDUCATION
		2301 .EDUCATIONAL ADMINISTRATION AND SUPERVISION
		2303 .SCHOOL STUDENT COUNSELING
		2304 .ELEMENTARY EDUCATION
		2305 .MATHEMATICS TEACHER EDUCATION
		2306 .PHYSICAL AND HEALTH EDUCATION TEACHING
		2307 .EARLY CHILDHOOD EDUCATION
		2308 .SCIENCE AND COMPUTER TEACHER EDUCATION
		2309 .SECONDARY TEACHER EDUCATION
		2310 .SPECIAL NEEDS EDUCATION
		2311 .SOCIAL SCIENCE OR HISTORY TEACHER EDUCATION
		2312 .TEACHER EDUCATION: MULTIPLE LEVELS
		2313 .LANGUAGE AND DRAMA EDUCATION
		2314 .ART AND MUSIC EDUCATION
		2399 .MISCELLANEOUS EDUCATION
		2400 .GENERAL ENGINEERING
		2401 .AEROSPACE ENGINEERING
		2402 .BIOLOGICAL ENGINEERING
		2403 .ARCHITECTURAL ENGINEERING
		2404 .BIOMEDICAL ENGINEERING
		2405 .CHEMICAL ENGINEERING
		2406 .CIVIL ENGINEERING
		2407 .COMPUTER ENGINEERING
		2408 .ELECTRICAL ENGINEERING
		2409 .ENGINEERING MECHANICS PHYSICS AND SCIENCE
		2410 .ENVIRONMENTAL ENGINEERING
		2411 .GEOLOGICAL AND GEOPHYSICAL ENGINEERING
		2412 .INDUSTRIAL AND MANUFACTURING ENGINEERING
		2413 .MATERIALS ENGINEERING AND MATERIALS SCIENCE
		2414 .MECHANICAL ENGINEERING
		2415 .METALLURGICAL ENGINEERING
		2416 .MINING AND MINERAL ENGINEERING
		2417 .NAVAL ARCHITECTURE AND MARINE ENGINEERING
		2418 .NUCLEAR ENGINEERING
		2419 .PETROLEUM ENGINEERING
		2499 .MISCELLANEOUS ENGINEERING
		2500 .ENGINEERING TECHNOLOGIES
		2501 .ENGINEERING AND INDUSTRIAL MANAGEMENT
		2502 .ELECTRICAL ENGINEERING TECHNOLOGY
		2503 .INDUSTRIAL PRODUCTION TECHNOLOGIES
		2504 .MECHANICAL ENGINEERING RELATED TECHNOLOGIES
		2599 .MISCELLANEOUS ENGINEERING TECHNOLOGIES
		2601 .LINGUISTICS AND COMPARATIVE LANGUAGE AND LITERATURE
		2602 .FRENCH GERMAN LATIN AND OTHER COMMON FOREIGN LANGUAGE 
            	     .STUDIES
		2603 .OTHER FOREIGN LANGUAGES
		2901 .FAMILY AND CONSUMER SCIENCES
		3201 .COURT REPORTING
		3202 .PRE-LAW AND LEGAL STUDIES
		3301 .ENGLISH LANGUAGE AND LITERATURE
		3302 .COMPOSITION AND RHETORIC
		3401 .LIBERAL ARTS
		3402 .HUMANITIES
		3501 .LIBRARY SCIENCE
		3600 .BIOLOGY
		3601 .BIOCHEMICAL SCIENCES
		3602 .BOTANY
		3603 .MOLECULAR BIOLOGY
		3604 .ECOLOGY
		3605 .GENETICS
		3606 .MICROBIOLOGY
		3607 .PHARMACOLOGY
		3608 .PHYSIOLOGY
		3609 .ZOOLOGY
		3611 .NEUROSCIENCE
		3699 .MISCELLANEOUS BIOLOGY
		3700 .MATHEMATICS
		3701 .APPLIED MATHEMATICS
		3702 .STATISTICS AND DECISION SCIENCE
		3801 .MILITARY TECHNOLOGIES
		4000 .MULTI/INTERDISCIPLINARY STUDIES
		4001 .INTERCULTURAL AND INTERNATIONAL STUDIES
		4002 .NUTRITION SCIENCES
		4005 .MATHEMATICS AND COMPUTER SCIENCE
		4006 .COGNITIVE SCIENCE AND BIOPSYCHOLOGY
		4007 .INTERDISCIPLINARY SOCIAL SCIENCES
		4101 .PHYSICAL FITNESS PARKS RECREATION AND LEISURE
		4801 .PHILOSOPHY AND RELIGIOUS STUDIES
		4901 .THEOLOGY AND RELIGIOUS VOCATIONS
		5000 .PHYSICAL SCIENCES
		5001 .ASTRONOMY AND ASTROPHYSICS
		5002 .ATMOSPHERIC SCIENCES AND METEOROLOGY
		5003 .CHEMISTRY
		5004 .GEOLOGY AND EARTH SCIENCE
		5005 .GEOSCIENCES
		5006 .OCEANOGRAPHY
		5007 .PHYSICS
		5008 .MATERIALS SCIENCE
		5098 .MULTI-DISCIPLINARY OR GENERAL SCIENCE
		5102 .NUCLEAR, INDUSTRIAL RADIOLOGY, AND BIOLOGICAL TECHNOLOGIES
		5200 .PSYCHOLOGY
		5201 .EDUCATIONAL PSYCHOLOGY
		5202 .CLINICAL PSYCHOLOGY
		5203 .COUNSELING PSYCHOLOGY
		5205 .INDUSTRIAL AND ORGANIZATIONAL PSYCHOLOGY
		5206 .SOCIAL PSYCHOLOGY
		5299 .MISCELLANEOUS PSYCHOLOGY
		5301 .CRIMINAL JUSTICE AND FIRE PROTECTION
		5401 .PUBLIC ADMINISTRATION
		5402 .PUBLIC POLICY
		5403 .HUMAN SERVICES AND COMMUNITY ORGANIZATION
		5404 .SOCIAL WORK
		5500 .GENERAL SOCIAL SCIENCES
		5501 .ECONOMICS
		5502 .ANTHROPOLOGY AND ARCHEOLOGY
		5503 .CRIMINOLOGY
		5504 .GEOGRAPHY
		5505 .INTERNATIONAL RELATIONS
		5506 .POLITICAL SCIENCE AND GOVERNMENT
		5507 .SOCIOLOGY
		5599 .MISCELLANEOUS SOCIAL SCIENCES
		5601 .CONSTRUCTION SERVICES
		5701 .ELECTRICAL, MECHANICAL, AND PRECISION TECHNOLOGIES AND
		     .PRODUCTION
		5901 .TRANSPORTATION SCIENCES AND TECHNOLOGIES
		6000 .FINE ARTS
		6001 .DRAMA AND THEATER ARTS
		6002 .MUSIC
		6003 .VISUAL AND PERFORMING ARTS
		6004 .COMMERCIAL ART AND GRAPHIC DESIGN
		6005 .FILM VIDEO AND PHOTOGRAPHIC ARTS
		6006 .ART HISTORY AND CRITICISM
		6007 .STUDIO ARTS
		6099 .MISCELLANEOUS FINE ARTS
		6100 .GENERAL MEDICAL AND HEALTH SERVICES
		6102 .COMMUNICATION DISORDERS SCIENCES AND SERVICES
		6103 .HEALTH AND MEDICAL ADMINISTRATIVE SERVICES
		6104 .MEDICAL ASSISTING SERVICES
		6105 .MEDICAL TECHNOLOGIES TECHNICIANS
		6106 .HEALTH AND MEDICAL PREPARATORY PROGRAMS
		6107 .NURSING
		6108 .PHARMACY PHARMACEUTICAL SCIENCES AND ADMINISTRATION
		6109 .TREATMENT THERAPY PROFESSIONS
		6110 .COMMUNITY AND PUBLIC HEALTH
		6199 .MISCELLANEOUS HEALTH MEDICAL PROFESSIONS
		6200 .GENERAL BUSINESS
		6201 .ACCOUNTING
		6202 .ACTUARIAL SCIENCE
		6203 .BUSINESS MANAGEMENT AND ADMINISTRATION
		6204 .OPERATIONS LOGISTICS AND E-COMMERCE
		6205 .BUSINESS ECONOMICS
		6206 .MARKETING AND MARKETING RESEARCH
		6207 .FINANCE
		6209 .HUMAN RESOURCES AND PERSONNEL MANAGEMENT
		6210 .INTERNATIONAL BUSINESS
		6211 .HOSPITALITY MANAGEMENT
		6212 .MANAGEMENT INFORMATION SYSTEMS AND STATISTICS
		6299 .MISCELLANEOUS BUSINESS & MEDICAL ADMINISTRATION
		6402 .HISTORY
		6403 .UNITED STATES HISTORY

HICOV		1	
	Health insurance coverage recode
		1 .With health insurance coverage
		2 .No health insurance coverage

HISP		2	
	Recoded detailed Hispanic origin 
		01 .Not Spanish/Hispanic/Latino
		02 .Mexican
		03 .Puerto Rican
		04 .Cuban
		05 .Dominican
		06 .Costa Rican
		07 .Guatemalan
		08 .Honduran
		09 .Nicaraguan
		10 .Panamanian
		11 .Salvadoran
		12 .Other Central American
		13 .Argentinean
		14 .Bolivian
		15 .Chilean
		16 .Colombian
		17 .Ecuadorian
		18 .Paraguayan
		19 .Peruvian
		20 .Uruguayan
		21 .Venezuelan
		22 .Other South American
		23 .Spaniard
		24 .All Other Spanish/Hispanic/Latino

INDP		4	
	Industry recode for 2013 and later based on 2012 IND codes	
		bbbb .N/A (less than 16 years old/NILF who last worked
		     .more than 5 years ago or never worked)
		0170 .AGR-CROP PRODUCTION
		0180 .AGR-ANIMAL PRODUCTION AND AQUACULTURE
		0190 .AGR-FORESTRY EXCEPT LOGGING
		0270 .AGR-LOGGING
		0280 .AGR-FISHING, HUNTING AND TRAPPING
		0290 .AGR-SUPPORT ACTIVITIES FOR AGRICULTURE AND FORESTRY
		0370 .EXT-OIL AND GAS EXTRACTION
		0380 .EXT-COAL MINING
		0390 .EXT-METAL ORE MINING
		0470 .EXT-NONMETALLIC MINERAL MINING AND QUARRYING
		0490 .EXT-SUPPORT ACTIVITIES FOR MINING
		0570 .UTL-ELECTRIC POWER GENERATION, TRANSMISSION AND
		     .DISTRIBUTION
		0580 .UTL-NATURAL GAS DISTRIBUTION
		0590 .UTL-ELECTRIC AND GAS, AND OTHER COMBINATIONS
		0670 .UTL-WATER, STEAM, AIR CONDITIONING, AND IRRIGATION SYSTEMS
		0680 .UTL-SEWAGE TREATMENT FACILITIES
		0690 .UTL-NOT SPECIFIED UTILITIES
		0770 .CON-CONSTRUCTION, INCL CLEANING DURING AND IMM AFTER
		1070 .MFG-ANIMAL FOOD, GRAIN AND OILSEED MILLING
		1080 .MFG-SUGAR AND CONFECTIONERY PRODUCTS
		1090 .MFG-FRUIT AND VEGETABLE PRESERVING AND SPECIALTY FOODS
		1170 .MFG-DAIRY PRODUCTS
		1180 .MFG-ANIMAL SLAUGHTERING AND PROCESSING
		1190 .MFG-RETAIL BAKERIES
		1270 .MFG-BAKERIES AND TORTILLA, EXCEPT RETAIL BAKERIES
		1280 .MFG-SEAFOOD AND OTHER MISCELLANEOUS FOODS, N.E.C.
		1290 .MFG-NOT SPECIFIED FOOD INDUSTRIES
		1370 .MFG-BEVERAGE
		1390 .MFG-TOBACCO
		1470 .MFG-FIBER, YARN, AND THREAD MILLS
		1480 .MFG-FABRIC MILLS, EXCEPT KNITTING MILLS
		1490 .MFG-TEXTILE AND FABRIC FINISHING AND FABRIC COATING MILLS
		1570 .MFG-CARPET AND RUG MILLS
		1590 .MFG-TEXTILE PRODUCT MILLS, EXCEPT CARPET AND RUG
		1670 .MFG-KNITTING FABRIC MILLS, AND APPAREL KNITTING MILLS
		1680 .MFG-CUT AND SEW APPAREL
		1690 .MFG-APPAREL ACCESSORIES AND OTHER APPAREL
		1770 .MFG-FOOTWEAR
		1790 .MFG-LEATHER TANNING AND FINISHING AND OTHER ALLIED PRODUCTS
		     .MANUFACTURING
		1870 .MFG-PULP, PAPER, AND PAPERBOARD MILLS
		1880 .MFG-PAPERBOARD CONTAINER
		1890 .MFG-MISCELLANEOUS PAPER AND PULP PRODUCTS
		1990 .MFG-PRINTING AND RELATED SUPPORT ACTIVITIES
		2070 .MFG-PETROLEUM REFINING
		2090 .MFG-MISCELLANEOUS PETROLEUM AND COAL PRODUCTS
		2170 .MFG-RESIN, SYNTHETIC RUBBER, AND FIBERS AND FILAMENTS
		2180 .MFG-AGRICULTURAL CHEMICALS
		2190 .MFG-PHARMACEUTICALS AND MEDICINES
		2270 .MFG-PAINT, COATING, AND ADHESIVES
		2280 .MFG-SOAP, CLEANING COMPOUND, AND COSMETICS
		2290 .MFG-INDUSTRIAL AND MISCELLANEOUS CHEMICALS
		2370 .MFG-PLASTICS PRODUCTS
		2380 .MFG-TIRES
		2390 .MFG-RUBBER PRODUCTS, EXCEPT TIRES
		2470 .MFG-POTTERY, CERAMICS, AND PLUMBING FIXTURE MANUFACTURING
		2480 .MFG- CLAY BUILDING MATERIAL AND REFRACTORIES
		2490 .MFG-GLASS AND GLASS PRODUCTS
		2570 .MFG-CEMENT, CONCRETE, LIME, AND GYPSUM PRODUCTS
		2590 .MFG-MISCELLANEOUS NONMETALLIC MINERAL PRODUCTS
		2670 .MFG-IRON AND STEEL MILLS AND STEEL PRODUCTS
		2680 .MFG-ALUMINUM PRODUCTION AND PROCESSING
		2690 .MFG-NONFERROUS METAL, EXCEPT ALUMINUM, PRODUCTION AND
		     .PROCESSING
		2770 .MFG-FOUNDRIES
		2780 .MFG-METAL FORGINGS AND STAMPINGS
		2790 .MFG-CUTLERY AND HAND TOOLS
		2870 .MFG-STRUCTURAL METALS, AND BOILER, TANK, AND SHIPPING
		     .CONTAINERS
		2880 .MFG-MACHINE SHOPS; TURNED PRODUCTS; SCREWS, NUTS AND BOLTS
		2890 .MFG-COATING, ENGRAVING, HEAT TREATING AND ALLIED ACTIVITIES
		2970 .MFG-ORDNANCE
		2980 .MFG-MISCELLANEOUS FABRICATED METAL PRODUCTS
		2990 .MFG-NOT SPECIFIED METAL INDUSTRIES
		3070 .MFG-AGRICULTURAL IMPLEMENTS
		3080 .MFG-CONSTRUCTION, AND MINING AND OIL AND GAS FIELD 
		     .MACHINERY
		3095 .MFG-COMMERCIAL AND SERVICE INDUSTRY MACHINERY
		3170 .MFG-METALWORKING MACHINERY
		3180 .MFG-ENGINE, TURBINE, AND POWER TRANSMISSION EQUIPMENT
		3190 .MFG-MACHINERY, N.E.C. OR NOT SPECIFIED
		3365 .MFG-COMPUTER AND PERIPHERAL EQUIPMENT
		3370 .MFG-COMMUNICATIONS, AND AUDIO AND VIDEO EQUIPMENT
		3380 .MFG-NAVIGATIONAL, MEASURING, ELECTROMEDICAL, AND CONTROL
		     .INSTRUMENTS
		3390 .MFG-ELECTRONIC COMPONENTS AND PRODUCTS, N.E.C.
		3470 .MFG-HOUSEHOLD APPLIANCES
		3490 .MFG-ELECTRIC LIGHTING AND ELECTRICAL EQUIPMENT 
		     .MANUFACTURING,AND OTHER ELECTRICAL COMPONENT 
            	     .MANUFACTURING,N.E.C.
		3570 .MFG-MOTOR VEHICLES AND MOTOR VEHICLE EQUIPMENT
		3580 .MFG-AIRCRAFT AND PARTS
		3590 .MFG-AEROSPACE PRODUCTS AND PARTS
		3670 .MFG-RAILROAD ROLLING STOCK
		3680 .MFG-SHIP AND BOAT BUILDING
		3690 .MFG-OTHER TRANSPORTATION EQUIPMENT
		3770 .MFG-SAWMILLS AND WOOD PRESERVATION
		3780 .MFG-VENEER, PLYWOOD, AND ENGINEERED WOOD PRODUCTS
		3790 .MFG-PREFABRICATED WOOD BUILDINGS AND MOBILE HOMES
		3875 .MFG-MISCELLANEOUS WOOD PRODUCTS
		3895 .MFG-FURNITURE AND RELATED PRODUCTS
		3960 .MFG-MEDICAL EQUIPMENT AND SUPPLIES
		3970 .MFG-SPORTING AND ATHLETIC GOODS, AND DOLL, TOY, AND GAME
		     .MANUFACTURING
		3980 .MFG-MISCELLANEOUS MANUFACTURING, N.E.C.
		3990 .MFG-NOT SPECIFIED MANUFACTURING INDUSTRIES
		4070 .WHL-MOTOR VEHICLE AND MOTOR VEHICLE PARTS AND SUPPLIES
		     .MERCHANT WHOLESALERS
		4080 .WHL-FURNITURE AND HOME FURNISHING MERCHANT WHOLESALERS
		4090 .WHL-LUMBER AND OTHER CONSTRUCTION MATERIALS MERCHANT
		     .WHOLESALERS
		4170 .WHL-PROFESSIONAL AND COMMERCIAL EQUIPMENT AND SUPPLIES
		     .MERCHANT WHOLESALERS
		4180 .WHL-METALS AND MINERALS, EXCEPT PETROLEUM, MERCHANT
		     .WHOLESALERS
		4195 .WHL-HOUSEHOLD APPLIANCES AND ELECTRICAL AND ELECTRONIC
		     .GOODS MERCHANT WHOLESALERS	
		4265 .WHL-HARDWARE, AND PLUMBING AND HEATING EQUIPMENT, AND
		     .SUPPLIES MERCHANT WHOLESALERS
		4270 .WHL-MACHINERY, EQUIPMENT, AND SUPPLIES MERCHANT WHOLESALERS
		4280 .WHL-RECYCLABLE MATERIAL MERCHANT WHOLESALERS
		4290 .WHL-MISCELLANEOUS DURABLE GOODS MERCHANT WHOLESALERS
		4370 .WHL-PAPER AND PAPER PRODUCTS MERCHANT WHOLESALERS
		4380 .WHL-DRUGS, SUNDRIES, AND CHEMICAL AND ALLIED PRODUCTS
		     .MERCHANT WHOLESALERS
		4390 .WHL-APPAREL, PIECE GOODS, AND NOTIONS MERCHANT WHOLESALERS
		4470 .WHL-GROCERY AND RELATED PRODUCT MERCHANT WHOLESALERS
		4480 .WHL-FARM PRODUCT RAW MATERIAL MERCHANT WHOLESALERS
		4490 .WHL-PETROLEUM AND PETROLEUM PRODUCTS MERCHANT WHOLESALERS
		4560 .WHL-ALCOHOLIC BEVERAGES MERCHANT WHOLESALERS
		4570 .WHL-FARM SUPPLIES MERCHANT WHOLESALERS
		4580 .WHL-MISCELLANEOUS NONDURABLE GOODS MERCHANT WHOLESALERS
		4585 .WHL-WHOLESALE ELECTRONIC MARKETS AND AGENTS AND BROKERS
		4590 .WHL-NOT SPECIFIED WHOLESALE TRADE
		4670 .RET-AUTOMOBILE DEALERS
		4680 .RET-OTHER MOTOR VEHICLE DEALERS
		4690 .RET-AUTOMOTIVE PARTS, ACCESSORIES, AND TIRE STORES
		4770 .RET-FURNITURE AND HOME FURNISHINGS STORES
		4780 .RET-HOUSEHOLD APPLIANCE STORES
		4795 .RET-ELECTRONICS STORES
		4870 .RET-BUILDING MATERIAL AND SUPPLIES DEALERS
		4880 .RET-HARDWARE STORES
		4890 .RET-LAWN AND GARDEN EQUIPMENT AND SUPPLIES STORES
		4970 .RET-GROCERY STORES
		4980 .RET-SPECIALTY FOOD STORES
		4990 .RET-BEER, WINE, AND LIQUOR STORES
		5070 .RET-PHARMACIES AND DRUG STORES
		5080 .RET-HEALTH AND PERSONAL CARE, EXCEPT DRUG, STORES
		5090 .RET-GASOLINE STATIONS
		5170 .RET-CLOTHING STORES
		5180 .RET-SHOE STORES
		5190 .RET-JEWELRY, LUGGAGE, AND LEATHER GOODS STORES
		5275 .RET-SPORTING GOODS, AND HOBBY AND TOY STORES
		5280 .RET-SEWING, NEEDLEWORK, AND PIECE GOODS STORES
		5295 .RET-MUSICAL INSTRUMENT AND SUPPLIES STORES
		5370 .RET-BOOK STORES AND NEWS DEALERS
		5380 .RET-DEPARTMENT AND DISCOUNT STORES
		5390 .RET-MISCELLANEOUS GENERAL MERCHANDISE STORES
		5470 .RET-FLORISTS
		5480 .RET-OFFICE SUPPLIES AND STATIONERY STORES
		5490 .RET-USED MERCHANDISE STORES
		5570 .RET-GIFT, NOVELTY, AND SOUVENIR SHOPS
		5580 .RET-MISCELLANEOUS RETAIL STORES
		5590 .RET-ELECTRONIC SHOPPING
		5591 .RET-ELECTRONIC AUCTIONS
		5592 .RET-MAIL-ORDER HOUSES
		5670 .RET-VENDING MACHINE OPERATORS
		5680 .RET-FUEL DEALERS
		5690 .RET-OTHER DIRECT SELLING ESTABLISHMENTS
		5790 .RET-NOT SPECIFIED RETAIL TRADE
		6070 .TRN-AIR TRANSPORTATION
		6080 .TRN-RAIL TRANSPORTATION
		6090 .TRN-WATER TRANSPORTATION
		6170 .TRN-TRUCK TRANSPORTATION
		6180 .TRN-BUS SERVICE AND URBAN TRANSIT
		6190 .TRN-TAXI AND LIMOUSINE SERVICE
		6270 .TRN-PIPELINE TRANSPORTATION
		6280 .TRN-SCENIC AND SIGHTSEEING TRANSPORTATION
		6290 .TRN-SERVICES INCIDENTAL TO TRANSPORTATION
		6370 .TRN-POSTAL SERVICE
		6380 .TRN-COURIERS AND MESSENGERS
		6390 .TRN-WAREHOUSING AND STORAGE
		6470 .INF-NEWSPAPER PUBLISHERS
		6480 .INF-PERIODICAL, BOOK, AND DIRECTORY PUBLISHERS
		6490 .INF-SOFTWARE PUBLISHERS
		6570 .INF-MOTION PICTURE AND VIDEO INDUSTRIES
		6590 .INF-SOUND RECORDING INDUSTRIES
		6670 .INF-BROADCASTING, EXCEPT INTERNET
		6672 .INF-INTERNET PUBLISHING AND BROADCASTING AND WEB SEARCH
		     .PORTALS
		6680 .INF-WIRED TELECOMMUNICATIONS CARRIERS
		6690 .INF-TELECOMMUNICATIONS, EXCEPT WIRED TELECOMMUNICATIONS
		     .CARRIERS
		6695 .INF-DATA PROCESSING, HOSTING, AND RELATED SERVICES
		6770 .INF-LIBRARIES AND ARCHIVES
		6780 .INF-OTHER INFORMATION SERVICES, EXCEPT LIBRARIES AND
		     .ARCHIVES, AND INTERNET PUBLISHING AND BROADCASTING AND
		     .WEB SEARCH PORTALS
		6870 .FIN-BANKING AND RELATED ACTIVITIES
		6880 .FIN-SAVINGS INSTITUTIONS, INCLUDING CREDIT UNIONS
		6890 .FIN-NONDEPOSITORY CREDIT AND RELATED ACTIVITIES
		6970 .FIN-SECURITIES, COMMODITIES, FUNDS, TRUSTS, AND OTHER
		     .FINANCIAL INVESTMENTS
		6990 .FIN-INSURANCE CARRIERS AND RELATED ACTIVITIES
		7070 .FIN-REAL ESTATE
		7080 .FIN-AUTOMOTIVE EQUIPMENT RENTAL AND LEASING
		7170 .FIN-VIDEO TAPE AND DISK RENTAL
		7180 .FIN-OTHER CONSUMER GOODS RENTAL
		7190 .FIN-COMMERCIAL, INDUSTRIAL, AND OTHER INTANGIBLE ASSETS
		     .RENTAL AND LEASING
		7270 .PRF-LEGAL SERVICES
		7280 .PRF-ACCOUNTING, TAX PREPARATION, BOOKKEEPING, AND PAYROLL
		     .SERVICES
		7290 .PRF-ARCHITECTURAL, ENGINEERING, AND RELATED SERVICES
		7370 .PRF-SPECIALIZED DESIGN SERVICES
		7380 .PRF-COMPUTER SYSTEMS DESIGN AND RELATED SERVICES
		7390 .PRF-MANAGEMENT, SCIENTIFIC, AND TECHNICAL CONSULTING
		     .SERVICES
		7460 .PRF-SCIENTIFIC RESEARCH AND DEVELOPMENT SERVICES
		7470 .PRF-ADVERTISING, PUBLIC RELATIONS,  AND RELATED SERVICES
		7480 .PRF-VETERINARY SERVICES
		7490 .PRF-OTHER PROFESSIONAL, SCIENTIFIC, AND TECHNICAL SERVICES
		7570 .PRF-MANAGEMENT OF COMPANIES AND ENTERPRISES
		7580 .PRF-EMPLOYMENT SERVICES
		7590 .PRF-BUSINESS SUPPORT SERVICES
		7670 .PRF-TRAVEL ARRANGEMENTS AND RESERVATION SERVICES
		7680 .PRF-INVESTIGATION AND SECURITY SERVICES
		7690 .PRF-SERVICES TO BUILDINGS AND DWELLINGS, EX CONSTR CLN
		7770 .PRF-LANDSCAPING SERVICES
		7780 .PRF-OTHER ADMINISTRATIVE AND OTHER SUPPORT SERVICES
		7790 .PRF-WASTE MANAGEMENT AND REMEDIATION SERVICES
		7860 .EDU-ELEMENTARY AND SECONDARY SCHOOLS
		7870 .EDU-COLLEGES, UNIVERSITIES, AND PROFESSIONAL SCHOOLS, 
		     .INCLUDING JUNIOR COLLEGES
		7880 .EDU-BUSINESS, TECHNICAL, AND TRADE SCHOOLS AND TRAINING
		7890 .EDU-OTHER SCHOOLS AND INSTRUCTION, AND EDUCATIONAL SUPPORT
		     .SERVICES
		7970 .MED-OFFICES OF PHYSICIANS
		7980 .MED-OFFICES OF DENTISTS
		7990 .MED-OFFICES OF CHIROPRACTORS
		8070 .MED-OFFICES OF OPTOMETRISTS
		8080 .MED-OFFICES OF OTHER HEALTH PRACTITIONERS
		8090 .MED-OUTPATIENT CARE CENTERS
		8170 .MED-HOME HEALTH CARE SERVICES
		8180 .MED-OTHER HEALTH CARE SERVICES
		8190 .MED-HOSPITALS
		8270 .MED-NURSING CARE FACILITIES (SKILLED NURSING FACILITIES)
		8290 .MED-RESIDENTIAL CARE FACILITIES, EXCEPT 
		     .SKILLED NURSING FACILITIES
		8370 .SCA-INDIVIDUAL AND FAMILY SERVICES
		8380 .SCA-COMMUNITY FOOD AND HOUSING, AND EMERGENCY SERVICES
		8390 .SCA-VOCATIONAL REHABILITATION SERVICES
		8470 .SCA-CHILD DAY CARE SERVICES
		8560 .ENT-PERFORMING ARTS, SPECTATOR SPORTS,
		     .AND RELATED INDUSTRIES
		8570 .ENT-MUSEUMS, ART GALLERIES, HISTORICAL SITES, AND SIMILAR
		     .INSTITUTIONS
		8580 .ENT-BOWLING CENTERS
		8590 .ENT-OTHER AMUSEMENT, GAMBLING, AND RECREATION INDUSTRIES
		8660 .ENT-TRAVELER ACCOMMODATION
		8670 .ENT-RECREATIONAL VEHICLE PARKS AND CAMPS, AND ROOMING AND
		     .BOARDING HOUSES
		8680 .ENT-RESTAURANTS AND OTHER FOOD SERVICES
		8690 .ENT-DRINKING PLACES, ALCOHOLIC BEVERAGES
		8770 .SRV-AUTOMOTIVE REPAIR AND MAINTENANCE
		8780 .SRV-CAR WASHES
		8790 .SRV-ELECTRONIC AND PRECISION EQUIPMENT REPAIR AND
		     .MAINTENANCE
		8870 .SRV-COMMERCIAL AND INDUSTRIAL MACHINERY AND EQUIPMENT
		     .REPAIR AND MAINTENANCE
		8880 .SRV-PERSONAL AND HOUSEHOLD GOODS REPAIR AND MAINTENANCE
		8970 .SRV-BARBER SHOPS
		8980 .SRV-BEAUTY SALONS
		8990 .SRV-NAIL SALONS AND OTHER PERSONAL CARE SERVICES
		9070 .SRV-DRYCLEANING AND LAUNDRY SERVICES
		9080 .SRV-FUNERAL HOMES, CEMETERIES AND CREMATORIES
		9090 .SRV-OTHER PERSONAL SERVICES
		9160 .SRV-RELIGIOUS ORGANIZATIONS
		9170 .SRV-CIVIC, SOCIAL, ADVOCACY ORGANIZATIONS, AND GRANTMAKING
		     .AND GIVING SERVICES
		9180 .SRV-LABOR UNIONS
		9190 .SRV-BUSINESS, PROFESSIONAL, POLITICAL, AND SIMILAR
		     .ORGANIZATIONS
		9290 .SRV-PRIVATE HOUSEHOLDS
		9370 .ADM-EXECUTIVE OFFICES AND LEGISLATIVE BODIES
		9380 .ADM-PUBLIC FINANCE ACTIVITIES
		9390 .ADM-OTHER GENERAL GOVERNMENT AND SUPPORT
		9470 .ADM-JUSTICE, PUBLIC ORDER, AND SAFETY ACTIVITIES
		9480 .ADM-ADMINISTRATION OF HUMAN RESOURCE PROGRAMS
		9490 .ADM-ADMINISTRATION OF ENVIRONMENTAL QUALITY AND HOUSING
		     .PROGRAMS
		9570 .ADM-ADMINISTRATION OF ECONOMIC PROGRAMS AND SPACE RESEARCH
		9590 .ADM-NATIONAL SECURITY AND INTERNATIONAL AFFAIRS
		9670 .MIL-U.S. ARMY
		9680 .MIL-U.S. AIR FORCE
		9690 .MIL-U.S. NAVY
		9770 .MIL-U.S. MARINES
		9780 .MIL-U.S. COAST GUARD
		9790 .MIL-U.S. ARMED FORCES, BRANCH NOT SPECIFIED
		9870 .MIL-MILITARY RESERVES OR NATIONAL GUARD
		9920 .UNEMPLOYED AND LAST WORKED 5 YEARS AGO OR EARLIER OR NEVER
		     .WORKED**

Note: For additional information on NAICS and SOC groupings within major categories visit our website at: https://www.census.gov/people/io/methodology/indexes.html.	

JWAP		3	
	Time of arrival at work - hour and minute
		bbb .N/A (not a worker; worker who worked at home)
		001 .12:00 a.m. to 12:04 a.m.
		002 .12:05 a.m. to 12:09 a.m.
		003 .12:10 a.m. to 12:14 a.m.
		004 .12:15 a.m. to 12:19 a.m.
		005 .12:20 a.m. to 12:24 a.m.
		006 .12:25 a.m. to 12:29 a.m.
		007 .12:30 a.m. to 12:39 a.m.
		008 .12:40 a.m. to 12:44 a.m.
		009 .12:45 a.m. to 12:49 a.m.
		010 .12:50 a.m. to 12:59 a.m.
		011 .1:00 a.m. to 1:04 a.m.
		012 .1:05 a.m. to 1:09 a.m.
		013 .1:10 a.m. to 1:14 a.m.
		014 .1:15 a.m. to 1:19 a.m.
		015 .1:20 a.m. to 1:24 a.m.
		016 .1:25 a.m. to 1:29 a.m.
		017 .1:30 a.m. to 1:34 a.m.
		018 .1:35 a.m. to 1:39 a.m.
		019 .1:40 a.m. to 1:44 a.m.
		020 .1:45 a.m. to 1:49 a.m.
		021 .1:50 a.m. to 1:59 a.m.
		022 .2:00 a.m. to 2:04 a.m.
		023 .2:05 a.m. to 2:09 a.m.
		024 .2:10 a.m. to 2:14 a.m.
		025 .2:15 a.m. to 2:19 a.m.
		026 .2:20 a.m. to 2:24 a.m.
		027 .2:25 a.m. to 2:29 a.m.
		028 .2:30 a.m. to 2:34 a.m.
		029 .2:35 a.m. to 2:39 a.m.
		030 .2:40 a.m. to 2:44 a.m.
		031 .2:45 a.m. to 2:49 a.m.
		032 .2:50 a.m. to 2:54 a.m.
		033 .2:55 a.m. to 2:59 a.m.
		034 .3:00 a.m. to 3:04 a.m.
		035 .3:05 a.m. to 3:09 a.m.
		036 .3:10 a.m. to 3:14 a.m.
		037 .3:15 a.m. to 3:19 a.m.
		038 .3:20 a.m. to 3:24 a.m.
		039 .3:25 a.m. to 3:29 a.m.
		040 .3:30 a.m. to 3:34 a.m.
		041 .3:35 a.m. to 3:39 a.m.
		042 .3:40 a.m. to 3:44 a.m.
		043 .3:45 a.m. to 3:49 a.m.
		044 .3:50 a.m. to 3:54 a.m.
		045 .3:55 a.m. to 3:59 a.m.
		046 .4:00 a.m. to 4:04 a.m.
		047 .4:05 a.m. to 4:09 a.m.
		048 .4:10 a.m. to 4:14 a.m.
		049 .4:15 a.m. to 4:19 a.m.
		050 .4:20 a.m. to 4:24 a.m.
		051 .4:25 a.m. to 4:29 a.m.
		052 .4:30 a.m. to 4:34 a.m.
		053 .4:35 a.m. to 4:39 a.m.
		054 .4:40 a.m. to 4:44 a.m.
		055 .4:45 a.m. to 4:49 a.m.
		056 .4:50 a.m. to 4:54 a.m.
		057 .4:55 a.m. to 4:59 a.m.
		058 .5:00 a.m. to 5:04 a.m.
		059 .5:05 a.m. to 5:09 a.m.
		060 .5:10 a.m. to 5:14 a.m.
		061 .5:15 a.m. to 5:19 a.m.
		062 .5:20 a.m. to 5:24 a.m.
		063 .5:25 a.m. to 5:29 a.m.
		064 .5:30 a.m. to 5:34 a.m.
		065 .5:35 a.m. to 5:39 a.m.
		066 .5:40 a.m. to 5:44 a.m.
		067 .5:45 a.m. to 5:49 a.m.
		068 .5:50 a.m. to 5:54 a.m.
		069 .5:55 a.m. to 5:59 a.m.
		070 .6:00 a.m. to 6:04 a.m.
		071 .6:05 a.m. to 6:09 a.m.
		072 .6:10 a.m. to 6:14 a.m.
		073 .6:15 a.m. to 6:19 a.m.
		074 .6:20 a.m. to 6:24 a.m.
		075 .6:25 a.m. to 6:29 a.m.
		076 .6:30 a.m. to 6:34 a.m.
		077 .6:35 a.m. to 6:39 a.m.
		078 .6:40 a.m. to 6:44 a.m.
		079 .6:45 a.m. to 6:49 a.m.
		080 .6:50 a.m. to 6:54 a.m.
		081 .6:55 a.m. to 6:59 a.m.
		082 .7:00 a.m. to 7:04 a.m.
		083 .7:05 a.m. to 7:09 a.m.
		084 .7:10 a.m. to 7:14 a.m.
		085 .7:15 a.m. to 7:19 a.m.
		086 .7:20 a.m. to 7:24 a.m.
		087 .7:25 a.m. to 7:29 a.m.
		088 .7:30 a.m. to 7:34 a.m.
		089 .7:35 a.m. to 7:39 a.m.
		090 .7:40 a.m. to 7:44 a.m.
		091 .7:45 a.m. to 7:49 a.m.
		092 .7:50 a.m. to 7:54 a.m.
		093 .7:55 a.m. to 7:59 a.m.
		094 .8:00 a.m. to 8:04 a.m.
		095 .8:05 a.m. to 8:09 a.m.
		096 .8:10 a.m. to 8:14 a.m.
		097 .8:15 a.m. to 8:19 a.m.
		098 .8:20 a.m. to 8:24 a.m.
		099 .8:25 a.m. to 8:29 a.m.
		100 .8:30 a.m. to 8:34 a.m.
		101 .8:35 a.m. to 8:39 a.m.
		102 .8:40 a.m. to 8:44 a.m.
		103 .8:45 a.m. to 8:49 a.m.
		104 .8:50 a.m. to 8:54 a.m.
		105 .8:55 a.m. to 8:59 a.m.
		106 .9:00 a.m. to 9:04 a.m.
		107 .9:05 a.m. to 9:09 a.m.
		108 .9:10 a.m. to 9:14 a.m.
		109 .9:15 a.m. to 9:19 a.m.
		110 .9:20 a.m. to 9:24 a.m.
		111 .9:25 a.m. to 9:29 a.m.
		112 .9:30 a.m. to 9:34 a.m.
		113 .9:35 a.m. to 9:39 a.m.
		114 .9:40 a.m. to 9:44 a.m.
		115 .9:45 a.m. to 9:49 a.m.
		116 .9:50 a.m. to 9:54 a.m.
		117 .9:55 a.m. to 9:59 a.m.
		118 .10:00 a.m. to 10:04 a.m.
		119 .10:05 a.m. to 10:09 a.m.
		120 .10:10 a.m. to 10:14 a.m.
		121 .10:15 a.m. to 10:19 a.m.
		122 .10:20 a.m. to 10:24 a.m.
		123 .10:25 a.m. to 10:29 a.m.
		124 .10:30 a.m. to 10:34 a.m.
		125 .10:35 a.m. to 10:39 a.m.
		126 .10:40 a.m. to 10:44 a.m.
		127 .10:45 a.m. to 10:49 a.m.
		128 .10:50 a.m. to 10:54 a.m.
		129 .10:55 a.m. to 10:59 a.m.
		130 .11:00 a.m. to 11:04 a.m.
		131 .11:05 a.m. to 11:09 a.m.
		132 .11:10 a.m. to 11:14 a.m.
		133 .11:15 a.m. to 11:19 a.m.
		134 .11:20 a.m. to 11:24 a.m.
		135 .11:25 a.m. to 11:29 a.m.
		136 .11:30 a.m. to 11:34 a.m.
		137 .11:35 a.m. to 11:39 a.m.
		138 .11:40 a.m. to 11:44 a.m.
		139 .11:45 a.m. to 11:49 a.m.
		140 .11:50 a.m. to 11:54 a.m.
		141 .11:55 a.m. to 11:59 a.m.
		142 .12:00 p.m. to 12:04 p.m.
		143 .12:05 p.m. to 12:09 p.m.
		144 .12:10 p.m. to 12:14 p.m.
		145 .12:15 p.m. to 12:19 p.m.
		146 .12:20 p.m. to 12:24 p.m.
		147 .12:25 p.m. to 12:29 p.m.
		148 .12:30 p.m. to 12:34 p.m.
		149 .12:35 p.m. to 12:39 p.m.
		150 .12:40 p.m. to 12:44 p.m.
		151 .12:45 p.m. to 12:49 p.m.
		152 .12:50 p.m. to 12:54 p.m.
		153 .12:55 p.m. to 12:59 p.m.
		154 .1:00 p.m. to 1:04 p.m.
		155 .1:05 p.m. to 1:09 p.m.
		156 .1:10 p.m. to 1:14 p.m.
		157 .1:15 p.m. to 1:19 p.m.
		158 .1:20 p.m. to 1:24 p.m.
		159 .1:25 p.m. to 1:29 p.m.
		160 .1:30 p.m. to 1:34 p.m.
		161 .1:35 p.m. to 1:39 p.m.
		162 .1:40 p.m. to 1:44 p.m.
		163 .1:45 p.m. to 1:49 p.m.
		164 .1:50 p.m. to 1:54 p.m.
		165 .1:55 p.m. to 1:59 p.m.
		166 .2:00 p.m. to 2:04 p.m.
		167 .2:05 p.m. to 2:09 p.m.
		168 .2:10 p.m. to 2:14 p.m.
		169 .2:15 p.m. to 2:19 p.m.
		170 .2:20 p.m. to 2:24 p.m.
		171 .2:25 p.m. to 2:29 p.m.
		172 .2:30 p.m. to 2:34 p.m.
		173 .2:35 p.m. to 2:39 p.m.
		174 .2:40 p.m. to 2:44 p.m.
		175 .2:45 p.m. to 2:49 p.m.
		176 .2:50 p.m. to 2:54 p.m.
		177 .2:55 p.m. to 2:59 p.m.
		178 .3:00 p.m. to 3:04 p.m.
		179 .3:05 p.m. to 3:09 p.m.
		180 .3:10 p.m. to 3:14 p.m.
		181 .3:15 p.m. to 3:19 p.m.
		182 .3:20 p.m. to 3:24 p.m.
		183 .3:25 p.m. to 3:29 p.m.
		184 .3:30 p.m. to 3:34 p.m.
		185 .3:35 p.m. to 3:39 p.m.
		186 .3:40 p.m. to 3:44 p.m.
		187 .3:45 p.m. to 3:49 p.m.
		188 .3:50 p.m. to 3:54 p.m.
		189 .3:55 p.m. to 3:59 p.m.
		190 .4:00 p.m. to 4:04 p.m.
		191 .4:05 p.m. to 4:09 p.m.
		192 .4:10 p.m. to 4:14 p.m.
		193 .4:15 p.m. to 4:19 p.m.
		194 .4:20 p.m. to 4:24 p.m.
		195 .4:25 p.m. to 4:29 p.m.
		196 .4:30 p.m. to 4:34 p.m.
		197 .4:35 p.m. to 4:39 p.m.
		198 .4:40 p.m. to 4:44 p.m.
		199 .4:45 p.m. to 4:49 p.m.
		200 .4:50 p.m. to 4:54 p.m.
		201 .4:55 p.m. to 4:59 p.m.
		202 .5:00 p.m. to 5:04 p.m.
		203 .5:05 p.m. to 5:09 p.m.
		204 .5:10 p.m. to 5:14 p.m.
		205 .5:15 p.m. to 5:19 p.m.
		206 .5:20 p.m. to 5:24 p.m.
		207 .5:25 p.m. to 5:29 p.m.
		208 .5:30 p.m. to 5:34 p.m.
		209 .5:35 p.m. to 5:39 p.m.
		210 .5:40 p.m. to 5:44 p.m.
		211 .5:45 p.m. to 5:49 p.m.
		212 .5:50 p.m. to 5:54 p.m.
		213 .5:55 p.m. to 5:59 p.m.
		214 .6:00 p.m. to 6:04 p.m.
		215 .6:05 p.m. to 6:09 p.m.
		216 .6:10 p.m. to 6:14 p.m.
		217 .6:15 p.m. to 6:19 p.m.
		218 .6:20 p.m. to 6:24 p.m.
		219 .6:25 p.m. to 6:29 p.m.
		220 .6:30 p.m. to 6:34 p.m.
		221 .6:35 p.m. to 6:39 p.m.
		222 .6:40 p.m. to 6:44 p.m.
		223 .6:45 p.m. to 6:49 p.m.
		224 .6:50 p.m. to 6:54 p.m.
		225 .6:55 p.m. to 6:59 p.m.
		226 .7:00 p.m. to 7:04 p.m.
		227 .7:05 p.m. to 7:09 p.m.
		228 .7:10 p.m. to 7:14 p.m.
		229 .7:15 p.m. to 7:19 p.m.
		230 .7:20 p.m. to 7:24 p.m.
		231 .7:25 p.m. to 7:29 p.m.
		232 .7:30 p.m. to 7:34 p.m.
		233 .7:35 p.m. to 7:39 p.m.
		234 .7:40 p.m. to 7:44 p.m.
		235 .7:45 p.m. to 7:49 p.m.
		236 .7:50 p.m. to 7:54 p.m.
		237 .7:55 p.m. to 7:59 p.m.
		238 .8:00 p.m. to 8:04 p.m.
		239 .8:05 p.m. to 8:09 p.m.
		240 .8:10 p.m. to 8:14 p.m.
		241 .8:15 p.m. to 8:19 p.m.
		242 .8:20 p.m. to 8:24 p.m.
		243 .8:25 p.m. to 8:29 p.m.
		244 .8:30 p.m. to 8:34 p.m.
		245 .8:35 p.m. to 8:39 p.m.
		246 .8:40 p.m. to 8:44 p.m.
		247 .8:45 p.m. to 8:49 p.m.
		248 .8:50 p.m. to 8:54 p.m.
		249 .8:55 p.m. to 8:59 p.m.
		250 .9:00 p.m. to 9:04 p.m.
		251 .9:05 p.m. to 9:09 p.m.
		252 .9:10 p.m. to 9:14 p.m.
		253 .9:15 p.m. to 9:19 p.m.
		254 .9:20 p.m. to 9:24 p.m.
		255 .9:25 p.m. to 9:29 p.m.
		256 .9:30 p.m. to 9:34 p.m.
		257 .9:35 p.m. to 9:39 p.m.
		258 .9:40 p.m. to 9:44 p.m.
		259 .9:45 p.m. to 9:49 p.m.
		260 .9:50 p.m. to 9:54 p.m.
		261 .9:55 p.m. to 9:59 p.m.
		262 .10:00 p.m. to 10:04 p.m.
		263 .10:05 p.m. to 10:09 p.m.
		264 .10:10 p.m. to 10:14 p.m.
		265 .10:15 p.m. to 10:19 p.m.
		266 .10:20 p.m. to 10:24 p.m.
		267 .10:25 p.m. to 10:29 p.m.
		268 .10:30 p.m. to 10:34 p.m.
		269 .10:35 p.m. to 10:39 p.m.
		270 .10:40 p.m. to 10:44 p.m.
		271 .10:45 p.m. to 10:49 p.m.
		272 .10:50 p.m. to 10:55 p.m.
		273 .10:55 p.m. to 10:59 p.m.
		274 .11:00 p.m. to 11:04 p.m.
		275 .11:05 p.m. to 11:09 p.m.
		276 .11:10 p.m. to 11:14 p.m.
		277 .11:15 p.m. to 11:19 p.m.
		278 .11:20 p.m. to 11:24 p.m.
		279 .11:25 p.m. to 11:29 p.m.
		280 .11:30 p.m. to 11:34 p.m.
		281 .11:35 p.m. to 11:39 p.m.
		282 .11:40 p.m. to 11:44 p.m.
		283 .11:45 p.m. to 11:49 p.m.
		284 .11:50 p.m. to 11:54 p.m.
		285 .11:55 p.m. to 11:59 p.m.

JWDP		3	
	Time of departure for work - hour and minute
		bbb .N/A (not a worker; worker who worked at home)
		001 .12:00 a.m. to 12:29 a.m.
		002 .12:30 a.m. to 12:59 a.m.
		003 .1:00 a.m. to 1:29 a.m.
		004 .1:30 a.m. to 1:59 a.m.
		005 .2:00 a.m. to 2:29 a.m.
		006 .2:30 a.m. to 2:59 a.m.
		007 .3:00 a.m. to 3:09 a.m.
		008 .3:10 a.m. to 3:19 a.m.
		009 .3:20 a.m. to 3:29 a.m.
		010 .3:30 a.m. to 3:39 a.m.
		011 .3:40 a.m. to 3:49 a.m.
		012 .3:50 a.m. to 3:59 a.m.
		013 .4:00 a.m. to 4:09 a.m.
		014 .4:10 a.m. to 4:19 a.m.
		015 .4:20 a.m. to 4:29 a.m.
		016 .4:30 a.m. to 4:39 a.m.
		017 .4:40 a.m. to 4:49 a.m.
		018 .4:50 a.m. to 4:59 a.m.
		019 .5:00 a.m. to 5:04 a.m.
		020 .5:05 a.m. to 5:09 a.m.
		021 .5:10 a.m. to 5:14 a.m.
		022 .5:15 a.m. to 5:19 a.m.
		023 .5:20 a.m. to 5:24 a.m.
		024 .5:25 a.m. to 5:29 a.m.
		025 .5:30 a.m. to 5:34 a.m.
		026 .5:35 a.m. to 5:39 a.m.
		027 .5:40 a.m. to 5:44 a.m.
		028 .5:45 a.m. to 5:49 a.m.
		029 .5:50 a.m. to 5:54 a.m.
		030 .5:55 a.m. to 5:59 a.m.
		031 .6:00 a.m. to 6:04 a.m.
		032 .6:05 a.m. to 6:09 a.m.
		033 .6:10 a.m. to 6:14 a.m.
		034 .6:15 a.m. to 6:19 a.m.
		035 .6:20 a.m. to 6:24 a.m.
		036 .6:25 a.m. to 6:29 a.m.
		037 .6:30 a.m. to 6:34 a.m.
		038 .6:35 a.m. to 6:39 a.m.
		039 .6:40 a.m. to 6:44 a.m.
		040 .6:45 a.m. to 6:49 a.m.
		041 .6:50 a.m. to 6:54 a.m.
		042 .6:55 a.m. to 6:59 a.m.
		043 .7:00 a.m. to 7:04 a.m.
		044 .7:05 a.m. to 7:09 a.m.
		045 .7:10 a.m. to 7:14 a.m.
		046 .7:15 a.m. to 7:19 a.m.
		047 .7:20 a.m. to 7:24 a.m.
		048 .7:25 a.m. to 7:29 a.m.
		049 .7:30 a.m. to 7:34 a.m.
		050 .7:35 a.m. to 7:39 a.m.
		051 .7:40 a.m. to 7:44 a.m.
		052 .7:45 a.m. to 7:49 a.m.
		053 .7:50 a.m. to 7:54 a.m.
		054 .7:55 a.m. to 7:59 a.m.
		055 .8:00 a.m. to 8:04 a.m.
		056 .8:05 a.m. to 8:09 a.m.
		057 .8:10 a.m. to 8:14 a.m.
		058 .8:15 a.m. to 8:19 a.m.
		059 .8:20 a.m. to 8:24 a.m.
		060 .8:25 a.m. to 8:29 a.m.
		061 .8:30 a.m. to 8:34 a.m.
		062 .8:35 a.m. to 8:39 a.m.
		063 .8:40 a.m. to 8:44 a.m.
		064 .8:45 a.m. to 8:49 a.m.
		065 .8:50 a.m. to 8:54 a.m.
		066 .8:55 a.m. to 8:59 a.m.
		067 .9:00 a.m. to 9:04 a.m.
		068 .9:05 a.m. to 9:09 a.m.
		069 .9:10 a.m. to 9:14 a.m.
		070 .9:15 a.m. to 9:19 a.m.
		071 .9:20 a.m. to 9:24 a.m.
		072 .9:25 a.m. to 9:29 a.m.
		073 .9:30 a.m. to 9:34 a.m.
		074 .9:35 a.m. to 9:39 a.m.
		075 .9:40 a.m. to 9:44 a.m.
		076 .9:45 a.m. to 9:49 a.m.
		077 .9:50 a.m. to 9:54 a.m.
		078 .9:55 a.m. to 9:59 a.m.
		079 .10:00 a.m. to 10:09 a.m.
		080 .10:10 a.m. to 10:19 a.m.
		081 .10:20 a.m. to 10:29 a.m.
		082 .10:30 a.m. to 10:39 a.m.
		083 .10:40 a.m. to 10:49 a.m.
		084 .10:50 a.m. to 10:59 a.m.
		085 .11:00 a.m. to 11:09 a.m.
		086 .11:10 a.m. to 11:19 a.m.
		087 .11:20 a.m. to 11:29 a.m.
		088 .11:30 a.m. to 11:39 a.m.
		089 .11:40 a.m. to 11:49 a.m.
		090 .11:50 a.m. to 11:59 a.m.
		091 .12:00 p.m. to 12:09 p.m.
		092 .12:10 p.m. to 12:19 p.m.
		093 .12:20 p.m. to 12:29 p.m.
		094 .12:30 p.m. to 12:39 p.m.
		095 .12:40 p.m. to 12:49 p.m.
		096 .12:50 p.m. to 12:59 p.m.
		097 .1:00 p.m. to 1:09 p.m.
		098 .1:10 p.m. to 1:19 p.m.
		099 .1:20 p.m. to 1:29 p.m.
		100 .1:30 p.m. to 1:39 p.m.
		101 .1:40 p.m. to 1:49 p.m.
		102 .1:50 p.m. to 1:59 p.m.
		103 .2:00 p.m. to 2:09 p.m.
		104 .2:10 p.m. to 2:19 p.m.
		105 .2:20 p.m. to 2:29 p.m.
		106 .2:30 p.m. to 2:39 p.m.
		107 .2:40 p.m. to 2:49 p.m.
		108 .2:50 p.m. to 2:59 p.m.
		109 .3:00 p.m. to 3:09 p.m.
		110 .3:10 p.m. to 3:19 p.m.
		111 .3:20 p.m. to 3:29 p.m.
		112 .3:30 p.m. to 3:39 p.m.
		113 .3:40 p.m. to 3:49 p.m.
		114 .3:50 p.m. to 3:59 p.m.
		115 .4:00 p.m. to 4:09 p.m.
		116 .4:10 p.m. to 4:19 p.m.
		117 .4:20 p.m. to 4:29 p.m.
		118 .4:30 p.m. to 4:39 p.m.
		119 .4:40 p.m. to 4:49 p.m.
		120 .4:50 p.m. to 4:59 p.m.
		121 .5:00 p.m. to 5:09 p.m.
		122 .5:10 p.m. to 5:19 p.m.
		123 .5:20 p.m. to 5:29 p.m.
		124 .5:30 p.m. to 5:39 p.m.
		125 .5:40 p.m. to 5:49 p.m.
		126 .5:50 p.m. to 5:59 p.m.
		127 .6:00 p.m. to 6:09 p.m.
		128 .6:10 p.m. to 6:19 p.m.
		129 .6:20 p.m. to 6:29 p.m.
		130 .6:30 p.m. to 6:39 p.m.
		131 .6:40 p.m. to 6:49 p.m.
		132 .6:50 p.m. to 6:59 p.m.
		133 .7:00 p.m. to 7:29 p.m.
		134 .7:30 p.m. to 7:59 p.m.
		135 .8:00 p.m. to 8:29 p.m.
		136 .8:30 p.m. to 8:59 p.m.
		137 .9:00 p.m. to 9:09 p.m.
		138 .9:10 p.m. to 9:19 p.m.
		139 .9:20 p.m. to 9:29 p.m.
		140 .9:30 p.m. to 9:39 p.m.
		141 .9:40 p.m. to 9:49 p.m.
		142 .9:50 p.m. to 9:59 p.m.
		143 .10:00 p.m. to 10:09 p.m.
		144 .10:10 p.m. to 10:19 p.m.
		145 .10:20 p.m. to 10:29 p.m.
		146 .10:30 p.m. to 10:39 p.m.
		147 .10:40 p.m. to 10:49 p.m.
		148 .10:50 p.m. to 10:59 p.m.
		149 .11:00 p.m. to 11:29 p.m.
		150 .11:30 p.m. to 11:59 p.m.

LANP		3	
	Language spoken at home
		bbb .N/A (less than 5 years old/speaks only English) 
		601 .Jamaican Creole
		602 .Krio
		607 .German
		608 .Pennsylvania Dutch
		609 .Yiddish
		610 .Dutch
		611 .Afrikaans
		614 .Swedish
		615 .Danish
		616 .Norwegian
		619 .Italian
		620 .French
		622 .Patois
		623 .French Creole
		624 .Cajun
		625 .Spanish
		629 .Portuguese
		631 .Romanian
		635 .Irish Gaelic
		637 .Greek
		638 .Albanian
		639 .Russian
		641 .Ukrainian
		642 .Czech
		645 .Polish
		646 .Slovak
		647 .Bulgarian
		648 .Macedonian
		649 .Serbo-Croatian
		650 .Croatian
		651 .Serbian
		653 .Lithuanian
		654 .Latvian
		655 .Armenian
		656 .Persian
		657 .Pashto
		658 .Kurdish
		662 .India N.E.C.
		663 .Hindi
		664 .Bengali
		665 .Panjabi
		666 .Marathi
		667 .Gujarati
		671 .Urdu
		674 .Nepali
		675 .Sindhi
		676 .Pakistan N.E.C.
		677 .Sinhalese
		679 .Finnish
		682 .Hungarian
		689 .Uighur
		691 .Turkish
		694 .Mongolian
		701 .Telugu
		702 .Kannada
		703 .Malayalam
		704 .Tamil
		708 .Chinese
		711 .Cantonese
		712 .Mandarin
		714 .Formosan
		717 .Burmese
		720 .Thai
		721 .Mien
		722 .Hmong
		723 .Japanese
		724 .Korean
		725 .Laotian
		726 .Mon-Khmer, Cambodian
		728 .Vietnamese
		732 .Indonesian
		739 .Malay
		742 .Tagalog
		743 .Bisayan
		744 .Sebuano
		746 .Ilocano
		750 .Micronesian
		752 .Chamorro
		761 .Trukese
		767 .Samoan
		768 .Tongan
		776 .Hawaiian
		777 .Arabic
		778 .Hebrew
		779 .Syriac
		780 .Amharic
		783 .Cushite
		791 .Swahili
		792 .Bantu
		793 .Mande
		794 .Fulani
		796 .Kru, Ibo, Yoruba
		799 .African
		806 .Other Algonquian languages
		819 .Ojibwa
		862 .Apache
		864 .Navajo
		907 .Dakota
		924 .Keres
		933 .Cherokee
		964 .Zuni
		985 .Other Indo-European languages
		986 .Other Asian languages
		988 .Other Pacific Island languages
		989 .Other specified African languages
		990 .Aleut-Eskimo languages
		992 .South/Central American Indian languages
		993 .Other Specified North American Indian languages
		994 .Other languages
		996 .Language not specified

MIGPUMA		5	
	Migration PUMA based on 2010 Census definition
		bbbbb 	     .N/A (person less than 1 year old/lived in		
			     .same house 1 year ago) 
		00001 	     .Did not live in the United States or in Puerto
			     .Rico one year ago
		00002 	     .Lived in Puerto Rico one year ago and current
			     .residence is in the U.S.
		00100..70100 .Assigned Migration PUMA. Use with MIGSP.

MIGSP		3	
	Migration recode - State or foreign country code
		bbb .N/A (person less than 1 year old/lived in		
		    .same house 1 year ago)
		001 .Alabama/AL
		002 .Alaska/AK
		004 .Arizona/AZ
		005 .Arkansas/AR
		006 .California/CA
		008 .Colorado/CO
		009 .Connecticut/CT
		010 .Delaware/DE
		011 .District of Columbia/DC
		012 .Florida/FL
		013 .Georgia/GA
		015 .Hawaii/HI
		016 .Idaho/ID
		017 .Illinois/IL
		018 .Indiana/IN
		019 .Iowa/IA
		020 .Kansas/KS
		021 .Kentucky/KY
		022 .Louisiana/LA
		023 .Maine/ME
		024 .Maryland/MD
		025 .Massachusetts/MA
		026 .Michigan/MI
		027 .Minnesota/MN
		028 .Mississippi/MS
		029 .Missouri/MO
		030 .Montana/MT
		031 .Nebraska/NE
		032 .Nevada/NV
		033 .New Hampshire/NH
		034 .New Jersey/NJ
		035 .New Mexico/NM
		036 .New York/NY
		037 .North Carolina/NC
		038 .North Dakota/ND
		039 .Ohio/OH
		040 .Oklahoma/OK
		041 .Oregon/OR
		042 .Pennsylvania/PA
		044 .Rhode Island/RI
		045 .South Carolina/SC
		046 .South Dakota/SD
		047 .Tennessee/TN
		048 .Texas/TX
		049 .Utah/UT
		050 .Vermont/VT
		051 .Virginia/VA
		053 .Washington/WA
		054 .West Virginia/WV
		055 .Wisconsin/WI
		056 .Wyoming/WY
		072 .Puerto Rico
		109 .France
		110 .Germany
		111 .Northern Europe, Not Specified
		113 .Eastern Europe, Not Specified
		114 .Western Europe or Other Europe, Not Specified
		120 .Italy
		134 .Spain
		138 .United Kingdom, Excluding England
		139 .England
		163 .Russia
		200 .Afghanistan
		207 .China, Hong Kong, Macau & Paracel Islands
		210 .India
		213 .Iraq
		215 .Japan
		217 .Korea
		229 .Nepal
		231 .Pakistan
		233 .Philippines
		235 .Saudi Arabia
		240 .Taiwan
		242 .Thailand
		243 .Turkey
		247 .Vietnam
		251 .Eastern Asia, Not Specified
		252 .Western Asia, Not Specified
		253 .South Central Asia or Asia, Not Specified
		301 .Canada
		303 .Mexico
		312 .El Salvador
		313 .Guatemala
		314 .Honduras
		317 .Central America, Not Specified
		327 .Cuba
		329 .Dominican Republic
		332 .Haiti
		333 .Jamaica
		344 .Caribbean and North America, Not Specified
		362 .Brazil
		364 .Colombia
		374 .South America, Not Specified
		414 .Egypt
		440 .Nigeria
		463 .Eastern Africa, Not Specified
		467 .Western Africa, Not Specified
		468 .Northern Africa or Other Africa, Not Specified
		501 .Australia
		555 .Other US Island Areas, Oceania, Not Specified, or At Sea

MSP		1	
	Married, spouse present/spouse absent
		b .N/A (age less than 15 years)
		1 .Now married, spouse present
		2 .Now married, spouse absent
		3 .Widowed
		4 .Divorced
		5 .Separated
		6 .Never married

NAICSP	8	
	NAICS Industry recode for 2013 and later based on 2012 NAICS codes  
		bbbbbbbb.N/A (less than 16 years old/NILF who last worked
			.more than 5 years ago or never worked)
		111	.AGR-CROP PRODUCTION
		112	.AGR-ANIMAL PRODUCTION AND AQUACULTURE
		1133	.AGR-LOGGING
		113M	.AGR-FORESTRY EXCEPT LOGGING
		114	.AGR-FISHING, HUNTING, AND TRAPPING
		115	.AGR-SUPPORT ACTIVITIES FOR AGRICULTURE AND FORESTRY
		211	.EXT-OIL AND GAS EXTRACTION
		2121	.EXT-COAL MINING
		2122	.EXT-METAL ORE MINING
		2123	.EXT-NONMETALLIC MINERAL MINING AND QUARRYING
		213	.EXT-SUPPORT ACTIVITIES FOR MINING
		2211P	.UTL-ELECTRIC POWER GENERATION, TRANSMISSION AND
			.DISTRIBUTION
		2212P	.UTL-NATURAL GAS DISTRIBUTION
		22132	.UTL-SEWAGE TREATMENT FACILITIES
		2213M	.UTL-WATER, STEAM, AIR CONDITIONING, AND IRRIGATION
			.SYSTEMS
		221MP	.UTL-ELECTRIC AND GAS, AND OTHER COMBINATIONS
		22S	.UTL-NOT SPECIFIED UTILITIES
		23	.CON-CONSTRUCTION, INCL CLEANING DURING AND IMM AFTER
		3113	.MFG-SUGAR AND CONFECTIONERY PRODUCTS
		3114	.MFG-FRUIT AND VEGETABLE PRESERVING AND SPECIALTY FOODS
		3115	.MFG-DAIRY PRODUCTS
		3116	.MFG-ANIMAL SLAUGHTERING AND PROCESSING
		311811  .MFG-RETAIL BAKERIES
		3118Z	.MFG-BAKERIES AND TORTILLA, EXCEPT RETAIL BAKERIES
		311M1	.MFG-ANIMAL FOOD, GRAIN AND OILSEED MILLING
		311M2	.MFG-SEAFOOD AND OTHER MISCELLANEOUS FOODS, N.E.C.
		311S	.MFG-NOT SPECIFIED FOOD INDUSTRIES
		3121	.MFG-BEVERAGE
		3122	.MFG-TOBACCO
		3131	.MFG-FIBER, YARN, AND THREAD MILLS
		3132Z	.MFG-FABRIC MILLS, EXCEPT KNITTING
		3133	.MFG-TEXTILE AND FABRIC FINISHING AND FABRIC COATING 
       			.MILLS
		31411	.MFG-CARPET AND RUG MILLS
		314Z	.MFG-TEXTILE PRODUCT MILLS, EXCEPT CARPET AND RUG
		3152	.MFG-CUT AND SEW APPAREL
		3159	.MFG-APPAREL ACCESSORIES AND OTHER APPAREL
		3162	.MFG-FOOTWEAR
		316M	.MFG-LEATHER TANNING AND FINISHING AND OTHER ALLIED
			.PRODUCTS MANUFACTURING
		31M	.MFG-KNITTING FABRIC MILLS, AND APPAREL KNITTING MILLS
		3211	.MFG-SAWMILLS AND WOOD PRESERVATION
		3212	.MFG-VENEER, PLYWOOD, AND ENGINEERED WOOD PRODUCTS
		32199M	.MFG-PREFABRICATED WOOD BUILDINGS AND MOBILE HOMES
		3219ZM	.MFG-MISCELLANEOUS WOOD PRODUCTS
		3221	.MFG-PULP, PAPER, AND PAPERBOARD MILLS
		32221	.MFG-PAPERBOARD CONTAINER
		3222M	.MFG-MISCELLANEOUS PAPER AND PULP PRODUCTS
		3231	.MFG-PRINTING AND RELATED SUPPORT ACTIVITIES
		3241M	.MFG-MISCELLANEOUS PETROLEUM AND COAL PRODUCTS
       		32411	.MFG-PETROLEUM REFINING
		3252	.MFG-RESIN, SYNTHETIC RUBBER, AND FIBERS AND FILAMENTS
		3253	.MFG-AGRICULTURAL CHEMICALS
		3254	.MFG-PHARMACEUTICALS AND MEDICINES
		3255	.MFG-PAINT, COATING, AND ADHESIVES
		3256	.MFG-SOAP, CLEANING COMPOUND, AND COSMETICS
		325M	.MFG-INDUSTRIAL AND MISCELLANEOUS CHEMICALS
		3261	.MFG-PLASTICS PRODUCTS
		32621	.MFG-TIRES
		3262M	.MFG-RUBBER PRODUCTS, EXCEPT TIRES
		32711	.MFG-POTTERY, CERAMICS, AND PLUMBING FIXTURE 
       			.MANUFACTURING
		327120	.MFG-CLAY BUILDING MATERIAL AND REFRACTORIES 
		3272	.MFG-GLASS AND GLASS PRODUCTS
		3279	.MFG-MISCELLANEOUS NONMETALLIC MINERAL PRODUCTS
		327M	.MFG-CEMENT, CONCRETE, LIME, AND GYPSUM PRODUCTS
		3313	.MFG-ALUMINUM PRODUCTION AND PROCESSING
		3314	.MFG-NONFERROUS METAL, EXCEPT ALUMINUM, PRODUCTION AND
			.PROCESSING
		3315	.MFG-FOUNDRIES
		331M	.MFG-IRON AND STEEL MILLS AND STEEL PRODUCTS
		3321	.MFG-METAL FORGINGS AND STAMPINGS
		3322	.MFG-CUTLERY AND HAND TOOLS
		3327	.MFG-MACHINE SHOPS; TURNED PRODUCTS; SCREWS, NUTS AND
			.BOLTS
		3328	.MFG-COATING, ENGRAVING, HEAT TREATING AND ALLIED
			.ACTIVITIES
		33299M	.MFG-ORDNANCE
		332M	.MFG-STRUCTURAL METALS, AND BOILER, TANK, AND SHIPPING
			.CONTAINERS
		332MZ	.MFG-MISCELLANEOUS FABRICATED METAL PRODUCTS
		33311	.MFG-AGRICULTURAL IMPLEMENTS
		3331M	.MFG-CONSTRUCTION, AND MINING AND OIL AND GAS FIELD
			.MACHINERY
		3333	.MFG-COMMERCIAL AND SERVICE INDUSTRY MACHINERY
		3335	.MFG-METALWORKING MACHINERY
		3336	.MFG-ENGINE, TURBINE, AND POWER TRANSMISSION EQUIPMENT
		333MS	.MFG-MACHINERY MANUFACTURING, N.E.C. OR NOT SPECIFIED
		3341	.MFG-COMPUTER AND PERIPHERAL EQUIPMENT
		3345	.MFG-NAVIGATIONAL, MEASURING, ELECTROMEDICAL, AND 
			.CONTROL INSTRUMENTS
		334M1	.MFG-COMMUNICATIONS, AND AUDIO AND VIDEO EQUIPMENT
		334M2	.MFG-ELECTRONIC COMPONENTS AND PRODUCTS, N.E.C.
		3352	.MFG-HOUSEHOLD APPLIANCES
		335M	.MFG-ELECTRIC LIGHTING, AND ELECTRICAL EQUIPMENT
			.MANUFACTURING, AND OTHER ELECTRICAL COMPONENT
			.MANUFACTURING, N.E.C.
		33641M1 .MFG-AIRCRAFT AND PARTS
		33641M2 .MFG-AEROSPACE PRODUCTS AND PARTS
		3365	.MFG-RAILROAD ROLLING STOCK
		3366	.MFG-SHIP AND BOAT BUILDING
		3369	.MFG-OTHER TRANSPORTATION EQUIPMENT
		336M	.MFG-MOTOR VEHICLES AND MOTOR VEHICLE EQUIPMENT
		337	.MFG-FURNITURE AND RELATED PRODUCTS
		3391	.MFG-MEDICAL EQUIPMENT AND SUPPLIES
		3399M	.MFG-SPORTING AND ATHLETIC GOODS, AND DOLL, TOY, AND 
			.GAME MANUFACTURING
		3399ZM  .MFG-MISCELLANEOUS MANUFACTURING, N.E.C.
		33MS	.MFG-NOT SPECIFIED METAL INDUSTRIES
		3MS	.MFG-NOT SPECIFIED INDUSTRIES
		4231	.WHL-MOTOR VEHICLES AND MOTOR VEHICLE PARTS AND SUPPLIES
			.MERCHANT WHOLESALERS
		4232	.WHL-FURNITURE AND HOME FURNISHING MERCHANT WHOLESALERS
		4233	.WHL-LUMBER AND OTHER CONSTRUCTION MATERIALS MERCHANT
			.WHOLESALERS
		4234	.WHL-PROFESSIONAL AND COMMERCIAL EQUIPMENT AND SUPPLIES
			.MERCHANT WHOLESALERS
		4235	.WHL-METALS AND MINERALS, EXCEPT PETROLEUM, MERCHANT
			.WHOLESALERS
		4236	.WHL-HOUSEHOLD APPLIANCES AND ELECTRICAL AND ELECTRONIC
			.GOODS MERCHANT WHOLESALERS
		4237	.WHL-HARDWARE, AND PLUMBING AND HEATING EQUIPMENT, AND
			.SUPPLIES MERCHANT WHOLESALERS
		4238	.WHL-MACHINERY, EQUIPMENT, AND SUPPLIES MERCHANT
			.WHOLESALERS
		42393	.WHL-RECYCLABLE MATERIAL MERCHANT WHOLESALERS
		4239Z	.WHL-MISCELLANEOUS DURABLE GOODS MERCHANT WHOLESALERS
		4241	.WHL-PAPER AND PAPER PRODUCTS MERCHANT WHOLESALERS
		4243	.WHL-APPAREL, PIECE GOODS, AND NOTIONS MERCHANT
			.WHOLESALERS
		4244	.WHL-GROCERY AND RELATED PRODUCT MERCHANT WHOLESALERS
		4245	.WHL-FARM PRODUCT RAW MATERIAL MERCHANT WHOLESALERS
		4247	.WHL-PETROLEUM AND PETROLEUM PRODUCTS MERCHANT 
         		.WHOLESALERS
		4248	.WHL-ALCOHOLIC BEVERAGES MERCHANT WHOLESALERS
		42491	.WHL-FARM SUPPLIES MERCHANT WHOLESALERS
		4249Z	.WHL-MISCELLANEOUS NONDURABLE GOODS MERCHANT WHOLESALERS
		424M	.WHL-DRUGS, SUNDRIES, AND CHEMICAL AND ALLIED PRODUCTS
			.MERCHANT WHOLESALERS
		4251	.WHL-WHOLESALE ELECTRONIC MARKETS AND AGENTS AND BROKERS
		42S	.WHL-NOT SPECIFIED WHOLESALE TRADE
		4411	.RET-AUTOMOBILE DEALERS
		4412	.RET-OTHER MOTOR VEHICLE DEALERS
		4413	.RET-AUTOMOTIVE PARTS, ACCESSORIES, AND TIRE STORES
		442	.RET-FURNITURE AND HOME FURNISHINGS STORES
		443141	.RET-HOUSEHOLD APPLIANCE STORES
		443142	.RET-ELECTRONICS STORES
		44413	.RET-HARDWARE STORES
		4441Z	.RET-BUILDING MATERIAL AND SUPPLIES DEALERS
		4442	.RET-LAWN AND GARDEN EQUIPMENT AND SUPPLIES STORES
		4451	.RET-GROCERY STORES
		4452	.RET-SPECIALTY FOOD STORES
		4453	.RET-BEER, WINE, AND LIQUOR STORES
		44611	.RET-PHARMACIES AND DRUG STORES
		446Z	.RET-HEALTH AND PERSONAL CARE, EXCEPT DRUG, STORES
		447	.RET-GASOLINE STATIONS
		44821	.RET-SHOE STORES
		4483	.RET-JEWELRY, LUGGAGE, AND LEATHER GOODS STORES
		4481	.RET-CLOTHING STORES
		45113	.RET-SEWING, NEEDLEWORK, AND PIECE GOODS STORES
		45114	.RET-MUSICAL INSTRUMENT AND SUPPLIES STORES
		4511M	.RET-SPORTING GOODS, AND HOBBY AND TOY STORES
		45121	.RET-BOOK STORES AND NEWS DEALERS
		45211	.RET-DEPARTMENT AND DISCOUNT STORES
		4529	.RET-MISCELLANEOUS GENERAL MERCHANDISE STORES
		4531	.RET-FLORISTS
		45321	.RET-OFFICE SUPPLIES AND STATIONERY STORES
		45322	.RET-GIFT, NOVELTY, AND SOUVENIR SHOPS
		4533	.RET-USED MERCHANDISE STORES
		4539	.RET-MISCELLANEOUS RETAIL STORES
		454111	.RET-ELECTRONIC SHOPPING
		454112	.RET-ELECTRONIC AUCTIONS
		454113	.RET-MAIL-ORDER HOUSES
		4542	.RET-VENDING MACHINE OPERATORS
		454310	.RET-FUEL DEALERS
		45439	.RET-OTHER DIRECT SELLING ESTABLISHMENTS
		4MS	.RET-NOT SPECIFIED RETAIL TRADE
		481	.TRN-AIR TRANSPORTATION
		482	.TRN-RAIL TRANSPORTATION
		483	.TRN-WATER TRANSPORTATION
		484	.TRN-TRUCK TRANSPORTATION
		4853	.TRN-TAXI AND LIMOUSINE SERVICE
		485M	.TRN-BUS SERVICE AND URBAN TRANSIT
		486	.TRN-PIPELINE TRANSPORTATION
		487	.TRN-SCENIC AND SIGHTSEEING TRANSPORTATION
		488	.TRN-SERVICES INCIDENTAL TO TRANSPORTATION
		491	.TRN-POSTAL SERVICE
		492	.TRN-COURIERS AND MESSENGERS
		493	.TRN-WAREHOUSING AND STORAGE
		51111	.INF-NEWSPAPER PUBLISHERS
		5111Z	.INF-PERIODICAL, BOOK, AND DIRECTORY PUBLISHERS
		5112	.INF-SOFTWARE PUBLISHERS
		5121	.INF-MOTION PICTURE AND VIDEO INDUSTRIES
		5122	.INF-SOUND RECORDING INDUSTRIES
		515	.INF-BROADCASTING, EXCEPT INTERNET
		5171	.INF-WIRED TELECOMMUNICATIONS CARRIERS
		517Z	.INF-TELECOMMUNICATIONS, EXCEPT WIRED TELECOMMUNICATIONS
			.CARRIERS
		5182	.INF-DATA PROCESSING, HOSTING, AND RELATED SERVICES
		51912	.INF-LIBRARIES AND ARCHIVES
		51913	.INF-INTERNET PUBLISHING AND BROADCASTING AND WEB SEARCH
			.PORTALS
		5191ZM  .INF-OTHER INFORMATION SERVICES, EXCEPT LIBRARIES AND
			.ARCHIVES, AND INTERNET PUBLISHING AND BROADCASTING AND
			.WEB SEARCH PORTALS 
		5221M	.FIN-SAVINGS INSTITUTIONS, INCLUDING CREDIT UNIONS
		522M	.FIN-NONDEPOSITORY CREDIT AND RELATED ACTIVITIES
		524	.FIN-INSURANCE CARRIERS AND RELATED ACTIVITIES
		52M1	.FIN-BANKING AND RELATED ACTIVITIES 
       		52M2	.FIN-SECURITIES, COMMODITIES, FUNDS, TRUSTS, AND OTHER
			.FINANCIAL INVESTMENTS
		531	.FIN-REAL ESTATE
		5321	.FIN-AUTOMOTIVE EQUIPMENT RENTAL AND LEASING
		53223	.FIN-VIDEO TAPE AND DISK RENTAL
		532M	.FIN-OTHER CONSUMER GOODS RENTAL
		53M	.FIN-COMMERCIAL, INDUSTRIAL, AND OTHER INTANGIBLE ASSETS
			.RENTAL AND LEASING
		5411	.PRF-LEGAL SERVICES
		5412	.PRF-ACCOUNTING, TAX PREPARATION, BOOKKEEPING, AND 
       			.PAYROLL SERVICES
		5413	.PRF-ARCHITECTURAL, ENGINEERING, AND RELATED SERVICES
		5414	.PRF-SPECIALIZED DESIGN SERVICES
		5415	.PRF-COMPUTER SYSTEMS DESIGN AND RELATED SERVICES
		5416	.PRF-MANAGEMENT, SCIENTIFIC, AND TECHNICAL CONSULTING
			.SERVICES
		5417	.PRF-SCIENTIFIC RESEARCH AND DEVELOPMENT SERVICES
		5418	.PRF-ADVERTISING, PUBLIC RELATIONS, AND RELATED SERVICES
		54194	.PRF-VETERINARY SERVICES
		5419Z	.PRF-OTHER PROFESSIONAL, SCIENTIFIC, AND TECHNICAL
			.SERVICES
		55	.PRF-MANAGEMENT OF COMPANIES AND ENTERPRISES
		5613	.PRF-EMPLOYMENT SERVICES
		5614	.PRF-BUSINESS SUPPORT SERVICES
		5615	.PRF-TRAVEL ARRANGEMENTS AND RESERVATION SERVICES
		5616	.PRF-INVESTIGATION AND SECURITY SERVICES
		56173	.PRF-LANDSCAPING SERVICES
		5617Z	.PRF-SERVICES TO BUILDINGS AND DWELLINGS, EX CONSTR CLN
		561M	.PRF-OTHER ADMINISTRATIVE AND OTHER SUPPORT SERVICES
		562	.PRF-WASTE MANAGEMENT AND REMEDIATION SERVICES
		6111	.EDU-ELEMENTARY AND SECONDARY SCHOOLS
		611M1	.EDU-COLLEGES, UNIVERSITIES, AND PROFESSIONAL 
			.SCHOOLS, INCLUDING JUNIOR COLLEGES
		611M2	.EDU-BUSINESS, TECHNICAL, AND TRADE SCHOOLS AND TRAINING
		611M3	.EDU-OTHER SCHOOLS AND INSTRUCTION, AND EDUCATIONAL
			.SUPPORT SERVICES
		6211	.MED-OFFICES OF PHYSICIANS
		6212	.MED-OFFICES OF DENTISTS
		62131	.MED-OFFICES OF CHIROPRACTORS
		62132	.MED-OFFICES OF OPTOMETRISTS
		6213ZM	.MED-OFFICES OF OTHER HEALTH PRACTITIONERS
		6214	.MED-OUTPATIENT CARE CENTERS
		6216	.MED-HOME HEALTH CARE SERVICES
		621M	.MED-OTHER HEALTH CARE SERVICES
		622	.MED-HOSPITALS
		6231	.MED-NURSING CARE FACILITIES (SKILLED NURSING 
			.FACILITIES)
		623M	.MED-RESIDENTIAL CARE FACILITIES, EXCEPT SKILLED
			.NURSING FACILITIES
		6241	.SCA-INDIVIDUAL AND FAMILY SERVICES
		6242	.SCA-COMMUNITY FOOD AND HOUSING, AND EMERGENCY SERVICES
		6243	.SCA-VOCATIONAL REHABILITATION SERVICES
		6244	.SCA-CHILD DAY CARE SERVICES
		711	.ENT-PERFORMING ARTS, SPECTATOR
			.SPORTS, AND RELATED INDUSTRIES
		712	.ENT-MUSEUMS, ART GALLERIES, HISTORICAL SITES, AND 
       			.SIMILAR INSTITUTIONS
		71395	.ENT-BOWLING CENTERS
		713Z	.ENT-OTHER AMUSEMENT, GAMBLING, AND RECREATION 
			.INDUSTRIES
		7211	.ENT-TRAVELER ACCOMMODATION
		721M	.ENT-RECREATIONAL VEHICLE PARKS AND CAMPS, AND ROOMING 
       			.AND BOARDING HOUSES
		7224	.ENT-DRINKING PLACES, ALCOHOLIC BEVERAGES
		722Z	.ENT-RESTAURANTS AND OTHER FOOD SERVICES
		811192	.SRV-CAR WASHES
		8111Z	.SRV-AUTOMOTIVE REPAIR AND MAINTENANCE
		8112	.SRV-ELECTRONIC AND PRECISION EQUIPMENT REPAIR AND
			.MAINTENANCE
		8113	.SRV-COMMERCIAL AND INDUSTRIAL MACHINERY AND EQUIPMENT
			.REPAIR AND MAINTENANCE
		8114	.SRV-PERSONAL AND HOUSEHOLD GOODS REPAIR AND MAINTENANCE
		812111	.SRV-BARBER SHOPS
		812112	.SRV-BEAUTY SALONS
		8121M	.SRV-NAIL SALONS AND OTHER PERSONAL CARE SERVICES
		8122	.SRV-FUNERAL HOMES, CEMETERIES AND CREMATORIES
		8123	.SRV-DRYCLEANING AND LAUNDRY SERVICES
		8129	.SRV-OTHER PERSONAL SERVICES
		8131	.SRV-RELIGIOUS ORGANIZATIONS
		81393	.SRV-LABOR UNIONS
		8139Z	.SRV-BUSINESS, PROFESSIONAL, POLITICAL, AND SIMILAR
			.ORGANIZATIONS
		813M	.SRV-CIVIC, SOCIAL, ADVOCACY ORGANIZATIONS, AND
			.GRANTMAKING AND GIVING SERVICES
		814	.SRV-PRIVATE HOUSEHOLDS
		92113	.ADM-PUBLIC FINANCE ACTIVITIES
		92119	.ADM-OTHER GENERAL GOVERNMENT AND SUPPORT
		9211MP	.ADM-EXECUTIVE OFFICES AND LEGISLATIVE BODIES
		923	.ADM-ADMINISTRATION OF HUMAN RESOURCE PROGRAMS
		928110P1.MIL-U.S. ARMY
		928110P2.MIL-U.S. AIR FORCE
		928110P3.MIL-U.S. NAVY
		928110P4.MIL-U.S. MARINES
		928110P5.MIL-U.S. COAST GUARD
		928110P6.MIL-U.S. ARMED FORCES, BRANCH NOT SPECIFIED
		928110P7.MIL-MILITARY RESERVES OR NATIONAL GUARD
		928P	.ADM-NATIONAL SECURITY AND INTERNATIONAL AFFAIRS
		92M1	.ADM-ADMINISTRATION OF ENVIRONMENTAL QUALITY AND HOUSING
			.PROGRAMS
		92M2	.ADM-ADMINISTRATION OF ECONOMIC PROGRAMS AND SPACE
			.RESEARCH
		92MP	.ADM-JUSTICE, PUBLIC ORDER, AND SAFETY ACTIVITIES
		9920	.UNEMPLOYED AND LAST WORKED 5 YEARS AGO OR EARLIER OR
                       	.NEVER WORKED**

Note: For additional information on NAICS and SOC groupings within major categories visit our website at: https://www.census.gov/people/io/methodology/indexes.html.	

NATIVITY	1	
	Nativity
		1 .Native
		2 .Foreign born

NOP		1
	Nativity of parent
       		b .N/A (greater than 17 years old/not an own child of   
 		  .householder,and not child in subfamily)
		1 .Living with two parents: Both parents NATIVE
		2 .Living with two parents: Father only FOREIGN BORN
		3 .Living with two parents: Mother only FOREIGN BORN
		4 .Living with two parents: BOTH parents FOREIGN BORN
		5 .Living with father only: Father NATIVE
		6 .Living with father only: Father FOREIGN BORN
		7 .Living with mother only: Mother NATIVE
		8 .Living with mother only: Mother FOREIGN BORN

OC		1	
	Own child
		0 .No (includes GQ)	
		1 .Yes	

OCCP		4	
	Occupation recode for 2012 and later based on 2010 OCC codes
		bbbb    .N/A (less than 16 years old/NILF who last worked
			.more than 5 years ago or never worked)
		0010	.MGR-CHIEF EXECUTIVES AND LEGISLATORS
		0020	.MGR-GENERAL AND OPERATIONS MANAGERS
		0040	.MGR-ADVERTISING AND PROMOTIONS MANAGERS
		0050	.MGR-MARKETING AND SALES MANAGERS
		0060	.MGR-PUBLIC RELATIONS AND FUNDRAISING MANAGERS
		0100	.MGR-ADMINISTRATIVE SERVICES MANAGERS
		0110	.MGR-COMPUTER AND INFORMATION SYSTEMS MANAGERS
		0120	.MGR-FINANCIAL MANAGERS
		0135	.MGR-COMPENSATION AND BENEFITS MANAGERS
		0136	.MGR-HUMAN RESOURCES MANAGERS
		0137	.MGR-TRAINING AND DEVELOPMENT MANAGERS
		0140	.MGR-INDUSTRIAL PRODUCTION MANAGERS
		0150	.MGR-PURCHASING MANAGERS
		0160	.MGR-TRANSPORTATION, STORAGE, AND DISTRIBUTION MANAGERS
		0205	.MGR-FARMERS, RANCHERS, AND OTHER AGRICULTURAL MANAGERS
		0220	.MGR-CONSTRUCTION MANAGERS
		0230	.MGR-EDUCATION ADMINISTRATORS
		0300	.MGR-ARCHITECTURAL AND ENGINEERING MANAGERS
		0310	.MGR-FOOD SERVICE MANAGERS
		0330	.MGR-GAMING MANAGERS
		0340	.MGR-LODGING MANAGERS
		0350	.MGR-MEDICAL AND HEALTH SERVICES MANAGERS
		0360	.MGR-NATURAL SCIENCES MANAGERS
		0410	.MGR-PROPERTY, REAL ESTATE, AND COMMUNITY ASSOCIATION
		    	.MANAGERS
		0420	.MGR-SOCIAL AND COMMUNITY SERVICE MANAGERS
		0425	.MGR-EMERGENCY MANAGEMENT DIRECTORS
		0430	.MGR-MISCELLANEOUS MANAGERS, INCLUDING FUNERAL SERVICE 
			.MANAGERS AND POSTMASTERS AND MAIL SUPERINTENDENTS
		0500	.BUS-AGENTS AND BUSINESS MANAGERS OF ARTISTS, PERFORMERS,
			.AND ATHLETES
		0510	.BUS-BUYERS AND PURCHASING AGENTS, FARM PRODUCTS
		0520	.BUS-WHOLESALE AND RETAIL BUYERS, EXCEPT FARM PRODUCTS
		0530	.BUS-PURCHASING AGENTS, EXCEPT WHOLESALE, RETAIL, AND FARM
			.PRODUCTS
		0540	.BUS-CLAIMS ADJUSTERS, APPRAISERS, EXAMINERS, AND
			.INVESTIGATORS
		0565	.BUS-COMPLIANCE OFFICERS
		0600	.BUS-COST ESTIMATORS
		0630	.BUS-HUMAN RESOURCES WORKERS
		0640	.BUS-COMPENSATION, BENEFITS, AND JOB ANALYSIS SPECIALISTS
		0650	.BUS-TRAINING AND DEVELOPMENT SPECIALISTS
		0700	.BUS-LOGISTICIANS
		0710	.BUS-MANAGEMENT ANALYSTS
		0725	.BUS-MEETING, CONVENTION, AND EVENT PLANNERS
		0726	.BUS-FUNDRAISERS
		0735	.BUS-MARKET RESEARCH ANALYSTS AND MARKETING SPECIALISTS
		0740	.BUS-BUSINESS OPERATIONS SPECIALISTS, ALL OTHER
		0800	.FIN-ACCOUNTANTS AND AUDITORS
		0810	.FIN-APPRAISERS AND ASSESSORS OF REAL ESTATE
		0820	.FIN-BUDGET ANALYSTS
		0830	.FIN-CREDIT ANALYSTS
		0840	.FIN-FINANCIAL ANALYSTS
		0850	.FIN-PERSONAL FINANCIAL ADVISORS
		0860	.FIN-INSURANCE UNDERWRITERS
		0900	.FIN-FINANCIAL EXAMINERS
		0910	.FIN-CREDIT COUNSELORS AND LOAN OFFICERS
		0930	.FIN-TAX EXAMINERS AND COLLECTORS, AND REVENUE AGENTS
		0940	.FIN-TAX PREPARERS
		0950	.FIN-FINANCIAL SPECIALISTS, ALL OTHER
		1005	.CMM-COMPUTER AND INFORMATION RESEARCH SCIENTISTS
		1006	.CMM-COMPUTER SYSTEMS ANALYSTS
		1007	.CMM-INFORMATION SECURITY ANALYSTS
		1010	.CMM-COMPUTER PROGRAMMERS
		1020	.CMM-SOFTWARE DEVELOPERS, APPLICATIONS AND SYSTEMS SOFTWARE
		1030	.CMM-WEB DEVELOPERS
		1050	.CMM-COMPUTER SUPPORT SPECIALISTS
		1060	.CMM-DATABASE ADMINISTRATORS
		1105	.CMM-NETWORK AND COMPUTER SYSTEMS ADMINISTRATORS
		1106	.CMM-COMPUTER NETWORK ARCHITECTS
		1107	.CMM-COMPUTER OCCUPATIONS, ALL OTHER
		1200	.CMM-ACTUARIES
		1220	.CMM-OPERATIONS RESEARCH ANALYSTS
		1240	.CMM-MISCELLANEOUS MATHEMATICAL SCIENCE OCCUPATIONS,
			.INCLUDING MATHEMATICIANS AND STATISTICIANS
		1300	.ENG-ARCHITECTS, EXCEPT NAVAL
		1310	.ENG-SURVEYORS, CARTOGRAPHERS, AND PHOTOGRAMMETRISTS
		1320	.ENG-AEROSPACE ENGINEERS
		1340	.ENG-BIOMEDICAL AND AGRICULTURAL ENGINEERS
		1350	.ENG-CHEMICAL ENGINEERS
		1360	.ENG-CIVIL ENGINEERS
		1400	.ENG-COMPUTER HARDWARE ENGINEERS
		1410	.ENG-ELECTRICAL AND ELECTRONICS ENGINEERS
		1420	.ENG-ENVIRONMENTAL ENGINEERS
		1430	.ENG-INDUSTRIAL ENGINEERS, INCLUDING HEALTH AND SAFETY
		1440	.ENG-MARINE ENGINEERS AND NAVAL ARCHITECTS
		1450	.ENG-MATERIALS ENGINEERS
		1460	.ENG-MECHANICAL ENGINEERS
		1520	.ENG-PETROLEUM, MINING AND GEOLOGICAL ENGINEERS, INCLUDING
			.MINING SAFETY ENGINEERS
		1530	.ENG-MISCELLANEOUS ENGINEERS, INCLUDING NUCLEAR ENGINEERS
		1540	.ENG-DRAFTERS
		1550	.ENG-ENGINEERING TECHNICIANS, EXCEPT DRAFTERS
		1560	.ENG-SURVEYING AND MAPPING TECHNICIANS
		1600	.SCI-AGRICULTURAL AND FOOD SCIENTISTS
		1610	.SCI-BIOLOGICAL SCIENTISTS
		1640	.SCI-CONSERVATION SCIENTISTS AND FORESTERS
		1650	.SCI-MEDICAL SCIENTISTS, AND LIFE SCIENTISTS, ALL OTHER
		1700	.SCI-ASTRONOMERS AND PHYSICISTS
		1710	.SCI-ATMOSPHERIC AND SPACE SCIENTISTS
		1720	.SCI-CHEMISTS AND MATERIALS SCIENTISTS
		1740	.SCI-ENVIRONMENTAL SCIENTISTS AND GEOSCIENTISTS
		1760	.SCI-PHYSICAL SCIENTISTS, ALL OTHER
		1800	.SCI-ECONOMISTS
		1820	.SCI-PSYCHOLOGISTS
		1840	.SCI-URBAN AND REGIONAL PLANNERS
		1860	.SCI-MISCELLANEOUS SOCIAL SCIENTISTS, INCLUDING SURVEY
			.RESEARCHERS AND SOCIOLOGISTS
		1900	.SCI-AGRICULTURAL AND FOOD SCIENCE TECHNICIANS
		1910	.SCI-BIOLOGICAL TECHNICIANS
		1920	.SCI-CHEMICAL TECHNICIANS
		1930	.SCI-GEOLOGICAL AND PETROLEUM TECHNICIANS, AND NUCLEAR
			.TECHNICIANS
		1965	.SCI-MISCELLANEOUS LIFE, PHYSICAL, AND SOCIAL SCIENCE
			.TECHNICIANS, INCLUDING SOCIAL SCIENCE RESEARCH ASSISTANTS
		2000	.CMS-COUNSELORS
		2010	.CMS-SOCIAL WORKERS
		2015	.CMS-PROBATION OFFICERS AND CORRECTIONAL TREATMENT 
			.SPECIALISTS
		2016	.CMS-SOCIAL AND HUMAN SERVICE ASSISTANTS
		2025	.CMS-MISCELLANEOUS COMMUNITY AND SOCIAL SERVICE 
			.SPECIALISTS,INCLUDING HEALTH EDUCATORS AND COMMUNITY 
			.HEALTH WORKERS
		2040	.CMS-CLERGY
		2050	.CMS-DIRECTORS, RELIGIOUS ACTIVITIES AND EDUCATION
		2060	.CMS-RELIGIOUS WORKERS, ALL OTHER
		2100	.LGL-LAWYERS, AND JUDGES, MAGISTRATES, AND OTHER JUDICIAL
			.WORKERS
		2105	.LGL-JUDICIAL LAW CLERKS
		2145	.LGL-PARALEGALS AND LEGAL ASSISTANTS
		2160	.LGL-MISCELLANEOUS LEGAL SUPPORT WORKERS
		2200	.EDU-POSTSECONDARY TEACHERS
		2300	.EDU-PRESCHOOL AND KINDERGARTEN TEACHERS
		2310	.EDU-ELEMENTARY AND MIDDLE SCHOOL TEACHERS
		2320	.EDU-SECONDARY SCHOOL TEACHERS
		2330	.EDU-SPECIAL EDUCATION TEACHERS
		2340	.EDU-OTHER TEACHERS AND INSTRUCTORS
		2400	.EDU-ARCHIVISTS, CURATORS, AND MUSEUM TECHNICIANS
		2430	.EDU-LIBRARIANS
		2440	.EDU-LIBRARY TECHNICIANS
		2540	.EDU-TEACHER ASSISTANTS
		2550	.EDU-OTHER EDUCATION, TRAINING, AND LIBRARY WORKERS
		2600	.ENT-ARTISTS AND RELATED WORKERS
		2630	.ENT-DESIGNERS
		2700	.ENT-ACTORS
		2710	.ENT-PRODUCERS AND DIRECTORS
		2720	.ENT-ATHLETES, COACHES, UMPIRES, AND RELATED WORKERS
		2740	.ENT-DANCERS AND CHOREOGRAPHERS
		2750	.ENT-MUSICIANS, SINGERS, AND RELATED WORKERS
		2760	.ENT-ENTERTAINERS AND PERFORMERS, SPORTS AND RELATED
			.WORKERS, ALL OTHER
		2800	.ENT-ANNOUNCERS
		2810	.ENT-NEWS ANALYSTS, REPORTERS AND CORRESPONDENTS
		2825	.ENT-PUBLIC RELATIONS SPECIALISTS
		2830	.ENT-EDITORS
		2840	.ENT-TECHNICAL WRITERS
		2850	.ENT-WRITERS AND AUTHORS
		2860	.ENT-MISCELLANEOUS MEDIA AND COMMUNICATION WORKERS
		2900	.ENT-BROADCAST AND SOUND ENGINEERING TECHNICIANS AND RADIO
			.OPERATORS, AND MEDIA AND COMMUNICATION EQUIPMENT WORKERS,
			.ALL OTHER
		2910	.ENT-PHOTOGRAPHERS
		2920	.ENT-TELEVISION, VIDEO, AND MOTION PICTURE CAMERA OPERATORS
			.AND EDITORS
		3000	.MED-CHIROPRACTORS
		3010	.MED-DENTISTS
		3030	.MED-DIETITIANS AND NUTRITIONISTS
		3040	.MED-OPTOMETRISTS
		3050	.MED-PHARMACISTS
		3060	.MED-PHYSICIANS AND SURGEONS
		3110	.MED-PHYSICIAN ASSISTANTS
		3120	.MED-PODIATRISTS
		3140	.MED-AUDIOLOGISTS
		3150	.MED-OCCUPATIONAL THERAPISTS
		3160	.MED-PHYSICAL THERAPISTS
		3200	.MED-RADIATION THERAPISTS
		3210	.MED-RECREATIONAL THERAPISTS
		3220	.MED-RESPIRATORY THERAPISTS
		3230	.MED-SPEECH-LANGUAGE PATHOLOGISTS
		3245	.MED-OTHER THERAPISTS, INCLUDING EXERCISE PHYSIOLOGISTS
		3250	.MED-VETERINARIANS
		3255	.MED-REGISTERED NURSES
		3256	.MED-NURSE ANESTHETISTS
		3258	.MED-NURSE PRACTITIONERS, AND NURSE MIDWIVES
		3260	.MED-HEALTH DIAGNOSING AND TREATING PRACTITIONERS, ALL 
			.OTHER
		3300	.MED-CLINICAL LABORATORY TECHNOLOGISTS AND TECHNICIANS
		3310	.MED-DENTAL HYGIENISTS
		3320	.MED-DIAGNOSTIC RELATED TECHNOLOGISTS AND TECHNICIANS
		3400	.MED-EMERGENCY MEDICAL TECHNICIANS AND PARAMEDICS
		3420	.MED-HEALTH PRACTITIONER SUPPORT TECHNOLOGISTS AND 
			.TECHNICIANS
		3500	.MED-LICENSED PRACTICAL AND LICENSED VOCATIONAL NURSES
		3510	.MED-MEDICAL RECORDS AND HEALTH INFORMATION TECHNICIANS
		3520	.MED-OPTICIANS, DISPENSING
		3535	.MED-MISCELLANEOUS HEALTH TECHNOLOGISTS AND TECHNICIANS
		3540	.MED-OTHER HEALTHCARE PRACTITIONERS AND TECHNICAL
			.OCCUPATIONS
		3600	.HLS-NURSING, PSYCHIATRIC, AND HOME HEALTH AIDES
		3610	.HLS-OCCUPATIONAL THERAPY ASSISTANTS AND AIDES
		3620	.HLS-PHYSICAL THERAPIST ASSISTANTS AND AIDES
		3630	.HLS-MASSAGE THERAPISTS
		3640	.HLS-DENTAL ASSISTANTS
		3645	.HLS-MEDICAL ASSISTANTS
		3646	.HLS-MEDICAL TRANSCRIPTIONISTS
		3647	.HLS-PHARMACY AIDES
		3648	.HLS-VETERINARY ASSISTANTS AND LABORATORY ANIMAL CARETAKERS
		3649	.HLS-PHLEBOTOMISTS
		3655	.HLS-HEALTHCARE SUPPORT WORKERS, ALL OTHER, INCLUDING
			.MEDICAL EQUIPMENT PREPARERS
		3700	.PRT-FIRST-LINE SUPERVISORS OF CORRECTIONAL OFFICERS
		3710	.PRT-FIRST-LINE SUPERVISORS OF POLICE AND DETECTIVES
		3720	.PRT-FIRST-LINE SUPERVISORS OF FIRE FIGHTING AND PREVENTION
			.WORKERS
		3730	.PRT-FIRST-LINE SUPERVISORS OF PROTECTIVE SERVICE WORKERS, 
       			.ALL OTHER
		3740	.PRT-FIREFIGHTERS
		3750	.PRT-FIRE INSPECTORS
		3800	.PRT-BAILIFFS, CORRECTIONAL OFFICERS, AND JAILERS
		3820	.PRT-DETECTIVES AND CRIMINAL INVESTIGATORS
		3840	.PRT-MISCELLANEOUS LAW ENFORCEMENT WORKERS
		3850	.PRT-POLICE OFFICERS
		3900	.PRT-ANIMAL CONTROL WORKERS
		3910	.PRT-PRIVATE DETECTIVES AND INVESTIGATORS
		3930	.PRT-SECURITY GUARDS AND GAMING SURVEILLANCE OFFICERS
		3940	.PRT-CROSSING GUARDS
		3945	.PRT-TRANSPORTATION SECURITY SCREENERS
		3955	.PRT-LIFEGUARDS AND OTHER RECREATIONAL, AND ALL OTHER
			.PROTECTIVE SERVICE WORKERS
		4000	.EAT-CHEFS AND HEAD COOKS
		4010	.EAT-FIRST-LINE SUPERVISORS OF FOOD PREPARATION AND SERVING
			.WORKERS
		4020	.EAT-COOKS
		4030	.EAT-FOOD PREPARATION WORKERS
		4040	.EAT-BARTENDERS
		4050	.EAT-COMBINED FOOD PREPARATION AND SERVING WORKERS,
			.INCLUDING FAST FOOD
		4060	.EAT-COUNTER ATTENDANTS, CAFETERIA, FOOD CONCESSION, AND
			.COFFEE SHOP
		4110	.EAT-WAITERS AND WAITRESSES
		4120	.EAT-FOOD SERVERS, NONRESTAURANT
		4130	.EAT-MISCELLANEOUS FOOD PREPARATION AND SERVING RELATED
       			.WORKERS, INCLUDING DINING ROOM AND CAFETERIA ATTENDANTS 
       			.AND BARTENDER HELPERS
		4140	.EAT-DISHWASHERS
		4150	.EAT-HOSTS AND HOSTESSES, RESTAURANT, LOUNGE, AND COFFEE 
       			.SHOP
		4200	.CLN-FIRST-LINE SUPERVISORS OF HOUSEKEEPING AND JANITORIAL
			.WORKERS
		4210	.CLN-FIRST-LINE SUPERVISORS OF LANDSCAPING, LAWN SERVICE, 
       			.AND GROUNDSKEEPING WORKERS
		4220	.CLN-JANITORS AND BUILDING CLEANERS
		4230	.CLN-MAIDS AND HOUSEKEEPING CLEANERS
		4240	.CLN-PEST CONTROL WORKERS
		4250	.CLN-GROUNDS MAINTENANCE WORKERS
		4300	.PRS-FIRST-LINE SUPERVISORS OF GAMING WORKERS
		4320	.PRS-FIRST-LINE SUPERVISORS OF PERSONAL SERVICE WORKERS
		4340	.PRS-ANIMAL TRAINERS
		4350	.PRS-NONFARM ANIMAL CARETAKERS
		4400	.PRS-GAMING SERVICES WORKERS
		4410	.PRS-MOTION PICTURE PROJECTIONISTS
		4420	.PRS-USHERS, LOBBY ATTENDANTS, AND TICKET TAKERS
		4430	.PRS-MISCELLANEOUS ENTERTAINMENT ATTENDANTS AND RELATED
			.WORKERS
		4460	.PRS-EMBALMERS AND FUNERAL ATTENDANTS
		4465	.PRS-MORTICIANS, UNDERTAKERS, AND FUNERAL DIRECTORS
		4500	.PRS-BARBERS					
		4510	.PRS-HAIRDRESSERS, HAIRSTYLISTS, AND COSMETOLOGISTS
		4520	.PRS-MISCELLANEOUS PERSONAL APPEARANCE WORKERS
		4530	.PRS-BAGGAGE PORTERS, BELLHOPS, AND CONCIERGES
		4540	.PRS-TOUR AND TRAVEL GUIDES
		4600	.PRS-CHILDCARE WORKERS
		4610	.PRS-PERSONAL CARE AIDES
		4620	.PRS-RECREATION AND FITNESS WORKERS
		4640	.PRS-RESIDENTIAL ADVISORS
		4650	.PRS-PERSONAL CARE AND SERVICE WORKERS, ALL OTHER
		4700	.SAL-FIRST-LINE SUPERVISORS OF RETAIL SALES WORKERS
		4710	.SAL-FIRST-LINE SUPERVISORS OF NON-RETAIL SALES WORKERS
		4720	.SAL-CASHIERS
		4740	.SAL-COUNTER AND RENTAL CLERKS
		4750	.SAL-PARTS SALESPERSONS
		4760	.SAL-RETAIL SALESPERSONS
		4800	.SAL-ADVERTISING SALES AGENTS
		4810	.SAL-INSURANCE SALES AGENTS
		4820	.SAL-SECURITIES, COMMODITIES, AND FINANCIAL SERVICES SALES
			.AGENTS
		4830	.SAL-TRAVEL AGENTS
		4840	.SAL-SALES REPRESENTATIVES, SERVICES, ALL OTHER
		4850	.SAL-SALES REPRESENTATIVES, WHOLESALE AND MANUFACTURING
		4900	.SAL-MODELS, DEMONSTRATORS, AND PRODUCT PROMOTERS
		4920	.SAL-REAL ESTATE BROKERS AND SALES AGENTS
		4930	.SAL-SALES ENGINEERS
		4940	.SAL-TELEMARKETERS
		4950	.SAL-DOOR-TO-DOOR SALES WORKERS, NEWS AND STREET VENDORS,
			.AND RELATED WORKERS
		4965	.SAL-SALES AND RELATED WORKERS, ALL OTHER
		5000	.OFF-FIRST-LINE SUPERVISORS OF OFFICE AND ADMINISTRATIVE
			.SUPPORT WORKERS
		5010	.OFF-SWITCHBOARD OPERATORS, INCLUDING ANSWERING SERVICE
		5020	.OFF-TELEPHONE OPERATORS
		5030	.OFF-COMMUNICATIONS EQUIPMENT OPERATORS, ALL OTHER
		5100	.OFF-BILL AND ACCOUNT COLLECTORS
		5110	.OFF-BILLING AND POSTING CLERKS
		5120	.OFF-BOOKKEEPING, ACCOUNTING, AND AUDITING CLERKS
		5130	.OFF-GAMING CAGE WORKERS
		5140	.OFF-PAYROLL AND TIMEKEEPING CLERKS
		5150	.OFF-PROCUREMENT CLERKS
		5160	.OFF-TELLERS
		5165	.OFF-FINANCIAL CLERKS, ALL OTHER
		5200	.OFF-BROKERAGE CLERKS
		5220	.OFF-COURT, MUNICIPAL, AND LICENSE CLERKS
		5230	.OFF-CREDIT AUTHORIZERS, CHECKERS, AND CLERKS
		5240	.OFF-CUSTOMER SERVICE REPRESENTATIVES
		5250	.OFF-ELIGIBILITY INTERVIEWERS, GOVERNMENT PROGRAMS
		5260	.OFF-FILE CLERKS
		5300	.OFF-HOTEL, MOTEL, AND RESORT DESK CLERKS
		5310	.OFF-INTERVIEWERS, EXCEPT ELIGIBILITY AND LOAN
		5320	.OFF-LIBRARY ASSISTANTS, CLERICAL
		5330	.OFF-LOAN INTERVIEWERS AND CLERKS
		5340	.OFF-NEW ACCOUNTS CLERKS
		5350	.OFF-CORRESPONDENCE CLERKS AND ORDER CLERKS
		5360	.OFF-HUMAN RESOURCES ASSISTANTS, EXCEPT PAYROLL AND
			.TIMEKEEPING
		5400	.OFF-RECEPTIONISTS AND INFORMATION CLERKS
		5410	.OFF-RESERVATION AND TRANSPORTATION TICKET AGENTS AND 
			.TRAVEL CLERKS
		5420	.OFF-INFORMATION AND RECORD CLERKS, ALL OTHER
		5500	.OFF-CARGO AND FREIGHT AGENTS
		5510	.OFF-COURIERS AND MESSENGERS
		5520	.OFF-DISPATCHERS
		5530	.OFF-METER READERS, UTILITIES
		5540	.OFF-POSTAL SERVICE CLERKS
		5550	.OFF-POSTAL SERVICE MAIL CARRIERS
		5560	.OFF-POSTAL SERVICE MAIL SORTERS, PROCESSORS, AND 
			.PROCESSING MACHINE OPERATORS
		5600	.OFF-PRODUCTION, PLANNING, AND EXPEDITING CLERKS
		5610	.OFF-SHIPPING, RECEIVING, AND TRAFFIC CLERKS
		5620	.OFF-STOCK CLERKS AND ORDER FILLERS
		5630	.OFF-WEIGHERS, MEASURERS, CHECKERS, AND SAMPLERS,
			.RECORDKEEPING
		5700	.OFF-SECRETARIES AND ADMINISTRATIVE ASSISTANTS
		5800	.OFF-COMPUTER OPERATORS
		5810	.OFF-DATA ENTRY KEYERS
		5820	.OFF-WORD PROCESSORS AND TYPISTS
		5840	.OFF-INSURANCE CLAIMS AND POLICY PROCESSING CLERKS
		5850	.OFF-MAIL CLERKS AND MAIL MACHINE OPERATORS, EXCEPT POSTAL
			.SERVICE
		5860	.OFF-OFFICE CLERKS, GENERAL
		5900	.OFF-OFFICE MACHINE OPERATORS, EXCEPT COMPUTER
		5910	.OFF-PROOFREADERS AND COPY MARKERS
		5920	.OFF-STATISTICAL ASSISTANTS
		5940	.OFF-MISCELLANEOUS OFFICE AND ADMINISTRATIVE SUPPORT
			.WORKERS, INCLUDING DESKTOP PUBLISHERS
       		6005	.FFF-FIRST-LINE SUPERVISORS OF FARMING, FISHING, AND 
       			.FORESTRY WORKERS
		6010	.FFF-AGRICULTURAL INSPECTORS
		6040	.FFF-GRADERS AND SORTERS, AGRICULTURAL PRODUCTS
		6050	.FFF-MISCELLANEOUS AGRICULTURAL WORKERS, INCLUDING ANIMAL
			.BREEDERS
		6100	.FFF-FISHING AND HUNTING WORKERS
		6120	.FFF-FOREST AND CONSERVATION WORKERS
		6130	.FFF-LOGGING WORKERS
		6200	.CON-FIRST-LINE SUPERVISORS OF CONSTRUCTION TRADES AND
			.EXTRACTION WORKERS
		6210	.CON-BOILERMAKERS
		6220	.CON-BRICKMASONS, BLOCKMASONS, STONEMASONS, AND 
			.REINFORCING IRON AND REBAR WORKERS
		6230	.CON-CARPENTERS
		6240	.CON-CARPET, FLOOR, AND TILE INSTALLERS AND FINISHERS
		6250	.CON-CEMENT MASONS, CONCRETE FINISHERS, AND TERRAZZO 
			.WORKERS
		6260	.CON-CONSTRUCTION LABORERS
		6300	.CON-PAVING, SURFACING, AND TAMPING EQUIPMENT OPERATORS
		6320	.CON-CONSTRUCTION EQUIPMENT OPERATORS, EXCEPT PAVING,
			.SURFACING, AND TAMPING EQUIPMENT OPERATORS
		6330	.CON-DRYWALL INSTALLERS, CEILING TILE INSTALLERS, AND 
			.TAPERS
		6355	.CON-ELECTRICIANS
		6360	.CON-GLAZIERS
		6400	.CON-INSULATION WORKERS
		6420	.CON-PAINTERS AND PAPERHANGERS
		6440	.CON-PIPELAYERS, PLUMBERS, PIPEFITTERS, AND STEAMFITTERS
		6460	.CON-PLASTERERS AND STUCCO MASONS
		6515	.CON-ROOFERS
		6520	.CON-SHEET METAL WORKERS
		6530	.CON-STRUCTURAL IRON AND STEEL WORKERS
		6600	.CON-HELPERS, CONSTRUCTION TRADES
		6660	.CON-CONSTRUCTION AND BUILDING INSPECTORS
		6700	.CON-ELEVATOR INSTALLERS AND REPAIRERS
		6710	.CON-FENCE ERECTORS
		6720	.CON-HAZARDOUS MATERIALS REMOVAL WORKERS
		6730	.CON-HIGHWAY MAINTENANCE WORKERS
		6740	.CON-RAIL-TRACK LAYING AND MAINTENANCE EQUIPMENT OPERATORS
		6765	.CON-MISCELLANEOUS CONSTRUCTION WORKERS, INCLUDING SOLAR
			.PHOTOVOLTAIC INSTALLERS, SEPTIC TANK SERVICERS AND SEWER
			.PIPE CLEANERS
		6800	.EXT-DERRICK, ROTARY DRILL, AND SERVICE UNIT OPERATORS, AND
			.ROUSTABOUTS, OIL, GAS, AND MINING
		6820	.EXT-EARTH DRILLERS, EXCEPT OIL AND GAS
		6830	.EXT-EXPLOSIVES WORKERS, ORDNANCE HANDLING EXPERTS, AND
			.BLASTERS
		6840	.EXT-MINING MACHINE OPERATORS
		6940	.EXT-MISCELLANEOUS EXTRACTION WORKERS, INCLUDING ROOF 
       			.BOLTERS AND HELPERS
		7000	.RPR-FIRST-LINE SUPERVISORS OF MECHANICS, INSTALLERS, AND
			.REPAIRERS
       		7010	.RPR-COMPUTER, AUTOMATED TELLER, AND OFFICE MACHINE 
       			.REPAIRERS
		7020	.RPR-RADIO AND TELECOMMUNICATIONS EQUIPMENT INSTALLERS AND
			.REPAIRERS
		7030	.RPR-AVIONICS TECHNICIANS
		7040	.RPR-ELECTRIC MOTOR, POWER TOOL, AND RELATED REPAIRERS
		7100	.RPR-ELECTRICAL AND ELECTRONICS REPAIRERS, TRANSPORTATION
			.EQUIPMENT, AND INDUSTRIAL AND UTILITY
		7110	.RPR-ELECTRONIC EQUIPMENT INSTALLERS AND REPAIRERS, MOTOR
			.VEHICLES
		7120	.RPR-ELECTRONIC HOME ENTERTAINMENT EQUIPMENT INSTALLERS AND
			.REPAIRERS
		7130	.RPR-SECURITY AND FIRE ALARM SYSTEMS INSTALLERS
		7140	.RPR-AIRCRAFT MECHANICS AND SERVICE TECHNICIANS
		7150	.RPR-AUTOMOTIVE BODY AND RELATED REPAIRERS
		7160	.RPR-AUTOMOTIVE GLASS INSTALLERS AND REPAIRERS
		7200	.RPR-AUTOMOTIVE SERVICE TECHNICIANS AND MECHANICS
		7210	.RPR-BUS AND TRUCK MECHANICS AND DIESEL ENGINE SPECIALISTS
		7220	.RPR-HEAVY VEHICLE AND MOBILE EQUIPMENT SERVICE TECHNICIANS
			.AND MECHANICS
		7240	.RPR-SMALL ENGINE MECHANICS
		7260	.RPR-MISCELLANEOUS VEHICLE AND MOBILE EQUIPMENT MECHANICS,
			.INSTALLERS, AND REPAIRERS
		7300	.RPR-CONTROL AND VALVE INSTALLERS AND REPAIRERS
		7315	.RPR-HEATING, AIR CONDITIONING, AND REFRIGERATION MECHANICS
			.AND INSTALLERS
		7320	.RPR-HOME APPLIANCE REPAIRERS
		7330	.RPR-INDUSTRIAL AND REFRACTORY MACHINERY MECHANICS
		7340	.RPR-MAINTENANCE AND REPAIR WORKERS, GENERAL
		7350	.RPR-MAINTENANCE WORKERS, MACHINERY
		7360	.RPR-MILLWRIGHTS
		7410	.RPR-ELECTRICAL POWER-LINE INSTALLERS AND REPAIRERS
		7420	.RPR-TELECOMMUNICATIONS LINE INSTALLERS AND REPAIRERS
		7430	.RPR-PRECISION INSTRUMENT AND EQUIPMENT REPAIRERS
		7510	.RPR-COIN, VENDING, AND AMUSEMENT MACHINE SERVICERS AND
			.REPAIRERS
		7540	.RPR-LOCKSMITHS AND SAFE REPAIRERS
		7560	.RPR-RIGGERS
		7610	.RPR-HELPERS--INSTALLATION, MAINTENANCE, AND REPAIR WORKERS
		7630	.RPR-MISCELLANEOUS INSTALLATION, MAINTENANCE, AND REPAIR
			.WORKERS, INCLUDING WIND TURBINE SERVICE TECHNICIANS
		7700	.PRD-FIRST-LINE SUPERVISORS OF PRODUCTION AND OPERATING
			.WORKERS
		7710	.PRD-AIRCRAFT STRUCTURE, SURFACES, RIGGING, AND SYSTEMS
			.ASSEMBLERS
		7720	.PRD-ELECTRICAL, ELECTRONICS, AND ELECTROMECHANICAL
			.ASSEMBLERS
		7730	.PRD-ENGINE AND OTHER MACHINE ASSEMBLERS
		7740	.PRD-STRUCTURAL METAL FABRICATORS AND FITTERS
		7750	.PRD-MISCELLANEOUS ASSEMBLERS AND FABRICATORS
		7800	.PRD-BAKERS
		7810	.PRD-BUTCHERS AND OTHER MEAT, POULTRY, AND FISH PROCESSING
			.WORKERS
		7830	.PRD-FOOD AND TOBACCO ROASTING, BAKING, AND DRYING MACHINE
			.OPERATORS AND TENDERS
		7840	.PRD-FOOD BATCHMAKERS
		7850	.PRD-FOOD COOKING MACHINE OPERATORS AND TENDERS\
		7855	.PRD-FOOD PROCESSING WORKERS, ALL OTHER
		7900	.PRD-COMPUTER CONTROL PROGRAMMERS AND OPERATORS
		7920	.PRD-EXTRUDING AND DRAWING MACHINE SETTERS, OPERATORS, AND
			.TENDERS, METAL AND PLASTIC
		7930	.PRD-FORGING MACHINE SETTERS, OPERATORS, AND TENDERS, METAL
			.AND PLASTIC
		7940	.PRD-ROLLING MACHINE SETTERS, OPERATORS, AND TENDERS, METAL
			.AND PLASTIC
		7950	.PRD-MACHINE TOOL CUTTING SETTERS, OPERATORS, AND TENDERS,	
			.METAL AND PLASTIC
		8030	.PRD-MACHINISTS
		8040	.PRD-METAL FURNACE OPERATORS, TENDERS, POURERS, AND CASTERS
		8100	.PRD-MODEL MAKERS, PATTERNMAKERS, AND MOLDING MACHINE 
			.SETTERS, METAL AND PLASTIC
		8130	.PRD-TOOL AND DIE MAKERS
		8140	.PRD-WELDING, SOLDERING, AND BRAZING WORKERS
		8220	.PRD-MISCELLANEOUS METAL WORKERS AND PLASTIC WORKERS, 
			.INCLUDING MULTIPLE MACHINE TOOL SETTERS
		8250	.PRD-PREPRESS TECHNICIANS AND WORKERS
		8255	.PRD-PRINTING PRESS OPERATORS
		8256	.PRD-PRINT BINDING AND FINISHING WORKERS
		8300	.PRD-LAUNDRY AND DRY-CLEANING WORKERS
		8310	.PRD-PRESSERS, TEXTILE, GARMENT, AND RELATED MATERIALS
		8320	.PRD-SEWING MACHINE OPERATORS
		8330	.PRD-SHOE AND LEATHER WORKERS		
       		8350	.PRD-TAILORS, DRESSMAKERS, AND SEWERS
		8400	.PRD-TEXTILE BLEACHING AND DYEING, AND CUTTING MACHINE
			.SETTERS, OPERATORS, AND TENDERS
		8410	.PRD-TEXTILE KNITTING AND WEAVING MACHINE SETTERS,
			.OPERATORS, AND TENDERS
		8420	.PRD-TEXTILE WINDING, TWISTING, AND DRAWING OUT MACHINE
			.SETTERS, OPERATORS, AND TENDERS
		8450	.PRD-UPHOLSTERERS
		8460	.PRD-MISCELLANEOUS TEXTILE, APPAREL, AND FURNISHINGS
			.WORKERS, EXCEPT UPHOLSTERERS
		8500	.PRD-CABINETMAKERS AND BENCH CARPENTERS
		8510	.PRD-FURNITURE FINISHERS
		8530	.PRD-SAWING MACHINE SETTERS, OPERATORS, AND TENDERS, WOOD
		8540	.PRD-WOODWORKING MACHINE SETTERS, OPERATORS, AND TENDERS,
			.EXCEPT SAWING
		8550	.PRD-MISCELLANEOUS WOODWORKERS, INCLUDING MODEL MAKERS AND
			.PATTERNMAKERS
		8600	.PRD-POWER PLANT OPERATORS, DISTRIBUTORS, AND DISPATCHERS
		8610	.PRD-STATIONARY ENGINEERS AND BOILER OPERATORS
		8620	.PRD-WATER AND WASTEWATER TREATMENT PLANT AND SYSTEM
			.OPERATORS
		8630	.PRD-MISCELLANEOUS PLANT AND SYSTEM OPERATORS
		8640	.PRD-CHEMICAL PROCESSING MACHINE SETTERS, OPERATORS, AND
			.TENDERS
		8650	.PRD-CRUSHING, GRINDING, POLISHING, MIXING, AND BLENDING
			.WORKERS
		8710	.PRD-CUTTING WORKERS
		8720	.PRD-EXTRUDING, FORMING, PRESSING, AND COMPACTING MACHINE
			.SETTERS, OPERATORS, AND TENDERS
		8730	.PRD-FURNACE, KILN, OVEN, DRIER, AND KETTLE OPERATORS AND
			.TENDERS
		8740	.PRD-INSPECTORS, TESTERS, SORTERS, SAMPLERS, AND WEIGHERS
		8750	.PRD-JEWELERS AND PRECIOUS STONE AND METAL WORKERS
		8760	.PRD-MEDICAL, DENTAL, AND OPHTHALMIC LABORATORY TECHNICIANS
		8800	.PRD-PACKAGING AND FILLING MACHINE OPERATORS AND TENDERS
		8810	.PRD-PAINTING WORKERS
		8830	.PRD-PHOTOGRAPHIC PROCESS WORKERS AND PROCESSING MACHINE
			.OPERATORS
		8850	.PRD-ADHESIVE BONDING MACHINE OPERATORS AND TENDERS
		8910	.PRD-ETCHERS AND ENGRAVERS
		8920	.PRD-MOLDERS, SHAPERS, AND CASTERS, EXCEPT METAL AND 
			.PLASTIC
		8930	.PRD-PAPER GOODS MACHINE SETTERS, OPERATORS, AND TENDERS
		8940	.PRD-TIRE BUILDERS
		8950	.PRD-HELPERS-PRODUCTION WORKERS
		8965	.PRD-MISCELLANEOUS PRODUCTION WORKERS, INCLUDING 
			.SEMICONDUCTOR PROCESSORS
		9000	.TRN-SUPERVISORS OF TRANSPORTATION AND MATERIAL MOVING 
       			.WORKERS
		9030	.TRN-AIRCRAFT PILOTS AND FLIGHT ENGINEERS
		9040	.TRN-AIR TRAFFIC CONTROLLERS AND AIRFIELD OPERATIONS
			.SPECIALISTS
		9050	.TRN-FLIGHT ATTENDANTS
		9110	.TRN-AMBULANCE DRIVERS AND ATTENDANTS, EXCEPT EMERGENCY
			.MEDICAL TECHNICIANS
		9120	.TRN-BUS DRIVERS
		9130	.TRN-DRIVER/SALES WORKERS AND TRUCK DRIVERS
		9140	.TRN-TAXI DRIVERS AND CHAUFFEURS
		9150	.TRN-MOTOR VEHICLE OPERATORS, ALL OTHER
		9200	.TRN-LOCOMOTIVE ENGINEERS AND OPERATORS
		9240	.TRN-RAILROAD CONDUCTORS AND YARDMASTERS
		9260	.TRN-SUBWAY, STREETCAR, AND OTHER RAIL TRANSPORTATION
			.WORKERS
		9300	.TRN-SAILORS AND MARINE OILERS, AND SHIP ENGINEERS
		9310	.TRN-SHIP AND BOAT CAPTAINS AND OPERATORS
		9350	.TRN-PARKING LOT ATTENDANTS
		9360	.TRN-AUTOMOTIVE AND WATERCRAFT SERVICE ATTENDANTS
		9410	.TRN-TRANSPORTATION INSPECTORS
		9415	.TRN-TRANSPORTATION ATTENDANTS, EXCEPT FLIGHT ATTENDANTS
		9420	.TRN-MISCELLANEOUS TRANSPORTATION WORKERS, INCLUDING BRIDGE
			.AND LOCK TENDERS AND TRAFFIC TECHNICIANS
		9510	.TRN-CRANE AND TOWER OPERATORS
		9520	.TRN-DREDGE, EXCAVATING, AND LOADING MACHINE OPERATORS
		9560	.TRN-CONVEYOR OPERATORS AND TENDERS, AND HOIST AND WINCH
			.OPERATORS
		9600	.TRN-INDUSTRIAL TRUCK AND TRACTOR OPERATORS
		9610	.TRN-CLEANERS OF VEHICLES AND EQUIPMENT
		9620	.TRN-LABORERS AND FREIGHT, STOCK, AND MATERIAL MOVERS, HAND
		9630	.TRN-MACHINE FEEDERS AND OFFBEARERS
		9640	.TRN-PACKERS AND PACKAGERS, HAND
		9650	.TRN-PUMPING STATION OPERATORS
		9720	.TRN-REFUSE AND RECYCLABLE MATERIAL COLLECTORS
		9750	.TRN-MISCELLANEOUS MATERIAL MOVING WORKERS, INCLUDING
			.MINE SHUTTLE CAR OPERATORS, AND TANK CAR, TRUCK, AND SHIP
			.LOADERS
		9800	.MIL-MILITARY OFFICER SPECIAL AND TACTICAL OPERATIONS 
       			.LEADERS
		9810	.MIL-FIRST-LINE ENLISTED MILITARY SUPERVISORS
		9820	.MIL-MILITARY ENLISTED TACTICAL OPERATIONS AND AIR/WEAPONS
			.SPECIALISTS AND CREW MEMBERS
		9830	.MIL-MILITARY, RANK NOT SPECIFIED **
		9920	.UNEMPLOYED AND LAST WORKED 5 YEARS AGO OR EARLIER OR NEVER
			.WORKED **

Note: For additional information on NAICS and SOC groupings within major categories visit our website at: https://www.census.gov/people/io/methodology/indexes.html.	

PAOC		1
	Presence and age of own children
		b .N/A (male/female under 16 years old/GQ)
		1 .Females with own children under 6 years only
		2 .Females with own children 6 to 17 years only
		3 .Females with own children under 6 years and 6 to 17 years
		4 .Females with no own children

PERNP		7	
	Total person's earnings
		bbbbbbb 		.N/A (less than 15 years old)
		0000000 		.No earnings
		-010000 		.Loss of $10000 or more (Rounded & bottom-coded 
       					.components) 
		-000001..-009999 	.Loss $1 to $9999 (Rounded components)
		0000001 		.$1 or break even
		0000002..9999999 	.$2 to $9999999 (Rounded & top-coded components)
		
Note: Use ADJINC to adjust PERNP to constant dollars.

PINCP		7	
	Total person's income (signed)
		bbbbbbb 		.N/A (less than 15 years old)
		0000000 		.None
		-019999 		.Loss of $19999 or more (Rounded & bottom-coded 
       					.components) 
		-000001..-019998 	.Loss $1 to $19998 (Rounded components)
		0000001 		.$1 or break even
		0000002..9999999 	.$2 to $9999999 (Rounded & top-coded components)
			
Note: Use ADJINC to adjust PINCP to constant dollars.

POBP		3	
	Place of birth (Recode)
		001 .Alabama/AL
		002 .Alaska/AK
		004 .Arizona/AZ
		005 .Arkansas/AR
		006 .California/CA
		008 .Colorado/CO
		009 .Connecticut/CT
		010 .Delaware/DE
		011 .District of Columbia/DC
		012 .Florida/FL
		013 .Georgia/GA
		015 .Hawaii/HI
		016 .Idaho/ID
		017 .Illinois/IL
		018 .Indiana/IN
		019 .Iowa/IA
		020 .Kansas/KS
		021 .Kentucky/KY
		022 .Louisiana/LA
		023 .Maine/ME
		024 .Maryland/MD
		025 .Massachusetts/MA
		026 .Michigan/MI
		027 .Minnesota/MN
		028 .Mississippi/MS
		029 .Missouri/MO
		030 .Montana/MT
		031 .Nebraska/NE
		032 .Nevada/NV
		033 .New Hampshire/NH
		034 .New Jersey/NJ
		035 .New Mexico/NM
		036 .New York/NY
		037 .North Carolina/NC
		038 .North Dakota/ND
		039 .Ohio/OH
		040 .Oklahoma/OK
		041 .Oregon/OR
		042 .Pennsylvania/PA
		044 .Rhode Island/RI
		045 .South Carolina/SC
		046 .South Dakota/SD
		047 .Tennessee/TN
		048 .Texas/TX
		049 .Utah/UT
		050 .Vermont/VT
		051 .Virginia/VA
		053 .Washington/WA
		054 .West Virginia/WV
		055 .Wisconsin/WI
		056 .Wyoming/WY
		060 .American Samoa
		066 .Guam
		069 .Commonwealth of the Northern Mariana Islands
		072 .Puerto Rico
		078 .US Virgin Islands
		100 .Albania
		102 .Austria
		103 .Belgium
		104 .Bulgaria
		105 .Czechoslovakia
		106 .Denmark
		108 .Finland
		109 .France
		110 .Germany
		116 .Greece
		117 .Hungary
		118 .Iceland
		119 .Ireland
		120 .Italy
		126 .Netherlands
		127 .Norway
		128 .Poland
		129 .Portugal
		130 .Azores Islands
		132 .Romania
		134 .Spain
		136 .Sweden
		137 .Switzerland
		138 .United Kingdom, Not Specified
		139 .England
		140 .Scotland
		147 .Yugoslavia
		148 .Czech Republic
		149 .Slovakia
		150 .Bosnia and Herzegovina
		151 .Croatia
		152 .Macedonia
		154 .Serbia
		156 .Latvia
		157 .Lithuania
		158 .Armenia
		159 .Azerbaijan
		160 .Belarus
		161 .Georgia
		162 .Moldova
		163 .Russia
		164 .Ukraine
		165 .USSR
		168 .Montenegro
		169 .Other Europe, Not Specified
		200 .Afghanistan
		202 .Bangladesh
		203 .Bhutan
		205 .Myanmar
		206 .Cambodia
		207 .China
		208 .Cyprus
		209 .Hong Kong
		210 .India
		211 .Indonesia
		212 .Iran
		213 .Iraq
		214 .Israel
		215 .Japan
		216 .Jordan
		217 .Korea
		218 .Kazakhstan
		222 .Kuwait
		223 .Laos
		224 .Lebanon
		226 .Malaysia
		229 .Nepal
		231 .Pakistan
		233 .Philippines
		235 .Saudi Arabia
		236 .Singapore
		238 .Sri Lanka
		239 .Syria
		240 .Taiwan
		242 .Thailand
		243 .Turkey
		245 .United Arab Emirates
		246 .Uzbekistan
		247 .Vietnam
		248 .Yemen
		249 .Asia
		253 .South Central Asia, Not Specified
		254 .Other Asia, Not Specified
		300 .Bermuda
		301 .Canada
		303 .Mexico
		310 .Belize
		311 .Costa Rica
		312 .El Salvador
		313 .Guatemala
		314 .Honduras
		315 .Nicaragua
		316 .Panama
		321 .Antigua & Barbuda
		323 .Bahamas
		324 .Barbados
		327 .Cuba
		328 .Dominica
		329 .Dominican Republic
		330 .Grenada
		332 .Haiti
		333 .Jamaica
		339 .St. Lucia
		340 .St. Vincent & the Grenadines
		341 .Trinidad & Tobago
		343 .West Indies
		344 .Caribbean, Not Specified
		360 .Argentina
		361 .Bolivia
		362 .Brazil
		363 .Chile
		364 .Colombia
		365 .Ecuador
		368 .Guyana
		369 .Paraguay
		370 .Peru
		372 .Uruguay
		373 .Venezuela
		374 .South America
		399 .Americas, Not Specified
		400 .Algeria
		407 .Cameroon
		408 .Cabo Verde
		412 .Congo
		414 .Egypt
		416 .Ethiopia
		417 .Eritrea
		420 .Gambia
		421 .Ghana
		423 .Guinea
		427 .Kenya
		429 .Liberia
		430 .Libya
		436 .Morocco
		440 .Nigeria
		444 .Senegal
		447 .Sierra Leone
		448 .Somalia
		449 .South Africa
		451 .Sudan
		453 .Tanzania
		454 .Togo
		457 .Uganda
		459 .Democratic Republic of Congo (Zaire)
		460 .Zambia
		461 .Zimbabwe
		462 .Africa
		463 .Eastern Africa, Not Specified
		464 .Northern Africa, Not Specified
		467 .Western Africa, Not Specified
		468 .Other Africa, Not Specified
		501 .Australia
		508 .Fiji
		511 .Marshall Islands
		512 .Micronesia
		515 .New Zealand
		523 .Tonga
		527 .Samoa
		554 .Other US Island Areas, Oceania, Not Specified, or at Sea

POVPIP		3	
	Income-to-poverty ratio recode
		bbb 	   .N/A
		000..500   .Below 501 percent
		501	   .501 percent or more

POWPUMA		5	
	Place of work PUMA based on 2010 Census definition
		bbbbb 	     .N/A (not a worker--not in the labor force,
			     .force, including persons under 16 years;
			     .unemployed; civilian employed, with a job not
			     .at work; Armed Forces, with a job but not at work)
		00001 	     .Did not work in the United States or in Puerto Rico
		00100..70100 .Assigned Place of work PUMA.  Use with POWSP.

POWSP		3	
	Place of work - State or foreign country recode	
		bbb .N/A (not a worker--not in the labor force,
		    .including persons under 16 years; unemployed;
		    .employed, with a job not at work; Armed Forces,
		    .with a job but not at work)
		001 .Alabama/AL
		002 .Alaska/AK
		004 .Arizona/AZ
		005 .Arkansas/AR
		006 .California/CA
		008 .Colorado/CO
		009 .Connecticut/CT
		010 .Delaware/DE
		011 .District of Columbia/DC
		012 .Florida/FL
		013 .Georgia/GA
		015 .Hawaii/HI
		016 .Idaho/ID
		017 .Illinois/IL
		018 .Indiana/IN
		019 .Iowa/IA
		020 .Kansas/KS
		021 .Kentucky/KY
		022 .Louisiana/LA
		023 .Maine/ME
		024 .Maryland/MD
		025 .Massachusetts/MA
		026 .Michigan/MI
		027 .Minnesota/MN
		028 .Mississippi/MS
		029 .Missouri/MO
		030 .Montana/MT
		031 .Nebraska/NE
		032 .Nevada/NV
		033 .New Hampshire/NH
		034 .New Jersey/NJ
		035 .New Mexico/NM
		036 .New York/NY
		037 .North Carolina/NC
		038 .North Dakota/ND
		039 .Ohio/OH
		040 .Oklahoma/OK
		041 .Oregon/OR
		042 .Pennsylvania/PA
		044 .Rhode Island/RI
		045 .South Carolina/SC
		046 .South Dakota/SD
		047 .Tennessee/TN
		048 .Texas/TX
		049 .Utah/UT
		050 .Vermont/VT
		051 .Virginia/VA
		053 .Washington/WA
		054 .West Virginia/WV
		055 .Wisconsin/WI
		056 .Wyoming/WY
		072 .Puerto Rico
		166 .Europe
		251 .Eastern Asia
		254 .Other Asia, Not Specified
		301 .Canada
		303 .Mexico
		399 .Americas, Not Specified
		555 .Other US Island Areas Not Specified, Africa, Oceania, at
		    .Sea, or Abroad, Not Specified

PRIVCOV		1	
	Private health insurance coverage recode
		1 .With private health insurance coverage
		2 .Without private health insurance coverage

PUBCOV		1	
	Public health coverage recode
		1 .With public health coverage
		2 .Without public health coverage

QTRBIR		1	
	Quarter of birth
		1 .January through March
		2 .April through June
		3 .July through September
		4 .October through December

RAC1P		1	
	Recoded detailed race code
		1 .White alone		
		2 .Black or African American alone	
		3 .American Indian alone		
		4 .Alaska Native alone		
		5 .American Indian and Alaska Native tribes specified; or American
		  .Indian or Alaska Native, not specified and no other races
		6 .Asian alone		
		7 .Native Hawaiian and Other Pacific Islander alone
		8 .Some Other Race alone		
		9 .Two or More Races		

RAC2P		2	
	Recoded detailed race code
		01 .White alone
		02 .Black or African American alone
		03 .Apache alone
		04 .Blackfeet alone
		05 .Cherokee alone
		06 .Cheyenne alone
		07 .Chickasaw alone
		08 .Chippewa alone
		09 .Choctaw alone
		10 .Comanche alone
		11 .Creek alone
		12 .Crow alone
		13 .Hopi alone
		14 .Iroquois alone
		15 .Lumbee alone
		16 .Mexican American Indian alone
		17 .Navajo alone
		18 .Pima alone
		19 .Potawatomi alone
		20 .Pueblo alone
		21 .Puget Sound Salish alone
		22 .Seminole alone
		23 .Sioux alone
		24 .South American Indian alone
		25 .Tohono O'Odham alone
		26 .Yaqui alone
		27 .Other specified American Indian tribes alone
		28 .All other specified American Indian tribe combinations
		29 .American Indian, tribe not specified
		30 .Alaskan Athabascan alone
		31 .Tlingit-Haida alone
		32 .Inupiat alone
		33 .Yup’ik alone
		34 .Aleut alone
		35 .Other Alaska Native
		36 .Other American Indian and Alaska Native specified
		37 .American Indian and Alaska Native, not specified
		38 .Asian Indian alone
		39 .Bangladeshi alone
		40 .Bhutanese alone
		41 .Burmese alone
		42 .Cambodian alone
		43 .Chinese, except Taiwanese, alone
		44 .Taiwanese alone
		45 .Filipino alone		
		46 .Hmong alone
		47 .Indonesian alone
		48 .Japanese alone
		49 .Korean alone
		50 .Laotian alone
		51 .Malaysian alone
		52 .Mongolian alone
		53 .Nepalese alone
		54 .Pakistani alone
		55 .Sri Lankan alone
		56 .Thai alone
		57 .Vietnamese alone
		58 .Other Asian alone
		59 .All combinations of Asian races only
		60 .Native Hawaiian alone
		61 .Samoan alone
		62 .Tongan alone
		63 .Guamanian or Chamorro alone
		64 .Marshallese alone
		65 .Fijian alone
		66 .Other Native Hawaiian and Other Pacific Islander
		67 .Some Other Race alone
		68 .Two or More Races

RAC3P		3	
	Recoded detailed race code 
		001 .White alone
		002 .Black or African American alone
		003 .American Indian and Alaska Native alone
		004 .Asian Indian alone
		005 .Chinese alone
		006 .Filipino alone
		007 .Japanese alone
		008 .Korean alone
		009 .Vietnamese alone
		010 .Other Asian alone
		011 .Native Hawaiian alone
		012 .Guamanian or Chamorro alone
		013 .Samoan alone
		014 .Other Pacific Islander alone
		015 .Some Other Race alone
		016 .White; Black or African American
		017 .White; American Indian and Alaska Native
		018 .White; Asian Indian
		019 .White; Chinese
		020 .White; Filipino
		021 .White; Japanese
		022 .White; Korean
		023 .White; Vietnamese
		024 .White; Other Asian
		025 .White; Native Hawaiian
		026 .White; Guamanian or Chamorro
		027 .White; Samoan
		028 .White; Other Pacific Islander
		029 .White; Some Other Race
		030 .Black or African American; American Indian and Alaska Native
		031 .Black or African American; Asian Indian
		032 .Black or African American; Chinese
		033 .Black or African American; Filipino
		034 .Black or African American; Japanese
		035 .Black or African American; Korean
		036 .Black or African American; Other Asian
		037 .Black or African American; Other Pacific Islander
		038 .Black or African American; Some Other Race
		039 .American Indian and Alaska Native; Asian Indian
		040 .American Indian and Alaska Native; Filipino
		041 .American Indian and Alaska Native; Some Other Race
		042 .Asian Indian; Other Asian
		043 .Asian Indian; Some Other Race
		044 .Chinese; Filipino
		045 .Chinese; Japanese
		046 .Chinese; Korean
		047 .Chinese; Vietnamese
		048 .Chinese; Other Asian
		049 .Chinese; Native Hawaiian
		050 .Filipino; Japanese
		051 .Filipino; Native Hawaiian
		052 .Filipino; Other Pacific Islander
		053 .Filipino; Some Other Race
		054 .Japanese; Korean
		055 .Japanese; Native Hawaiian
		056 .Vietnamese; Other Asian
		057 .Other Asian; Other Pacific Islander
		058 .Other Asian; Some Other Race
		059 .Other Pacific Islander; Some Other Race
		060 .White; Black or African American; American Indian and Alaska
		    .Native	
		061 .White; Black or African American; Filipino
		062 .White; Black or African American; Some Other Race
		063 .White; American Indian and Alaska Native; Filipino
		064 .White; American Indian and Alaska Native; Some Other Race
		065 .White; Chinese; Filipino
		066 .White; Chinese; Japanese
		067 .White; Chinese; Native Hawaiian
		068 .White; Filipino; Native Hawaiian
		069 .White; Japanese; Native Hawaiian
		070 .White; Other Asian; Some Other Race
		071 .Chinese; Filipino; Native Hawaiian
		072 .White; Chinese; Filipino; Native Hawaiian
		073 .White; Chinese; Japanese; Native Hawaiian
		074 .Black or African American; Asian groups
		075 .Black or African American; Native Hawaiian and Other Pacific
		    .Islander groups
		076 .Asian Indian; Asian groups
		077 .Filipino; Asian groups
		078 .White; Black or African American; Asian groups
		079 .White; American Indian and Alaska Native; Asian groups
		080 .White; Native Hawaiian and Other Pacific Islander groups;
		    .and/or Some Other Race
		081 .White; Black or African American; American Indian and Alaska
		    .Native; Asian groups
		082 .White; Black or African American; American Indian and Alaska
		    .Native; and/or Native Hawaiian and Other Pacific Islander
		    .groups; and/or Some Other Race
		083 .White; Black or African American; and/or Asian groups;
		    .and/or Native Hawaiian and Other Pacific Islander groups;
		    .and/or Some Other Race
		084 .White; American Indian and Alaska Native; and/or Asian
		    .groups; and/or Native Hawaiian and Other Pacific Islander
		    .groups
		085 .White; Chinese; Filipino; and/or Asian groups; and/or Native
		    .Hawaiian and Other Pacific Islander groups; and/or Some
		    .Other Race
		086 .White; Chinese; and/or Asian groups; and/or Native Hawaiian
		    .and Other Pacific Islander groups; and/or Some Other Race
		087 .White; Filipino; and/or Native Hawaiian and Other Pacific
		    .Islander groups; and/or Some Other Race
		088 .White; Japanese; and/or Asian groups; and/or Native Hawaiian
		    .and Other Pacific Islander groups; and/or Some Other Race
		089 .White; Asian groups; and/or Native Hawaiian and Other 
		    .Pacific Islander groups; and/or Some Other Race
		090 .Black or African American; American Indian and Alaska 
		    .Native; and/or Asian groups; and/or Native Hawaiian and
		    .Other
		    .Pacific Islander groups; and/or Some Other Race
		091 .Black or African American; Asian groups; and/or Native
		    .Hawaiian and Other Pacific Islander groups; and/or Some 
		    .Other Race
		092 .American Indian and Alaska Native; Asian groups; and/or 
		    .Native Hawaiian and Other Pacific Islander groups; and/or
		    .Some Other Race
		093 .Asian Indian; and/or White; and/or Asian groups; and/or 
		    .Native Hawaiian and Other Pacific Islander groups; and/or 
		    .Some Other Race
		094 .Chinese; Japanese; Native Hawaiian; and/or other Asian 
		    .and/or Pacific Islander groups
		095 .Chinese; and/or Asian groups; and/or Native Hawaiian and 
		    .Other Pacific Islander groups; and/or Some Other Race
		096 .Filipino; and/or Asian groups; and/or Native Hawaiian and
		    .Other Pacific Islander groups; and/or Some Other Race
		097 .Japanese; and/or Asian groups; and/or Native Hawaiian and
		    .Other Pacific Islander groups; and/or Some Other Race
		098 .Korean; and/or Vietnamese; and/or Other Asian; and/or Native
		    .Hawaiian and Other Pacific Islander groups; and/or Some
		    . Other Race
		099 .Native Hawaiian; and/or Pacific Islander groups; and/or Some
		    .Other Race
		100 .White; and/or Black or African American; and/or American
		    .Indian and Alaska Native; and/or Asian groups; and/or Native
		    .Hawaiian and Other Pacific Islander groups; and/or Some
		    .Other Race

RACAIAN		1	
	American Indian and Alaska Native recode (American Indian and 
	Alaska Native alone or in combination with one or more other  
	races)
		0 .No
		1 .Yes

RACASN		1	
	Asian recode (Asian alone or in combination with one or more
	other races)
		0 .No
		1 .Yes

RACBLK		1
	Black or African American recode (Black alone or in		
	combination with one or more other races)
		0 .No
		1 .Yes

RACNH		1
	Native Hawaiian recode (Native Hawaiian alone or in combination with 
	one or more other races)
		0 .No
		1 .Yes

RACNUM		1	
	Number of major race groups represented
		1..6 .Race groups

RACPI		1
	Other Pacific Islander recode (Other Pacific Islander alone or in
	combination with one or more other races)
		0 .No
		1 .Yes

RACSOR		1	
	Some other race recode (Some other race alone or in		
	combination with one or more other races)
		0 .No
		1 .Yes

RACWHT		1	
	White recode (White alone or in combination with one or more
	other races)
		0 .No
		1 .Yes

RC		1	
	Related child
		0 .No (includes GQ)	
		1 .Yes		

SCIENGP		1	
	Field of Degree Science and Engineering Flag – NSF Definition
		b .N/A (less than bachelor’s degree)
		1 .Yes		
		2 .No		

SCIENGRLP	1	
	Field of Degree Science and Engineering Related Flag – NSF Definition
		b .N/A (less than bachelor’s degree)
		1 .Yes		
		2 .No		

SFN		1	
	Subfamily number
		b .N/A (GQ/not in a subfamily)
		1 .In subfamily 1
		2 .In subfamily 2
		3 .In subfamily 3
		4 .In subfamily 4

SFR		1	
	Subfamily relationship
		b .N/A (GQ/not in a subfamily)
		1 .Husband/wife no children
		2 .Husband/wife with children
		3 .Parent in a parent/child subfamily
		4 .Child in a married-couple subfamily
		5 .Child in a mother-child subfamily
		6 .Child in a father-child subfamily

SOCP		6	
	SOC Occupation code for 2012 and later based on 2010 SOC codes   
		bbbbbb .N/A (less than 16 years old/NILF who last worked	
			 .more than 5 years ago or never worked)
		1110XX .MGR-CHIEF EXECUTIVES AND LEGISLATORS *
		111021 .MGR-GENERAL AND OPERATIONS MANAGERS
		112011 .MGR-ADVERTISING AND PROMOTIONS MANAGERS
		112020 .MGR-MARKETING AND SALES MANAGERS
		112031 .MGR-PUBLIC RELATIONS AND FUNDRAISING MANAGERS
		113011 .MGR-ADMINISTRATIVE SERVICES MANAGERS
		113021 .MGR-COMPUTER AND INFORMATION SYSTEMS MANAGERS
		113031 .MGR-FINANCIAL MANAGERS
		113111 .MGR-COMPENSATION AND BENEFITS MANAGERS
		113121 .MGR-HUMAN RESOURCES MANAGERS
		113131 .MGR-TRAINING AND DEVELOPMENT MANAGERS
		113051 .MGR-INDUSTRIAL PRODUCTION MANAGERS
		113061 .MGR-PURCHASING MANAGERS
		113071 .MGR-TRANSPORTATION, STORAGE, AND DISTRIBUTION MANAGERS
		119013 .MGR-FARMERS, RANCHERS, AND OTHER AGRICULTURAL MANAGERS
		119021 .MGR-CONSTRUCTION MANAGERS
		119030 .MGR-EDUCATION ADMINISTRATORS
		119041 .MGR-ARCHITECTURAL AND ENGINEERING MANAGERS
		119051 .MGR-FOOD SERVICE MANAGERS
		119071 .MGR-GAMING MANAGERS
		119081 .MGR-LODGING MANAGERS
		119111 .MGR-MEDICAL AND HEALTH SERVICES MANAGERS
		119121 .MGR-NATURAL SCIENCES MANAGERS
		119141 .MGR-PROPERTY, REAL ESTATE, AND COMMUNITY ASSOCIATION
		       .MANAGERS
		119151 .MGR-SOCIAL AND COMMUNITY SERVICE MANAGERS
		119161 .MGR-EMERGENCY MANAGEMENT DIRECTORS
		119XXX .MGR-MISCELLANEOUS MANAGERS, INCLUDING FUNERAL SERVICE
		       .MANAGERS AND POSTMASTERS AND MAIL SUPERINTENDENTS *
		131011 .BUS-AGENTS AND BUSINESS MANAGERS OF ARTISTS, PERFORMERS,
		       .AND ATHLETES
		131021 .BUS-BUYERS AND PURCHASING AGENTS, FARM PRODUCTS
		131022 .BUS-WHOLESALE AND RETAIL BUYERS, EXCEPT FARM PRODUCTS
		131023 .BUS-PURCHASING AGENTS, EXCEPT WHOLESALE, RETAIL, AND FARM
		       .PRODUCTS
		131030 .BUS-CLAIMS ADJUSTERS, APPRAISERS, EXAMINERS, AND
		       .INVESTIGATORS
		131041 .BUS-COMPLIANCE OFFICERS
		131051 .BUS-COST ESTIMATORS
		131070 .BUS-HUMAN RESOURCES WORKERS
		131081 .BUS-LOGISTICIANS
		131111 .BUS-MANAGEMENT ANALYSTS
		131121 .BUS-MEETING CONVENTION, AND EVENT PLANNERS
		131131 .BUS-FUNDRAISERS
		131141 .BUS-COMPENSATION, BENEFITS, AND JOB ANALYSIS SPECIALISTS
		131151 .BUS-TRAINING AND DEVELOPMENT SPECIALISTS
		131161 .BUS-MARKET RESEARCH ANALYSTS AND MARKETING SPECIALISTS
		131199 .BUS-BUSINESS OPERATIONS SPECIALISTS, ALL OTHER
		132011 .FIN-ACCOUNTANTS AND AUDITORS
		132021 .FIN-APPRAISERS AND ASSESSORS OF REAL ESTATE
		132031 .FIN-BUDGET ANALYSTS
		132041 .FIN-CREDIT ANALYSTS
		132051 .FIN-FINANCIAL ANALYSTS
		132052 .FIN-PERSONAL FINANCIAL ADVISORS
		132053 .FIN-INSURANCE UNDERWRITERS
		132061 .FIN-FINANCIAL EXAMINERS
		132070 .FIN-CREDIT COUNSELORS AND LOAN OFFICERS
		132081 .FIN-TAX EXAMINERS AND COLLECTORS, AND REVENUE AGENTS
		132082 .FIN-TAX PREPARERS
		132099 .FIN-FINANCIAL SPECIALISTS, ALL OTHER
		151111 .CMM-COMPUTER AND INFORMATION RESEARCH SCIENTISTS
		151121 .CMM-COMPUTER SYSTEMS ANALYSTS
		151122 .CMM-INFORMATION SECURITY ANALYSTS
		151131 .CMM-COMPUTER PROGRAMMERS
		15113X .CMM-SOFTWARE DEVELOPERS,APPLICATIONS AND SYSTEMS SOFTWARE*
		151134 .CMM-WEB DEVELOPERS
		151141 .CMM-DATABASE ADMINISTRATORS
		151142 .CMM-NETWORK AND COMPUTER SYSTEMS ADMINISTRATORS
		151143 .CMM-COMPUTER NETWORK ARCHITECTS
		151150 .CMM-COMPUTER SUPPORT SPECIALISTS
		151199 .CMM-COMPUTER OCCUPATIONS, ALL OTHER
		152011 .CMM-ACTUARIES
		152031 .CMM-OPERATIONS RESEARCH ANALYSTS
		1520XX .CMM-MISCELLANEOUS MATHEMATICAL SCIENCE OCCUPATIONS,
		       .INCLUDING MATHEMATICIANS AND STATISTICIANS *
		171010 .ENG-ARCHITECTS, EXCEPT NAVAL
		171020 .ENG-SURVEYORS, CARTOGRAPHERS, AND PHOTOGRAMMETRISTS
		172011 .ENG-AEROSPACE ENGINEERS
		1720XX .ENG-BIOMEDICAL AND AGRICULTURAL ENGINEERS *
		172041 .ENG-CHEMICAL ENGINEERS
		172051 .ENG-CIVIL ENGINEERS
		172061 .ENG-COMPUTER HARDWARE ENGINEERS
		172070 .ENG-ELECTRICAL AND ELECTRONICS ENGINEERS
		172081 .ENG-ENVIRONMENTAL ENGINEERS
		172110 .ENG-INDUSTRIAL ENGINEERS, INCLUDING HEALTH AND SAFETY
		172121 .ENG-MARINE ENGINEERS AND NAVAL ARCHITECTS
		172131 .ENG-MATERIALS ENGINEERS
		172141 .ENG-MECHANICAL ENGINEERS
		1721XX .ENG-PETROLEUM, MINING AND GEOLOGICAL ENGINEERS, INCLUDING
		       .MINING SAFETY ENGINEERS *
		1721YY .ENG-MISCELLANEOUS ENGINEERS, INCLUDING NUCLEAR ENGINEERS *
		173010 .ENG-DRAFTERS
		173020 .ENG-ENGINEERING TECHNICIANS, EXCEPT DRAFTERS
		173031 .ENG-SURVEYING AND MAPPING TECHNICIANS
		191010 .SCI-AGRICULTURAL AND FOOD SCIENTISTS
		191020 .SCI-BIOLOGICAL SCIENTISTS
		191030 .SCI-CONSERVATION SCIENTISTS AND FORESTERS
		1910XX .SCI-MEDICAL SCIENTISTS, AND LIFE SCIENTISTS, ALL OTHER *
		192010 .SCI-ASTRONOMERS AND PHYSICISTS
		192021 .SCI-ATMOSPHERIC AND SPACE SCIENTISTS
		192030 .SCI-CHEMISTS AND MATERIALS SCIENTISTS
		192040 .SCI-ENVIRONMENTAL SCIENTISTS AND GEOSCIENTISTS
		192099 .SCI-PHYSICAL SCIENTISTS, ALL OTHER
		193011 .SCI-ECONOMISTS
		193030 .SCI-PSYCHOLOGISTS
		193051 .SCI-URBAN AND REGIONAL PLANNERS
		1930XX .SCI-MISCELLANEOUS SOCIAL SCIENTISTS, INCLUDING SURVEY
		       .RESEARCHERS AND SOCIOLOGISTS *
		194011 .SCI-AGRICULTURAL AND FOOD SCIENCE TECHNICIANS
		194021 .SCI-BIOLOGICAL TECHNICIANS
		194031 .SCI-CHEMICAL TECHNICIANS
		1940XX .SCI-GEOLOGICAL AND PETROLEUM TECHNICIANS, AND NUCLEAR
		       .TECHNICIANS *
		1940YY .SCI-MISCELLANEOUS LIFE, PHYSICAL, AND SOCIAL SCIENCE
		       .TECHNICIANS, INCLUDING SOCIAL SCIENCE RESEARCH ASSISTANTS*
		211010 .CMS-COUNSELORS
		211020 .CMS-SOCIAL WORKERS
		211092 .CMS-PROBATION OFFICERS AND CORRECTIONAL TREATMENT
		       .SPECIALISTS
		211093 .CMS-SOCIAL AND HUMAN SERVICE ASSISTANTS
		21109X .CMS-MISCELLANEOUS COMMUNITY AND SOCIAL SERVICE 
       		       .SPECIALISTS,INCLUDING HEALTH EDUCATORS AND COMMUNITY 
        	       .HEALTH WORKERS *
		212011 .CMS-CLERGY
		212021 .CMS-DIRECTORS, RELIGIOUS ACTIVITIES AND EDUCATION
		212099 .CMS-RELIGIOUS WORKERS, ALL OTHER
		2310XX .LGL-LAWYERS, AND JUDGES, MAGISTRATES, AND OTHER JUDICIAL
		       .WORKERS *
		231012 .LGL-JUDICIAL LAW CLERKS
		232011 .LGL-PARALEGALS AND LEGAL ASSISTANTS
		232090 .LGL-MISCELLANEOUS LEGAL SUPPORT WORKERS
		251000 .EDU-POSTSECONDARY TEACHERS
		252010 .EDU-PRESCHOOL AND KINDERGARTEN TEACHERS
		252020 .EDU-ELEMENTARY AND MIDDLE SCHOOL TEACHERS
		252030 .EDU-SECONDARY SCHOOL TEACHERS
		252050 .EDU-SPECIAL EDUCATION TEACHERS
		253000 .EDU-OTHER TEACHERS AND INSTRUCTORS
		254010 .EDU-ARCHIVISTS, CURATORS, AND MUSEUM TECHNICIANS
		254021 .EDU-LIBRARIANS
		254031 .EDU-LIBRARY TECHNICIANS
		259041 .EDU-TEACHER ASSISTANTS
		2590XX .EDU-OTHER EDUCATION, TRAINING, AND LIBRARY WORKERS *
		271010 .ENT-ARTISTS AND RELATED WORKERS
		271020 .ENT-DESIGNERS
		272011 .ENT-ACTORS
		272012 .ENT-PRODUCERS AND DIRECTORS
		272020 .ENT-ATHLETES, COACHES, UMPIRES, AND RELATED WORKERS
		272030 .ENT-DANCERS AND CHOREOGRAPHERS
		272040 .ENT-MUSICIANS, SINGERS, AND RELATED WORKERS
		272099 .ENT-ENTERTAINERS AND PERFORMERS, SPORTS AND RELATED
		       .WORKERS, ALL OTHER
		273010 .ENT-ANNOUNCERS
		273020 .ENT-NEWS ANALYSTS, REPORTERS AND CORRESPONDENTS
		273031 .ENT-PUBLIC RELATIONS SPECIALISTS
		273041 .ENT-EDITORS
		273042 .ENT-TECHNICAL WRITERS
		273043 .ENT-WRITERS AND AUTHORS
		273090 .ENT-MISCELLANEOUS MEDIA AND COMMUNICATION WORKERS
		2740XX .ENT-BROADCAST AND SOUND ENGINEERING TECHNICIANS AND RADIO
		       .OPERATORS, AND MEDIA AND COMMUNICATION EQUIPMENT WORKERS,
		       .ALL OTHER *
		274021 .ENT-PHOTOGRAPHERS
		274030 .ENT-TELEVISION, VIDEO, AND MOTION PICTURE CAMERA OPERATORS
		       .AND EDITORS
		291011 .MED-CHIROPRACTORS
		291020 .MED-DENTISTS
		291031 .MED-DIETITIANS AND NUTRITIONISTS
		291041 .MED-OPTOMETRISTS
		291051 .MED-PHARMACISTS
		291060 .MED-PHYSICIANS AND SURGEONS
		291071 .MED-PHYSICIAN ASSISTANTS
		291081 .MED-PODIATRISTS
		291122 .MED-OCCUPATIONAL THERAPISTS
		291123 .MED-PHYSICAL THERAPISTS
		291124 .MED-RADIATION THERAPISTS
		291125 .MED-RECREATIONAL THERAPISTS
		291126 .MED-RESPIRATORY THERAPISTS
		291127 .MED-SPEECH-LANGUAGE PATHOLOGISTS
		29112X .MED-OTHER THERAPISTS, INCLUDING EXERCISE PHYSIOLOGISTS *
		291131 .MED-VETERINARIANS
		291141 .MED-REGISTERED NURSES
		291151 .MED-NURSE ANESTHETISTS
		291181 .MED-AUDIOLOGISTS
		291199 .MED-HEALTH DIAGNOSING AND TREATING PRACTITIONERS, ALL 
        	       .OTHER
		2911XX .MED-NURSE PRACTITIONERS AND NURSE MIDWIVES *
		292010 .MED-CLINICAL LABORATORY TECHNOLOGISTS AND TECHNICIANS
		292021 .MED-DENTAL HYGIENISTS
		292030 .MED-DIAGNOSTIC RELATED TECHNOLOGISTS AND TECHNICIANS
		292041 .MED-EMERGENCY MEDICAL TECHNICIANS AND PARAMEDICS
		292050 .MED-HEALTH PRACTITIONER SUPPORT TECHNOLOGISTS AND 
		       .TECHNICIANS
		292061 .MED-LICENSED PRACTICAL AND LICENSED VOCATIONAL NURSES
		292071 .MED-MEDICAL RECORDS AND HEALTH INFORMATION TECHNICIANS
		292081 .MED-OPTICIANS, DISPENSING
		292090 .MED-MISCELLANEOUS HEALTH TECHNOLOGISTS AND TECHNICIANS
		299000 .MED-OTHER HEALTHCARE PRACTITIONERS AND TECHNICAL
		       .OCCUPATIONS
		311010 .HLS-NURSING, PSYCHIATRIC, AND HOME HEALTH AIDES
		312010 .HLS-OCCUPATIONAL THERAPY ASSISTANTS AND AIDES
		312020 .HLS-PHYSICAL THERAPIST ASSISTANTS AND AIDES
		319011 .HLS-MASSAGE THERAPISTS
		319091 .HLS-DENTAL ASSISTANTS
		319092 .HLS-MEDICAL ASSISTANTS
		319094 .HLS-MEDICAL TRANSCRIPTIONISTS
		319095 .HLS-PHARMACY AIDES
		319096 .HLS-VETERINARY ASSISTANTS AND LABORATORY ANIMAL CARETAKERS
		319097 .HLS-PHLEBOTOMISTS
		31909X .HEALTHCARE SUPPORT WORKERS, ALL OTHER, INCLUDING MEDICAL
		       .EQUIPMENT PREPARERS *
		331011 .PRT-FIRST-LINE SUPERVISORS OF CORRECTIONAL OFFICERS
		331012 .PRT-FIRST-LINE SUPERVISORS OF POLICE AND DETECTIVES
		331021 .PRT-FIRST-LINE SUPERVISORS OF FIRE FIGHTING AND PREVENTION
		       .WORKERS
		331099 .PRT-FIRST-LINE SUPERVISORS OF PROTECTIVE SERVICE WORKERS,
		       .ALL OTHER
		332011 .PRT-FIREFIGHTERS
		332020 .PRT-FIRE INSPECTORS
		333010 .PRT-BAILIFFS, CORRECTIONAL OFFICERS, AND JAILERS
		333021 .PRT-DETECTIVES AND CRIMINAL INVESTIGATORS
		3330XX .PRT-MISCELLANEOUS LAW ENFORCEMENT WORKERS *
		333050 .PRT-POLICE OFFICERS
		339011 .PRT-ANIMAL CONTROL WORKERS
		339021 .PRT-PRIVATE DETECTIVES AND INVESTIGATORS
		339030 .PRT-SECURITY GUARDS AND GAMING SURVEILLANCE OFFICERS
		339091 .PRT-CROSSING GUARDS
		339093 .PRT-TRANSPORTATION SECURITY SCREENERS
		33909X .PRT-LIFEGUARDS AND OTHER RECREATIONAL, AND ALL OTHER
		       .PROTECTIVE SERVICE WORKERS *
		351011 .EAT-CHEFS AND HEAD COOKS
		351012 .EAT-FIRST-LINE SUPERVISORS OF FOOD PREPARATION AND SERVING
		       .WORKERS
		352010 .EAT-COOKS
		352021 .EAT-FOOD PREPARATION WORKERS
		353011 .EAT-BARTENDERS
		353021 .EAT-COMBINED FOOD PREPARATION AND SERVING WORKERS,
		       .INCLUDING FAST FOOD
		353022 .EAT-COUNTER ATTENDANTS, CAFETERIA, FOOD CONCESSION, AND
		       .COFFEE SHOP
		353031 .EAT-WAITERS AND WAITRESSES
		353041 .EAT-FOOD SERVERS, NONRESTAURANT
		3590XX .EAT-MISCELLANEOUS FOOD PREPARATION AND SERVING RELATED
		       .WORKERS, INCLUDING DINING ROOM AND CAFETERIA ATTENDANTS 
 		       .AND BARTENDER HELPERS *
		359021 .EAT-DISHWASHERS
		359031 .EAT-HOSTS AND HOSTESSES, RESTAURANT, LOUNGE, AND COFFEE
		       .SHOP
		371011 .CLN-FIRST-LINE SUPERVISORS OF HOUSEKEEPING AND JANITORIAL
		       .WORKERS
		371012 .CLN-FIRST-LINE SUPERVISORS OF LANDSCAPING, LAWN SERVICE, 
		       .AND GROUNDSKEEPING WORKERS
		37201X .CLN-JANITORS AND BUILDING CLEANERS *
		372012 .CLN-MAIDS AND HOUSEKEEPING CLEANERS
		372021 .CLN-PEST CONTROL WORKERS
		373010 .CLN-GROUNDS MAINTENANCE WORKERS
		391010 .PRS-FIRST-LINE SUPERVISORS OF GAMING WORKERS
		391021 .PRS-FIRST-LINE SUPERVISORS OF PERSONAL SERVICE WORKERS
		392011 .PRS-ANIMAL TRAINERS
		392021 .PRS-NONFARM ANIMAL CARETAKERS
		393010 .PRS-GAMING SERVICES WORKERS
		393021 .PRS-MOTION PICTURE PROJECTIONISTS
		393031 .PRS-USHERS, LOBBY ATTENDANTS, AND TICKET TAKERS
		393090 .PRS-MISCELLANEOUS ENTERTAINMENT ATTENDANTS AND RELATED
		       .WORKERS
		394031 .PRS-MORTICIANS, UNDERTAKERS, AND FUNERAL DIRECTORS
		3940XX .PRS-EMBALMERS AND FUNERAL ATTENDANTS *
		395011 .PRS-BARBERS
		395012 .PRS-HAIRDRESSERS, HAIRSTYLISTS, AND COSMETOLOGISTS
		395090 .PRS-MISCELLANEOUS PERSONAL APPEARANCE WORKERS
		396010 .PRS-BAGGAGE PORTERS, BELLHOPS, AND CONCIERGES
		397010 .PRS-TOUR AND TRAVEL GUIDES
		399011 .PRS-CHILDCARE WORKERS
		399021 .PRS-PERSONAL CARE AIDES
		399030 .PRS-RECREATION AND FITNESS WORKERS
		399041 .PRS-RESIDENTIAL ADVISORS
		399099 .PRS-PERSONAL CARE AND SERVICE WORKERS, ALL OTHER
		411011 .SAL-FIRST-LINE SUPERVISORS OF RETAIL SALES WORKERS
		411012 .SAL-FIRST-LINE SUPERVISORS OF NON-RETAIL SALES WORKERS
		412010 .SAL-CASHIERS
		412021 .SAL-COUNTER AND RENTAL CLERKS
		412022 .SAL-PARTS SALESPERSONS
		412031 .SAL-RETAIL SALESPERSONS
		413011 .SAL-ADVERTISING SALES AGENTS
		413021 .SAL-INSURANCE SALES AGENTS
		413031 .SAL-SECURITIES, COMMODITIES, AND FINANCIAL SERVICES SALES
		       .AGENTS
		413041 .SAL-TRAVEL AGENTS
		413099 .SAL-SALES REPRESENTATIVES, SERVICES, ALL OTHER
		414010 .SAL-SALES REPRESENTATIVES, WHOLESALE AND MANUFACTURING
		419010 .SAL-MODELS, DEMONSTRATORS, AND PRODUCT PROMOTERS
		419020 .SAL-REAL ESTATE BROKERS AND SALES AGENTS
		419031 .SAL-SALES ENGINEERS
		419041 .SAL-TELEMARKETERS
		419091 .SAL-DOOR-TO-DOOR SALES WORKERS, NEWS AND STREET VENDORS,
		       .AND RELATED WORKERS
		419099 .SAL-SALES AND RELATED WORKERS, ALL OTHER
		431011 .OFF-FIRST-LINE SUPERVISORS OF OFFICE AND ADMINISTRATIVE
		       .SUPPORT WORKERS
		432011 .OFF-SWITCHBOARD OPERATORS, INCLUDING ANSWERING SERVICE
		432021 .OFF-TELEPHONE OPERATORS
		432099 .OFF-COMMUNICATIONS EQUIPMENT OPERATORS, ALL OTHER
		433011 .OFF-BILL AND ACCOUNT COLLECTORS
		433021 .OFF-BILLING AND POSTING CLERKS
		433031 .OFF-BOOKKEEPING, ACCOUNTING, AND AUDITING CLERKS
		433041 .OFF-GAMING CAGE WORKERS
		433051 .OFF-PAYROLL AND TIMEKEEPING CLERKS
		433061 .OFF-PROCUREMENT CLERKS
		433071 .OFF-TELLERS
		433099 .OFF-FINANCIAL CLERKS, ALL OTHER
		434011 .OFF-BROKERAGE CLERKS
		434031 .OFF-COURT, MUNICIPAL, AND LICENSE CLERKS
		434041 .OFF-CREDIT AUTHORIZERS, CHECKERS, AND CLERKS
		434051 .OFF-CUSTOMER SERVICE REPRESENTATIVES
		434061 .OFF-ELIGIBILITY INTERVIEWERS, GOVERNMENT PROGRAMS
		434071 .OFF-FILE CLERKS
		434081 .OFF-HOTEL, MOTEL, AND RESORT DESK CLERKS
		434111 .OFF-INTERVIEWERS, EXCEPT ELIGIBILITY AND LOAN
		434121 .OFF-LIBRARY ASSISTANTS, CLERICAL
		434131 .OFF-LOAN INTERVIEWERS AND CLERKS
		434141 .OFF-NEW ACCOUNTS CLERKS
		434XXX .OFF-CORRESPONDENCE CLERKS AND ORDER CLERKS *
		434161 .OFF-HUMAN RESOURCES ASSISTANTS, EXCEPT PAYROLL AND
		       .TIMEKEEPING
		434171 .OFF-RECEPTIONISTS AND INFORMATION CLERKS
		434181 .OFF-RESERVATION AND TRANSPORTATION TICKET AGENTS AND 
        	       .TRAVEL CLERKS
		434199 .OFF-INFORMATION AND RECORD CLERKS, ALL OTHER
		435011 .OFF-CARGO AND FREIGHT AGENTS
		435021 .OFF-COURIERS AND MESSENGERS
		435030 .OFF-DISPATCHERS
		435041 .OFF-METER READERS, UTILITIES
		435051 .OFF-POSTAL SERVICE CLERKS
		435052 .OFF-POSTAL SERVICE MAIL CARRIERS
       		435053 .OFF-POSTAL SERVICE MAIL SORTERS, PROCESSORS, AND 
       		       .PROCESSING MACHINE OPERATORS
		435061 .OFF-PRODUCTION, PLANNING, AND EXPEDITING CLERKS
		435071 .OFF-SHIPPING, RECEIVING, AND TRAFFIC CLERKS
		435081 .OFF-STOCK CLERKS AND ORDER FILLERS
		435111 .OFF-WEIGHERS, MEASURERS, CHECKERS, AND SAMPLERS,
		       .RECORDKEEPING
		436010 .OFF-SECRETARIES AND ADMINISTRATIVE ASSISTANTS
		439011 .OFF-COMPUTER OPERATORS
		439021 .OFF-DATA ENTRY KEYERS
		439022 .OFF-WORD PROCESSORS AND TYPISTS
		439041 .OFF-INSURANCE CLAIMS AND POLICY PROCESSING CLERKS
		439051 .OFF-MAIL CLERKS AND MAIL MACHINE OPERATORS, EXCEPT POSTAL
		       .SERVICE
		439061 .OFF-OFFICE CLERKS, GENERAL
		439071 .OFF-OFFICE MACHINE OPERATORS, EXCEPT COMPUTER
		439081 .OFF-PROOFREADERS AND COPY MARKERS
		439111 .OFF-STATISTICAL ASSISTANTS
		439XXX .OFF-MISCELLANEOUS OFFICE AND ADMINISTRATIVE SUPPORT
		       .WORKERS, INCLUDING DESKTOP PUBLISHERS *
		451011 .FFF-FIRST-LINE SUPERVISORS OF FARMING, FISHING, AND
		       .FORESTRY WORKERS
		452011 .FFF-AGRICULTURAL INSPECTORS
		452041 .FFF-GRADERS AND SORTERS, AGRICULTURAL PRODUCTS
		4520XX .FFF-MISCELLANEOUS AGRICULTURAL WORKERS, INCLUDING ANIMAL
		       .BREEDERS *
		453000 .FFF-FISHING AND HUNTING WORKERS
		454011 .FFF-FOREST AND CONSERVATION WORKERS
		454020 .FFF-LOGGING WORKERS
		471011 .CON-FIRST-LINE SUPERVISORS OF CONSTRUCTION TRADES AND
		       .EXTRACTION WORKERS
		472011 .CON-BOILERMAKERS
		472031 .CON-CARPENTERS
		472040 .CON-CARPET, FLOOR, AND TILE INSTALLERS AND FINISHERS
		472050 .CON-CEMENT MASONS, CONCRETE FINISHERS, AND TERRAZZO 
        	       .WORKERS
		472061 .CON-CONSTRUCTION LABORERS
		472071 .CON-PAVING, SURFACING, AND TAMPING EQUIPMENT OPERATORS
		47207X .CON-CONSTRUCTION EQUIPMENT OPERATORS, EXCEPT PAVING,
		       .SURFACING, AND TAMPING EQUIPMENT OPERATORS *
		472080 .CON-DRYWALL INSTALLERS, CEILING TILE INSTALLERS, AND 
        	       .TAPERS
		472111 .CON-ELECTRICIANS
		472121 .CON-GLAZIERS
		472130 .CON-INSULATION WORKERS
		472140 .CON-PAINTERS AND PAPERHANGERS
		472150 .CON-PIPELAYERS, PLUMBERS, PIPEFITTERS, AND STEAMFITTERS
		472161 .CON-PLASTERERS AND STUCCO MASONS
		472181 .CON-ROOFERS
		472211 .CON-SHEET METAL WORKERS
		472221 .CON-STRUCTURAL IRON AND STEEL WORKERS
      		472XXX .CON-BRICKMASONS, BLOCKMASONS, STONEMASONS, AND REINFORCING 
		       .IRON AND REBAR WORKERS
		473010 .CON-HELPERS, CONSTRUCTION TRADES
		474011 .CON-CONSTRUCTION AND BUILDING INSPECTORS
		474021 .CON-ELEVATOR INSTALLERS AND REPAIRERS
		474031 .CON-FENCE ERECTORS
		474041 .CON-HAZARDOUS MATERIALS REMOVAL WORKERS
		474051 .CON-HIGHWAY MAINTENANCE WORKERS
		474061 .CON-RAIL-TRACK LAYING AND MAINTENANCE EQUIPMENT OPERATORS
		47XXXX .CON-MISCELLANEOUS CONSTRUCTION WORKERS, INCLUDING SOLAR
		       .PHOTOVOLTAIC INSTALLERS, SEPTIC TANK SERVICERS AND
		       .SEWER PIPE CLEANERS *
		4750YY .EXT-DERRICK, ROTARY DRILL, AND SERVICE UNIT OPERATORS, AND
		       .ROUSTABOUTS, OIL, GAS, AND MINING *
		475021 .EXT-EARTH DRILLERS, EXCEPT OIL AND GAS
		475031 .EXT-EXPLOSIVES WORKERS, ORDNANCE HANDLING EXPERTS, AND
		       .BLASTERS
		475040 .EXT-MINING MACHINE OPERATORS
		4750XX .EXT-MISCELLANEOUS EXTRACTION WORKERS, INCLUDING ROOF
		       .BOLTERS AND HELPERS *
		491011 .RPR-FIRST-LINE SUPERVISORS OF MECHANICS, INSTALLERS, AND
		       .REPAIRERS
		492011 .RPR-COMPUTER, AUTOMATED TELLER, AND OFFICE MACHINE
		       .REPAIRERS
		492020 .RPR-RADIO AND TELECOMMUNICATIONS EQUIPMENT INSTALLERS AND
		       .REPAIRERS
		492091 .RPR-AVIONICS TECHNICIANS
		492092 .RPR-ELECTRIC MOTOR, POWER TOOL, AND RELATED REPAIRERS
		49209X .RPR-ELECTRICAL AND ELECTRONICS REPAIRERS, TRANSPORTATION
		       .EQUIPMENT, AND INDUSTRIAL AND UTILITY *
		492096 .RPR-ELECTRONIC EQUIPMENT INSTALLERS AND REPAIRERS, MOTOR
		       .VEHICLES
		492097 .RPR-ELECTRONIC HOME ENTERTAINMENT EQUIPMENT INSTALLERS AND
		       .REPAIRERS
		492098 .RPR-SECURITY AND FIRE ALARM SYSTEMS INSTALLERS
		493011 .RPR-AIRCRAFT MECHANICS AND SERVICE TECHNICIANS
		493021 .RPR-AUTOMOTIVE BODY AND RELATED REPAIRERS
		493022 .RPR-AUTOMOTIVE GLASS INSTALLERS AND REPAIRERS
		493023 .RPR-AUTOMOTIVE SERVICE TECHNICIANS AND MECHANICS
		493031 .RPR-BUS AND TRUCK MECHANICS AND DIESEL ENGINE SPECIALISTS
		493040 .RPR-HEAVY VEHICLE AND MOBILE EQUIPMENT SERVICE TECHNICIANS
		       .AND MECHANICS
		493050 .RPR-SMALL ENGINE MECHANICS
		493090 .RPR-MISCELLANEOUS VEHICLE AND MOBILE EQUIPMENT MECHANICS,
		       .INSTALLERS, AND REPAIRERS
		499010 .RPR-CONTROL AND VALVE INSTALLERS AND REPAIRERS
		499021 .RPR-HEATING, AIR CONDITIONING, AND REFRIGERATION MECHANICS
		       .AND INSTALLERS
		499031 .RPR-HOME APPLIANCE REPAIRERS
		49904X .RPR-INDUSTRIAL AND REFRACTORY MACHINERY MECHANICS *
		499043 .RPR-MAINTENANCE WORKERS, MACHINERY
		499044 .RPR-MILLWRIGHTS
		499051 .RPR-ELECTRICAL POWER-LINE INSTALLERS AND REPAIRERS
		499052 .RPR-TELECOMMUNICATIONS LINE INSTALLERS AND REPAIRERS
		499060 .RPR-PRECISION INSTRUMENT AND EQUIPMENT REPAIRERS
		499071 .RPR-MAINTENANCE AND REPAIR WORKERS, GENERAL
		499091 .RPR-COIN, VENDING, AND AMUSEMENT MACHINE SERVICERS AND
		       .REPAIRERS
		499094 .RPR-LOCKSMITHS AND SAFE REPAIRERS		
       		499096 .RPR-RIGGERS
		499098 .RPR-HELPERS--INSTALLATION, MAINTENANCE, AND REPAIR WORKERS
		4990XX .RPR-MISCELLANEOUS INSTALLATION, MAINTENANCE, AND REPAIR
		       .WORKERS, INCLUDING WIND TURBINE SERVICE TECHNICIANS
		511011 .PRD-FIRST-LINE SUPERVISORS OF PRODUCTION AND OPERATING
		       .WORKERS
		512011 .PRD-AIRCRAFT STRUCTURE, SURFACES, RIGGING, AND SYSTEMS
		       .ASSEMBLERS
		512020 .PRD-ELECTRICAL, ELECTRONICS, AND ELECTROMECHANICAL
		       .ASSEMBLERS
		512031 .PRD-ENGINE AND OTHER MACHINE ASSEMBLERS
		512041 .PRD-STRUCTURAL METAL FABRICATORS AND FITTERS
		512090 .PRD-MISCELLANEOUS ASSEMBLERS AND FABRICATORS
		513011 .PRD-BAKERS
		513020 .PRD-BUTCHERS AND OTHER MEAT, POULTRY, AND FISH PROCESSING
		       .WORKERS
		513091 .PRD-FOOD AND TOBACCO ROASTING, BAKING, AND DRYING MACHINE
		       .OPERATORS AND TENDERS
		513092 .PRD-FOOD BATCHMAKERS
		513093 .PRD-FOOD COOKING MACHINE OPERATORS AND TENDERS
		513099 .PRD-FOOD PROCESSING WORKERS, ALL OTHER
		514010 .PRD-COMPUTER CONTROL PROGRAMMERS AND OPERATORS
		514021 .PRD-EXTRUDING AND DRAWING MACHINE SETTERS, OPERATORS, AND
		       .TENDERS, METAL AND PLASTIC
		514022 .PRD-FORGING MACHINE SETTERS, OPERATORS, AND TENDERS, METAL
		       .AND PLASTIC
		514023 .PRD-ROLLING MACHINE SETTERS, OPERATORS, AND TENDERS, METAL
		       .AND PLASTIC
		514030 .PRD-MACHINE TOOL CUTTING SETTERS, OPERATORS, AND 
        	       .TENDERS,METAL AND PLASTIC
		514041 .PRD-MACHINISTS
		514050 .PRD-METAL FURNACE OPERATORS, TENDERS, POURERS, AND CASTERS
		5140XX .PRD-MODEL MAKERS, PATTERNMAKERS, AND MOLDING MACHINE
		       .SETTERS, METAL AND PLASTIC
		514111 .PRD-TOOL AND DIE MAKERS
		514120 .PRD-WELDING, SOLDERING, AND BRAZING WORKERS
		514XXX .PRD-MISCELLANEOUS METAL WORKERS AND PLASTIC WORKERS,
		       .INCLUDING MULTIPLE MACHINE TOOL SETTERS *
		515111 .PRD-PREPRESS TECHNICIANS AND WORKERS
		515112 .PRD-PRINTING PRESS OPERATORS
		515113 .PRD-PRINT BINDING AND FINISHING WORKERS
		516011 .PRD-LAUNDRY AND DRY-CLEANING WORKERS
		516021 .PRD-PRESSERS, TEXTILE, GARMENT, AND RELATED MATERIALS
		516031 .PRD-SEWING MACHINE OPERATORS
		516040 .PRD-SHOE AND LEATHER WORKERS
		516050 .PRD-TAILORS, DRESSMAKERS, AND SEWERS
		51606X .PRD-TEXTILE BLEACHING AND DYEING, AND CUTTING MACHINE
		       .SETTERS, OPERATORS, AND TENDERS *
		516063 .PRD-TEXTILE KNITTING AND WEAVING MACHINE SETTERS,
		       .OPERATORS, AND TENDERS
		516064 .PRD-TEXTILE WINDING, TWISTING, AND DRAWING OUT MACHINE
		       .SETTERS, OPERATORS, AND TENDERS
		516093 .PRD-UPHOLSTERERS
		51609X .PRD-MISCELLANEOUS TEXTILE, APPAREL, AND FURNISHINGS
		       .WORKERS, EXCEPT UPHOLSTERERS *
		517011 .PRD-CABINETMAKERS AND BENCH CARPENTERS
		517021 .PRD-FURNITURE FINISHERS
		517041 .PRD-SAWING MACHINE SETTERS, OPERATORS, AND TENDERS, WOOD
		517042 .PRD-WOODWORKING MACHINE SETTERS, OPERATORS, AND TENDERS,
		       .EXCEPT SAWING
		5170XX .PRD-MISCELLANEOUS WOODWORKERS, INCLUDING MODEL MAKERS AND
		       .PATTERNMAKERS *
		518010 .PRD-POWER PLANT OPERATORS, DISTRIBUTORS, AND DISPATCHERS
		518021 .PRD-STATIONARY ENGINEERS AND BOILER OPERATORS
		518031 .PRD-WATER AND WASTEWATER TREATMENT PLANT AND SYSTEM
		       .OPERATORS
		518090 .PRD-MISCELLANEOUS PLANT AND SYSTEM OPERATORS
		519010 .PRD-CHEMICAL PROCESSING MACHINE SETTERS, OPERATORS, AND
		       .TENDERS
		519020 .PRD-CRUSHING, GRINDING, POLISHING, MIXING, AND BLENDING
		       .WORKERS
		519030 .PRD-CUTTING WORKERS
		519041 .PRD-EXTRUDING, FORMING, PRESSING, AND COMPACTING MACHINE
		       .SETTERS, OPERATORS, AND TENDERS
		519051 .PRD-FURNACE, KILN, OVEN, DRIER, AND KETTLE OPERATORS AND
		       .TENDERS
		519061 .PRD-INSPECTORS, TESTERS, SORTERS, SAMPLERS, AND WEIGHERS
		519071 .PRD-JEWELERS AND PRECIOUS STONE AND METAL WORKERS
		519080 .PRD-MEDICAL, DENTAL, AND OPHTHALMIC LABORATORY TECHNICIANS
		519111 .PRD-PACKAGING AND FILLING MACHINE OPERATORS AND TENDERS
		519120 .PRD-PAINTING WORKERS
		519151 .PRD-PHOTOGRAPHIC PROCESS WORKERS AND PROCESSING MACHINE
		       .OPERATORS
		519191 .PRD-ADHESIVE BONDING MACHINE OPERATORS AND TENDERS
		519194 .PRD-ETCHERS AND ENGRAVERS
		519195 .PRD-MOLDERS, SHAPERS, AND CASTERS, EXCEPT METAL AND 
        	       .PLASTIC
		519196 .PRD-PAPER GOODS MACHINE SETTERS, OPERATORS, AND TENDERS
		519197 .PRD-TIRE BUILDERS
		519198 .PRD-HELPERS-PRODUCTION WORKERS
		5191XX .PRD-MISCELLANEOUS PRODUCTION WORKERS, INCLUDING 
       		       .SEMICONDUCTOR PROCESSORS *
		531000 .TRN-SUPERVISORS OF TRANSPORTATION AND MATERIAL MOVING
		       .WORKERS
		532010 .TRN-AIRCRAFT PILOTS AND FLIGHT ENGINEERS
		532020 .TRN-AIR TRAFFIC CONTROLLERS AND AIRFIELD OPERATIONS
		       .SPECIALISTS
		532031 .TRN-FLIGHT ATTENDANTS
		533011 .TRN-AMBULANCE DRIVERS AND ATTENDANTS, EXCEPT EMERGENCY
	               .MEDICAL TECHNICIANS
		533020 .TRN-BUS DRIVERS
		533030 .TRN-DRIVER/SALES WORKERS AND TRUCK DRIVERS
		533041 .TRN-TAXI DRIVERS AND CHAUFFEURS
		533099 .TRN-MOTOR VEHICLE OPERATORS, ALL OTHER
		534010 .TRN-LOCOMOTIVE ENGINEERS AND OPERATORS
		534031 .TRN-RAILROAD CONDUCTORS AND YARDMASTERS
		5340XX .TRN-SUBWAY, STREETCAR, AND OTHER RAIL TRANSPORTATION
		       .WORKERS *
		5350XX .TRN-SAILORS AND MARINE OILERS, AND SHIP ENGINEERS *
		535020 .TRN-SHIP AND BOAT CAPTAINS AND OPERATORS
		536021 .TRN-PARKING LOT ATTENDANTS
		536031 .TRN-AUTOMOTIVE AND WATERCRAFT SERVICE ATTENDANTS
		536051 .TRN-TRANSPORTATION INSPECTORS
		536061 .PRS-TRANSPORTATION ATTENDANTS, EXCEPT FLIGHT ATTENDANTS
		5360XX .TRN-MISCELLANEOUS TRANSPORTATION WORKERS, INCLUDING BRIDGE
		       .AND LOCK TENDERS AND TRAFFIC TECHNICIANS *
		537021 .TRN-CRANE AND TOWER OPERATORS
		537030 .TRN-DREDGE, EXCAVATING, AND LOADING MACHINE OPERATORS
		5370XX .TRN-CONVEYOR OPERATORS AND TENDERS, AND HOIST AND WINCH
		       .OPERATORS *
		537051 .TRN-INDUSTRIAL TRUCK AND TRACTOR OPERATORS
		537061 .TRN-CLEANERS OF VEHICLES AND EQUIPMENT
		537062 .TRN-LABORERS AND FREIGHT, STOCK, AND MATERIAL MOVERS, HAND
		537063 .TRN-MACHINE FEEDERS AND OFFBEARERS
		537064 .TRN-PACKERS AND PACKAGERS, HAND
		537070 .TRN-PUMPING STATION OPERATORS
		537081 .TRN-REFUSE AND RECYCLABLE MATERIAL COLLECTORS
		5371XX .TRN-MISCELLANEOUS MATERIAL MOVING WORKERS, INCLUDING
		       .MINE SHUTTLE CAR OPERATORS, AND TANK CAR, TRUCK, AND SHIP
		       .LOADERS *
		551010 .MIL-MILITARY OFFICER SPECIAL AND TACTICAL OPERATIONS
		       .LEADERS
		552010 .MIL-FIRST-LINE ENLISTED MILITARY SUPERVISORS
		553010 .MIL-MILITARY ENLISTED TACTICAL OPERATIONS AND AIR/WEAPONS
		       .SPECIALISTS AND CREW MEMBERS
		559830 .MIL-MILITARY, RANK NOT SPECIFIED
		999920 .UNEMPLOYED AND LAST WORKED 5 YEARS AGO OR EARLIER OR NEVER
		       .WORKED

Note: For additional information on NAICS and SOC groupings within major categories visit our website at:  https://www.census.gov/people/io/methodology/indexes.html.	

VPS		2	
	Veteran period of service
		bb .N/A (less than 17 years old, no active duty)
		   .War Times:
		01 .Gulf War: 9/2001 or later
		02 .Gulf War: 9/2001 or later and Gulf War: 8/1990 - 8/2001 
		03 .Gulf War: 9/2001 or later and Gulf War: 8/1990 - 8/2001 and
		   .Vietnam Era 
		04 .Gulf War: 8/1990 - 8/2001
		05 .Gulf War: 8/1990 - 8/2001 and Vietnam Era
		06 .Vietnam Era
		07 .Vietnam Era and Korean War 
		08 .Vietnam Era, Korean War, and WWII 
		09 .Korean War
		10 .Korean War and WWII 
		11 .WWII
		12 .Between Gulf War and Vietnam Era only 
		13 .Between Vietnam Era and Korean War only 
		14 .Between Korean War and World War II only 
		15 .Pre-WWII only 

WAOB		1	
	World area of birth ****
		1 .US state (POB = 001-059)
		2 .PR and US Island Areas (POB = 060-099)
		3 .Latin America (POB = 303,310-399)
		4 .Asia (POB = 158-159,161,200-299)
		5 .Europe (POB = 100-157,160,162-199)
		6 .Africa (POB = 400-499)	
		7 .Northern America (POB = 300-302,304-309)
		8 .Oceania and at Sea (POB = 500-554)

FAGEP		1	
	Age allocation flag
		0 .No
		1 .Yes

FANCP		1	
	Ancestry allocation flag
		0 .No
		1 .Yes

FCITP		1	
	Citizenship allocation flag
		0 .No
		1 .Yes

FCITWP		1	
	Year of naturalization write-in allocation flag
		0 .No
		1 .Yes

FCOWP		1	
	Class of worker allocation flag
		0 .No
		1 .Yes

FDDRSP		1	
	Self-care difficulty allocation flag
		0 .No
		1 .Yes

FDEARP		1	
	Hearing difficulty allocation flag
		0 .No
		1 .Yes

FDEYEP		1	
	Vision difficulty allocation flag
		0 .No
		1 .Yes

FDISP		1
	Disability recode allocation flag
       		0 .No
       		1 .Yes

FDOUTP		1	
	Independent living difficulty allocation flag
		0 .No
		1 .Yes

FDPHYP		1	
	Ambulatory difficulty allocation flag
		0 .No
		1 .Yes

FDRATP		1	
	Disability rating percentage allocation flag
		0 .No
		1 .Yes

FDRATXP		1	
	Disability rating checkbox allocation flag
		0 .No
		1 .Yes

FDREMP		1	
	Cognitive difficulty allocation flag
		0 .No
		1 .Yes

FENGP		1	
	Ability to speak English allocation flag
		0 .No
		1 .Yes

FESRP		1	
	Employment status recode allocation flag
		0 .No
		1 .Yes

FFERP		1	
	Gave birth to child within the past 12 months allocation flag
		0 .No
		1 .Yes

FFODP		1	
	Field of Degree allocation flag
		0 .No
		1 .Yes

FGCLP		1	
	Grandparents living with grandchildren allocation flag
		0 .No
		1 .Yes

FGCMP		1	
	Length of time responsible for grandchildren allocation flag
		0 .No
		1 .Yes

FGCRP		1	
	Grandparents responsible for grandchildren allocation flag
		0 .No
		1 .Yes

FHICOVP		1	
	Health insurance coverage recode allocation flag
		0 .No
		1 .Yes

FHINS1P		1	
	Insurance through a current or former employer or union allocation flag
		0 .No
		1 .Yes

FHINS2P		1	
	Insurance purchased directly from an insurance company allocation flag
		0 .No
		1 .Yes
	
FHINS3C		1	
	Medicare coverage given through the eligibility coverage edit
       		b .Does not have Medicare coverage (HINS3=2)
		0 .No
		1 .Yes

FHINS3P		1	
	Medicare, for people 65 or older, or people with certain disabilities 
	allocation flag
		0 .No
		1 .Yes

FHINS4C		1	
	Medicaid coverage given through the eligibility coverage edit
       		b .Does not have Medicaid coverage (HINS4=2)
		0 .No
		1 .Yes

FHINS4P		1	
	Medicaid, medical assistance, or any kind of government-assistance plan 
	for people with low incomes or a disability allocation flag
		0 .No
		1 .Yes

FHINS5C		1	
	TRICARE coverage given through the eligibility coverage edit
       		b .Does not have TRICARE coverage (HINS5=2)
		0 .No
		1 .Yes

FHINS5P		1	
	TRICARE or other military health care allocation flag
		0 .No
		1 .Yes

FHINS6P		1	
	VA (including those who have ever used or enrolled for VA health care) 
	allocation flag
		0 .No
		1 .Yes

FHINS7P		1	
	Indian health service allocation flag
		0 .No
		1 .Yes

FHISP		1	
	Detailed Hispanic origin allocation flag
		0 .No
		1 .Yes

FINDP		1	
	Industry allocation flag
		0 .No
		1 .Yes

FINTP		1		
	Interest, dividend, and net rental income allocation flag
		0 .No
		1 .Yes

FJWDP		1	
	Time of departure to work allocation flag
		0 .No
		1 .Yes

FJWMNP		1	
	Travel time to work allocation flag
		0 .No
		1 .Yes

FJWRIP		1	
	Vehicle occupancy allocation flag
		0 .No
		1 .Yes

FJWTRP		1	
	Means of transportation to work allocation flag
		0 .No
		1 .Yes

FLANP		1	
	Language spoken at home allocation flag
		0 .No
		1 .Yes

FLANXP		1	
	Language other than English allocation flag
		0 .No
		1 .Yes

FMARP		1	
	Marital status allocation flag
		0 .No
		1 .Yes

FMARHDP		1	
	Divorced in the past 12 months allocation flag
		0 .No
		1 .Yes

FMARHMP		1	
	Married in the past 12 months allocation flag
		0 .No
		1 .Yes

FMARHTP		1	
	Times married allocation flag
		0 .No
		1 .Yes

FMARHWP		1	
	Widowed in the past 12 months allocation flag
		0 .No
		1 .Yes

FMARHYP		1	
	Year last married allocation flag
		0 .No
		1 .Yes

FMIGP		1	
	Mobility status allocation flag
		0 .No
		1 .Yes

FMIGSP		1	
	Migration state allocation flag
		0 .No
		1 .Yes

FMILPP		1	
	Military periods of service allocation flag
		0 .No
		1 .Yes

FMILSP		1	
	Military service allocation flag
		0 .No
		1 .Yes

FOCCP		1	
	Occupation allocation flag
		0 .No
		1 .Yes

FOIP		1	
	All other income allocation flag
		0 .No
		1 .Yes

FPAP		1	
	Public assistance income allocation flag
		0 .No
		1 .Yes

FPERNP		1
	Total person’s earnings allocation flag
		0 .No
		1 .Yes

FPINCP		1
	Total person’s income (signed) allocation flag
		0 .No
		1 .Yes

FPOBP		1	
	Place of birth allocation flag
		0 .No
		1 .Yes

FPOWSP		1	
	Place of work state allocation flag
		0 .No
		1 .Yes

FPRIVCOVP 	 1
	Private health insurance coverage recode allocation flag
		0 .No
		1 .Yes

FPUBCOVP  	1
	Public health coverage recode allocation flag
		0 .No
		1 .Yes

FRACP		1	
	Detailed race allocation flag
		0 .No
		1 .Yes

FRELP		1	
	Relationship allocation flag
		0 .No
		1 .Yes

FRETP		1	
	Retirement income allocation flag
		0 .No
		1 .Yes

FSCHGP		1	
	Grade attending allocation flag
		0 .No
		1 .Yes

FSCHLP		1	
	Highest education allocation flag
		0 .No
		1 .Yes

FSCHP		1	
	School enrollment allocation flag
		0 .No
		1 .Yes

FSEMP		1	
	Self-employment income allocation flag
		0 .No
		1 .Yes

FSEXP		1	
	Sex allocation flag
		0 .No
		1 .Yes

FSSIP		1	
	Supplementary Security Income allocation flag
		0 .No
		1 .Yes

FSSP		1	
	Social Security income allocation flag
		0 .No
		1 .Yes

FWAGP		1	
	Wages and salary income allocation flag
		0 .No
		1 .Yes

FWKHP		1	
	Usual hours worked per week past 12 months allocation flag
		0 .No
		1 .Yes

FWKLP		1	
	Last worked allocation flag
		0 .No
		1 .Yes

FWKWP		1	
	Weeks worked past 12 months allocation flag
		0 .No
		1 .Yes

FWRKP		1	
	Worked last week allocation flag
		0 .No
		1 .Yes

FYOEP		1	
	Year of entry allocation flag
		0 .No
		1 .Yes

PWGTP1		5
	Person's Weight replicate 1
		-9999..09999 .Integer weight of person

PWGTP2		5
	Person's Weight replicate 2
		-9999..09999 .Integer weight of person

PWGTP3		5
	Person's Weight replicate 3
		-9999..09999 .Integer weight of person

PWGTP4		5
	Person's Weight replicate 4
		-9999..09999 .Integer weight of person

PWGTP5		5
	Person's Weight replicate 5
		-9999..09999 .Integer weight of person

PWGTP6		5
	Person's Weight replicate 6
		-9999..09999 .Integer weight of person

PWGTP7		5
	Person's Weight replicate 7
		-9999..09999 .Integer weight of person

PWGTP8		5
	Person's Weight replicate 8
		-9999..09999 .Integer weight of person

PWGTP9		5
	Person's Weight replicate 9
		-9999..09999 .Integer weight of person

PWGTP10		5
	Person's Weight replicate 10
		-9999..09999 .Integer weight of person

PWGTP11		5
	Person's Weight replicate 11
		-9999..09999 .Integer weight of person

PWGTP12		5
	Person's Weight replicate 12
		-9999..09999 .Integer weight of person

PWGTP13		5
	Person's Weight replicate 13
		-9999..09999 .Integer weight of person

PWGTP14		5
	Person's Weight replicate 14
		-9999..09999 .Integer weight of person

PWGTP15		5
	Person's Weight replicate 15
		-9999..09999 .Integer weight of person

PWGTP16		5
	Person's Weight replicate 16
		-9999..09999 .Integer weight of person

PWGTP17		5
	Person's Weight replicate 17
		-9999..09999 .Integer weight of person

PWGTP18		5
	Person's Weight replicate 18
		-9999..09999 .Integer weight of person

PWGTP19		5
	Person's Weight replicate 19
		-9999..09999 .Integer weight of person
	
PWGTP20		5
	Person's Weight replicate 20
		-9999..09999 .Integer weight of person

PWGTP21		5
	Person's Weight replicate 21
		-9999..09999 .Integer weight of person

PWGTP22		5
	Person's Weight replicate 22
		-9999..09999 .Integer weight of person

PWGTP23		5
	Person's Weight replicate 23
		-9999..09999 .Integer weight of person

PWGTP24		5
	Person's Weight replicate 24
		-9999..09999 .Integer weight of person

PWGTP25		5
	Person's Weight replicate 25
		-9999..09999 .Integer weight of person

PWGTP26		5
	Person's Weight replicate 26
		-9999..09999 .Integer weight of person

PWGTP27		5
	Person's Weight replicate 27
		-9999..09999 .Integer weight of person

PWGTP28		5
	Person's Weight replicate 28
		-9999..09999 .Integer weight of person

PWGTP29		5
	Person's Weight replicate 29
		-9999..09999 .Integer weight of person

PWGTP30		5
	Person's Weight replicate 30
		-9999..09999 .Integer weight of person

PWGTP31		5
	Person's Weight replicate 31
		-9999..09999 .Integer weight of person

PWGTP32		5
	Person's Weight replicate 32
		-9999..09999 .Integer weight of person
	
PWGTP33		5
	Person's Weight replicate 33
		-9999..09999 .Integer weight of person

PWGTP34		5
	Person's Weight replicate 34
		-9999..09999 .Integer weight of person

PWGTP35		5
	Person's Weight replicate 35
		-9999..09999 .Integer weight of person

PWGTP36		5
	Person's Weight replicate 36
		-9999..09999 .Integer weight of person

PWGTP37		5
	Person's Weight replicate 37
		-9999..09999 .Integer weight of person

PWGTP38		5
	Person's Weight replicate 38
		-9999..09999 .Integer weight of person

PWGTP39		5
	Person's Weight replicate 39
		-9999..09999 .Integer weight of person

PWGTP40		5
	Person's Weight replicate 40
		-9999..09999 .Integer weight of person

PWGTP41		5
	Person's Weight replicate 41
		-9999..09999 .Integer weight of person

PWGTP42		5
	Person's Weight replicate 42
		-9999..09999 .Integer weight of person

PWGTP43		5
	Person's Weight replicate 43
		-9999..09999 .Integer weight of person

PWGTP44		5
	Person's Weight replicate 44
		-9999..09999 .Integer weight of person

PWGTP45		5
	Person's Weight replicate 45
		-9999..09999 .Integer weight of person

PWGTP46		5
	Person's Weight replicate 46
		-9999..09999 .Integer weight of person

PWGTP47		5
	Person's Weight replicate 47
		-9999..09999 .Integer weight of person

PWGTP48		5
	Person's Weight replicate 48
		-9999..09999 .Integer weight of person

PWGTP49		5
	Person's Weight replicate 49
		-9999..09999 .Integer weight of person

PWGTP50		5
	Person's Weight replicate 50
		-9999..09999 .Integer weight of person

PWGTP51		5
	Person's Weight replicate 51
		-9999..09999 .Integer weight of person

PWGTP52		5
	Person's Weight replicate 52
		-9999..09999 .Integer weight of person

PWGTP53		5
	Person's Weight replicate 53
		-9999..09999 .Integer weight of person

PWGTP54		5
	Person's Weight replicate 54
		-9999..09999 .Integer weight of person

PWGTP55		5
	Person's Weight replicate 55
		-9999..09999 .Integer weight of person

PWGTP56		5
	Person's Weight replicate 56
		-9999..09999 .Integer weight of person

PWGTP57		5
	Person's Weight replicate 57
		-9999..09999 .Integer weight of person
	
PWGTP58		5
	Person's Weight replicate 58
		-9999..09999 .Integer weight of person

PWGTP59		5
	Person's Weight replicate 59
		-9999..09999 .Integer weight of person

PWGTP60		5
	Person's Weight replicate 60
		-9999..09999 .Integer weight of person

PWGTP61		5
	Person's Weight replicate 61
		-9999..09999 .Integer weight of person

PWGTP62		5
	Person's Weight replicate 62
		-9999..09999 .Integer weight of person

PWGTP63		5
	Person's Weight replicate 63
		-9999..09999 .Integer weight of person

PWGTP64		5
	Person's Weight replicate 64
		-9999..09999 .Integer weight of person

PWGTP65		5	
	Person's Weight replicate 65
		-9999..09999 .Integer weight of person

PWGTP66		5
	Person's Weight replicate 66
		-9999..09999 .Integer weight of person

PWGTP67		5
	Person's Weight replicate 67
		-9999..09999 .Integer weight of person

PWGTP68		5
	Person's Weight replicate 68
		-9999..09999 .Integer weight of person

PWGTP69		5
	Person's Weight replicate 69
		-9999..09999 .Integer weight of person

PWGTP70		5
	Person's Weight replicate 70
		-9999..09999 .Integer weight of person

PWGTP71		5
	Person's Weight replicate 71
		-9999..09999 .Integer weight of person

PWGTP72		5
	Person's Weight replicate 72
		-9999..09999 .Integer weight of person

PWGTP73		5
	Person's Weight replicate 73
		-9999..09999 .Integer weight of person

PWGTP74		5
	Person's Weight replicate 74
		-9999..09999 .Integer weight of person

PWGTP75		5
	Person's Weight replicate 75
		-9999..09999 .Integer weight of person

PWGTP76		5
	Person's Weight replicate 76
		-9999..09999 .Integer weight of person

PWGTP77		5
	Person's Weight replicate 77
		-9999..09999 .Integer weight of person

PWGTP78		5
	Person's Weight replicate 78
		-9999..09999 .Integer weight of person

PWGTP79		5
	Person's Weight replicate 79
		-9999..09999 .Integer weight of person

PWGTP80		5
	Person's Weight replicate 80
		-9999..09999 .Integer weight of person

Note for both Industry and Occupation lists in Data Dictionary:

*  In cases where the SOC occupation code ends in X(s) or Y(s), two or more 
SOC occupation codes were aggregated to correspond to a specific Census 
occupation code. In these cases, the Census occupation description is used 
for the SOC occupation title.

** These codes are pseudo codes developed by the Census Bureau and are not
official or equivalent NAICS or SOC codes.

Legend to Identify NAICS Equivalents
	M = Multiple NAICS codes
	P = Part of a NAICS code - NAICS code split between two or more Census	
	codes
	S = Not specified Industry in NAICS sector - Specific to Census codes	
	only
	Z = Exception to NAICS code - Part of NAICS industry but has a unique
	Census code

Note: For additional information on NAICS and SOC groupings
within major categories visit our website at: 
https://www.census.gov/people/io/methodology/indexes.html.

*** Refer to top-coding documentation at:
https://www.census.gov/programs-surveys/acs/technical-documentation/pums/documentation.html 

**** The World Area of Birth (WAOB) in the PUMS dataset is based upon the
Place of Birth (POB) variable in the full dataset, not the Place of Birth
recode created for PUMS (POBP).  POB contains more detailed place of birth
information than POBP.  Therefore, WAOB cannot be recoded from POBP.

Data users should be aware that the area groupings for WAOB are based
primarily upon United States administrative boundaries, not geographic
boundaries.  For example, Guam is grouped into "Puerto Rico and the U.S.
Island Areas," although geographically it is located in Oceania.  Not all
places of birth are listed individually in POBP.  Therefore the user
cannot recreate an exact world area of birth variable from POBP based upon
geographic location of birth.



</pre>



In [ ]:

plt.style.use('fivethirtyeight')

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 10
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 12


a=df[(df['AGEP']>=20) &(df['AGEP']<30)]['INCOME']
b=df[(df['AGEP']>=30) &(df['AGEP']<40)]['INCOME']
c=df[(df['AGEP']>=40) &(df['AGEP']<55)]['INCOME']
sns.distplot(a,bins=100,  hist=False,  label="20 to 30 (Age in years)"  );
sns.distplot(b,bins=100,  hist=False,  label="30 to 40"  );
g=sns.distplot(c,bins=100,  hist=False,  label="40 to 55"  );
#ax.xaxis.set_major_formatter(FuncFormatter(lambda x, _: '{:,}'.format(int(x))))
sns.despine()
g.tick_params(labelsize=12,labelcolor="black")
#ax.xaxis.set_major_formatter(tck.FormatStrFormatter('{:,}'))
#x.xaxis.set_major_formatter(x_format)
plt.xlim(0, 200000)
plt.xticks([50000,100000,150000,200000], ['$50K', '$100K', '$120K', '$150K','$200K'])
plt.title("Income Distribution\nby Age, White Males", fontname='Ubuntu', fontsize=14,
            fontstyle='italic', fontweight='bold')

plt.legend();